In [3]:
from  LibraryFunctions import *

#from fhir.resources.patient import Patient
import pandas as pd 
import numpy as np 
import os 
import re 
import matplotlib.pyplot as plt 
import utils 
import json 
import seaborn as sns

import csv

from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error

from gensim.models import Word2Vec 
from nltk.tokenize import word_tokenize

from sklearn.preprocessing import OneHotEncoder

import warnings 
warnings.filterwarnings("ignore")

from collections import Counter 
import pickle

#Data over sampling , augmentation libraries
#from imblearn.over_sampling import SMOTE #from imblearn.over_sampling import BorderlineSMOTE #from imblearn.over_sampling import ADASYN

import itertools

import seaborn as sns 
from itertools import combinations 
from collections import defaultdict 
from sklearn.metrics import confusion_matrix

In [4]:
import torch
import numpy as np

In [5]:
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

In [6]:
from docx import Document
from docx.shared import Inches
#from docx.enum.text import WD_PA


In [8]:
# Create a new Word document or open an existing one
document = Document()

title = document.add_heading('Disparity  in Asthma Treatment', level=1)
document.add_page_break()
#doc = Document('Report.docx')

In [6]:
#Add a picture to a word document
def addPicture(fileName,figureNum, figureTitle):
    doc.add_picture(fileName, width=Inches(8))
    last_paragraph = doc.paragraphs[-1] 
    last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Add caption
    caption = doc.add_paragraph(f'Figure {figureNum} :{figureTitle}')
    caption.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    

In [10]:
#add a table to a word document
def doctable(data, tabletitle, labels):
    from docx import Document
    import pandas as pd
    #document = Document()

    data = pd.DataFrame(data)  # My input data is in the 2D list form
    document.add_heading(tabletitle)
    
    data = data.round(decimals=1)

    t = document.add_table(data.shape[0]+1, data.shape[1]+1)
    t.style = 'TableGrid'
    # add the header rows.
    for j in range(data.shape[-1]+1):
        t.cell(0,j).text = str(labels[j])
    
    # add the rest of the data frame
    for i in range(data.shape[0]):
        t.cell(i+1,0).text = str(data.index[i])
        for j in range(data.shape[-1]):
            t.cell(i+1,j+1).text = str(data.values[i,j])
    document.add_page_break()
    
                
    

In [11]:
#DataFolder='C:\\Users\\mbagci\\Desktop\\DataSet\\'
DataFolder='../AsthmaData/'
#DataFolder=r'C:\Users\yozturk\Documents\AsthmaEHRData\\'
#DataFolder='/Users/yozturk/yozturk@sdsu.edu - Google Drive/Shared drives/AstmaEHRData/DataSet/'

In [13]:
# FLAGS 

# if the code will be run for the first time Flag should be true
First_Run=True


In [15]:
if First_Run:
    # Get a list of file names in the 'DataFolder' directory
    List_File_Names = os.listdir(DataFolder)

    # Define a list of file types to categorize the data
    file_types = ['MEDICATIONS', 'VITALS', 'DIAGNOSES', 'LABS', 'ENCOUNTERS', 'PROCEDURES', 'HHSFINANCIALDATA', 'DEMOGRAPHICS']

    # Define a list of column names for each file type
    Column_list = [
        ['PATIENTHASHMRN', 'MEDDATEDIFFNO', 'GENERICNAME', 'MEDNAME', 'PHARMACEUTICALCLASS', 'THERAPEUTICCLASS'],
        ['PATIENTHASHMRN', 'VITALDATEDIFFNO', 'UNIT', 'VALUE', 'VALUETYPE', 'VITALNAME'],
        ['PATIENTHASHMRN', 'ENCDATEDIFFNO', 'ENCOUNTERHASHKEY', 'CHRONIC', 'ICD10CODE', 'ICD10DESCRIPTION', 'ISPRIMARY'],
        ['PATIENTHASHMRN', 'LABDATEDIFFNO', 'LAB_NAME', 'ORDERINGDEPTSPECIALTY', 'RESULTNUMERICVALUE', 'RESULTSTATUS', 'RESULTUNIT', 'RESULTVALUE', 'SOURCETYPE'],
        #['PATIENTHASHMRN', 'LABDATEDIFFNO', 'LAB_NAME', 'ORDERINGDEPTSPECIALTY', 'RESULTNUMERICVALUE', 'RESULTSTATUS', 'RESULTUNIT', 'RESULTVALUE', 'SOURCETYPE'],
        ['PATIENTHASHMRN', 'ENCDATEDIFFNO', 'ENCOUNTERHASHKEY', 'ADMISSIONTYPE', 'DEPTLOCATION', 'DEPTSPECIALTY', 'TYPE', 'VISITTYPE', 'ADMITDEPTSPECIALTY'],
        ['PATIENTHASHMRN', 'PROCDATEDIFFNO', 'CODESET', 'PROCEDURECODE', 'PROCEDURENAME', 'QUANTITY', 'TYPE'],
        ['PATIENTHASHMRN', 'MEDIANINCOME', 'ASTHMAHOSPRATE', 'FINANCIALCLASS', 'HHSREGION', 'POPULATION'],
        ['PATIENTHASHMRN', 'AGE', 'ETHNICITY', 'MARITALSTATUS', 'RACE', 'SEX', 'SMOKINGSTATUS', 'STATUS']


    ]

    # Iterate over the file types and process the corresponding files
    for i, type in enumerate(file_types):
        # Filter files based on the current file type
        Files = [x for x in List_File_Names if type in x]

        # Initialize an empty list to store DataFrames
        df_list = []

        # Iterate over the filtered files
        for ii, name in enumerate(Files):
            file = DataFolder + name
            print(file)
            if ii == 0:
                exec('df_' + type + " = read_json_to_df(file)")
            exec('df_' + type + " = read_json_to_df(file)")
            exec('df_' + type + ' = df_' + type + '.reindex(Column_list[i], axis=1)')

            # Find the index of the column with "DATE" in its name
            index_with_date = next((i for i, s in enumerate(Column_list[i]) if "DATE" in s), None)

            # If a column with "DATE" exists, rename it to "DATE_DIF"
            if "DATE" in str(Column_list[i]):
                exec('df_' + type + '.rename(columns={"' + str(Column_list[i][index_with_date]) + '": "DATE_DIF"}, inplace=True)')

            # Append the processed DataFrame to the list
            exec('df_list.append(df_' + type + ')')

        # Concatenate the DataFrames in the list to create a single DataFrame for the current file type
        exec('df_' + type + '= pd.concat(df_list, axis=0)')

    # The code generates DataFrames for each file type and concatenates them.

    #df_MEDICATIONS, df_VITALS, df_DIAGNOSES, df_LABS,df_ENCOUNTERS, df_PROCEDURES, df_HHSFINANCIALDATA, df_DEMOGRAPHICS'

../AsthmaData/EXPORT_MEDICATIONS.json


../AsthmaData/EXPORT_VITALS.json


../AsthmaData/EXPORT_DIAGNOSES.json


../AsthmaData/EXPORT_LABS.json


../AsthmaData/EXPORT_ENCOUNTERS.json


../AsthmaData/EXPORT_PROCEDURES.json


../AsthmaData/EXPORT_HHSFINANCIALDATA.json


../AsthmaData/EXPORT_DEMOGRAPHICS.json




In [11]:
if First_Run:
    df_MEDICATIONS.to_csv(DataFolder+'df_MEDICATIONS.csv')
    df_VITALS.to_csv(DataFolder+'df_VITALS.csv')
    df_DIAGNOSES.to_csv(DataFolder+'df_DIAGNOSES.csv')
    df_LABS.to_csv(DataFolder+'df_LABS.csv')
    df_ENCOUNTERS.to_csv(DataFolder+'df_ENCOUNTERS.csv')
    df_PROCEDURES.to_csv(DataFolder+'df_PROCEDURES.csv')
    df_HHSFINANCIALDATA.to_csv(DataFolder+'df_HHSFINANCIALDATA.csv')
    df_DEMOGRAPHICS.to_csv(DataFolder+'df_DEMOGRAPHICS.csv')
    
else:
    df_MEDICATIONS     =pd.read_csv(DataFolder+'df_MEDICATIONS.csv')
    df_DIAGNOSES       =pd.read_csv(DataFolder+'df_DIAGNOSES.csv')
    df_LABS            =pd.read_csv(DataFolder+'df_LABS.csv')
    df_ENCOUNTERS      =pd.read_csv(DataFolder+'df_ENCOUNTERS.csv')
    df_HHSFINANCIALDATA=pd.read_csv(DataFolder+'df_HHSFINANCIALDATA.csv')
    df_DEMOGRAPHICS    =pd.read_csv(DataFolder+'df_DEMOGRAPHICS.csv')
    df_PROCEDURES      =pd.read_csv(DataFolder+'df_PROCEDURES.csv')
    df_VITALS          =pd.read_csv(DataFolder+'df_VITALS.csv')
    

In [16]:
import os
import pandas as pd
import json

# Define the folder containing the JSON files
DataFolder = "../AsthmaData/"

# List all files in the directory starting with "NOTE" and ending with ".json"
files = [file for file in os.listdir(DataFolder) if file.startswith("NOTE") and file.endswith(".json")]

# Initialize an empty list to store data from all files
data_frames = []

# Iterate through each file and read the content
for file in files:
    location = os.path.join(DataFolder, file)
    with open(location, encoding='utf-8-sig') as f:
        data = f.readlines()
    # Convert each file's content into a DataFrame and append to the list
    df = pd.DataFrame([json.loads(line) for line in data])
    data_frames.append(df)

# Combine all the DataFrames into a single DataFrame
df_NOTES = pd.concat(data_frames, ignore_index=True)

# Display or use the combined DataFrame
print(df_NOTES)

                                            PATIENTHASHMRN  \
0        a22939fe5ab2274e3027fc1f8b06ed62f3bcdc89f6e631...   
1        ed8a8344ef79c6fc4e7185e678cc93dca1159b51524b81...   
2        f297d34271e9bd97bb0a458b77b8dc252b74af99d9bb2e...   
3        415fef0c2ee20d7d639f405da0b929d8fca8b0b6a35e69...   
4        2e6d6dd0868b8ac9b6e406a26ae81b816c6dbdd4e38f94...   
...                                                    ...   
1499338  3b6893088bf024559c43dd4ba426a2d6119f642d29e868...   
1499339  2a14c717d623c597d0dc9e61e2c291cbc8e5c1d721f910...   
1499340  8bfb40ab6a3c70344ab086e267d43f2e8b9071d8599f29...   
1499341  1bf261dd7bc5a61524ad25e3b722256189bf2b1c6bf702...   
1499342  710c7273c5fc41862f7ab141aaa4ee8ff8dab7f90ea236...   

                                          ENCOUNTERHASHKEY  ENCDATEDIFFNO  \
0        8d12f885ad41c16e0eb9ab4f4b92503b39c22e3bad1779...        28208.0   
1        a80a14b7f2512ed64dd1505f3dc769f4465d0e2633b476...        28162.0   
2        9de0fa32f5e77df

In [13]:
df_ENCOUNTERS.columns

Index(['PATIENTHASHMRN', 'DATE_DIF', 'ENCOUNTERHASHKEY', 'ADMISSIONTYPE',
       'DEPTLOCATION', 'DEPTSPECIALTY', 'TYPE', 'VISITTYPE',
       'ADMITDEPTSPECIALTY'],
      dtype='object')

In [21]:
#Merge encounters and Notes 
df_ENCNOTES = pd.merge(df_ENCOUNTERS, df_NOTES, on=['PATIENTHASHMRN', 'ENCOUNTERHASHKEY'], how='inner')


In [22]:
df_ENCNOTES

,PATIENTHASHMRN,DATE_DIF,ENCOUNTERHASHKEY,ADMISSIONTYPE,DEPTLOCATION,DEPTSPECIALTY,TYPE,VISITTYPE,ADMITDEPTSPECIALTY,ENCDATEDIFFNO,NOTE_KEY,DEIDENTIFIED_TEXT
0,5ad92b595e7f465adbbb1c73bc52061d5c8f9b96e5e195...,26928,b1edf80b95f31b45c69a9746d2625debef8da76e225ae5...,ELECTIVE,Scripps Green Hospital,*Unspecified,Hospital Encounter,*Unspecified,*Unspecified,26928.0,9540556,"H&P reviewed., The patient was examined and th..."
1,5ad92b595e7f465adbbb1c73bc52061d5c8f9b96e5e195...,26928,b1edf80b95f31b45c69a9746d2625debef8da76e225ae5...,ELECTIVE,Scripps Green Hospital,*Unspecified,Hospital Encounter,*Unspecified,*Unspecified,26928.0,11747265,VASCULAR SURGERY OPERATIVE NOTE SURGEON: ...
2,5ad92b595e7f465adbbb1c73bc52061d5c8f9b96e5e195...,26928,b1edf80b95f31b45c69a9746d2625debef8da76e225ae5...,ELECTIVE,Scripps Green Hospital,*Unspecified,Hospital Encounter,*Unspecified,*Unspecified,26928.0,10905579,"Pt seen <STATE> ICU <DATE> around 1830.,Pt ext..."
3,5ad92b595e7f465adbbb1c73bc52061d5c8f9b96e5e195...,26928,b1edf80b95f31b45c69a9746d2625debef8da76e225ae5...,ELECTIVE,Scripps Green Hospital,*Unspecified,Hospital Encounter,*Unspecified,*Unspecified,26928.0,11517919,CARDIOLOGY - INPATIENT - CONSULTATION PAT...
4,5ad92b595e7f465adbbb1c73bc52061d5c8f9b96e5e195...,26928,b1edf80b95f31b45c69a9746d2625debef8da76e225ae5...,ELECTIVE,Scripps Green Hospital,*Unspecified,Hospital Encounter,*Unspecified,*Unspecified,26928.0,11509770,"H&P reviewed., The patient was examined and th..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1300439,966b341acf9fa3648e26af6b0dea77a09f127a06642249...,26940,df4d3b09369e71901a26949387b8c8c3e4e884c7cc5260...,*Not Applicable,SC Carmel Valley Loc,Pediatrics,Office Visit,PED WELL CHILD VISIT EXTENDED,None,26940.0,12324659,Chief Complaint(s) Well Child (16yr wcc) ...
1300440,966b341acf9fa3648e26af6b0dea77a09f127a06642249...,26940,df4d3b09369e71901a26949387b8c8c3e4e884c7cc5260...,*Not Applicable,SC Carmel Valley Loc,Pediatrics,Office Visit,PED WELL CHILD VISIT EXTENDED,None,26940.0,12869067,"Staying Healthy <STATE> (Teens, <AGE>-<AGE> ..."
1300441,d5c7e835433495b72c99695354e3af1c2e601e8eb38c9f...,27095,e3b9995c56660a72045e050a8721c6e67cb6939ab7dec4...,*Not Applicable,SC Carmel Valley Loc,Internal Medicine,Telephone,*Unspecified,None,27095.0,6729835,"Rx for Xanax #10 has been sent in for her., P..."
1300442,91b89c048f3f56c3d6562d9dc86c49c852be69e06861ac...,28221,e0dc64559ef1bd08d228c56c66d58e7cba6d236e6651ac...,*Not Applicable,CO Oceanside Mission Ave Loc,Family Medicine,Telephone,*Unspecified,None,28221.0,45034666,"----- Message from <DOCTOR>, MD sent at <DATE>..."


In [23]:
df_HHSFINANCIALDATA['HHSREGION'] = df_HHSFINANCIALDATA['HHSREGION'].replace('0', np.nan)
df_HHSFINANCIALDATA['HHSREGION'] = df_HHSFINANCIALDATA['HHSREGION'].fillna('Other')

df_HHSFINANCIALDATA['HHSREGION'] = df_HHSFINANCIALDATA['HHSREGION'].replace({
    'NORTH COASTAL': 'North Coastal',
    'NORTH CENTRAL': 'North Central',
    'NORTH INLAND': 'North Inland',
    'CENTRAL': 'Central',
    'EAST': 'East',
    'SOUTH': 'South'
})

In [24]:
Counter(df_HHSFINANCIALDATA['HHSREGION'])

Counter({'North Coastal': 8870,
         'North Central': 6262,
         'North Inland': 5944,
         'Other': 3207,
         'Central': 2690,
         'East': 2447,
         'South': 2375})

In [25]:
list(df_ENCOUNTERS.TYPE.unique())

['Hospital Encounter',
 'Patient Message',
 'External Face to Face',
 'Appointment',
 'Telephone',
 'Office Visit',
 'Treatment',
 'E-Visit',
 'Consult',
 'Telemedicine',
 'Patient Outreach',
 'E-Consult',
 'Medication Management',
 'Evaluation']

In [26]:
EncounterTypes = [
    'Hospital Encounter',
    'Patient Message',
    'External Face to Face',
    'Appointment',
    'Telephone',
    'Office Visit',
    #'History',
    'Treatment',
    #'Ancillary Procedure',
    'E-Visit',
    #'Initial Prenatal',
    #'Procedure visit',
    'Consult',
    #'Nurse Triage',
    'Telemedicine',
    #'Prep for Case',
    #'Immunization',
    #'Patient Self-Triage',
    'Patient Outreach',
    #'Surgery',
    #'Ultrasound Visit',
    #'Postpartum Visit',
    #'Nurse Only',
    'E-Consult',
    'Medication Management',
    'Evaluation'
    #'Medicare Advantage Annual Assessment Outreach',
    #'Device Check (In Clinic)',
    #'Lactation Encounter',
    #'SNF Visit',
    #'HCC BPA Review'
]

df_ENCOUNTERS = df_ENCOUNTERS[df_ENCOUNTERS.TYPE.isin(EncounterTypes)]
df_ENCNOTES = df_ENCNOTES[df_ENCNOTES.TYPE.isin(EncounterTypes)]



In [27]:
df_DEMOGRAPHICS['AGE_RANGE'] = 'Other'

# Define function to determine age range
def determine_age_range(age):
    if age < 17:
        return '6-17'
    elif age < 30:
        return '18-30'
    elif age < 40:
        return '31-40'
    elif age < 50:
        return '41-50'
    elif age < 63:
        return '51-63'
    else:
        return '64+'

# Apply the function to create the age range for each row
df_DEMOGRAPHICS['AGE_RANGE'] = df_DEMOGRAPHICS['AGE'].apply(determine_age_range)


In [21]:
p1=document.add_paragraph("The total number of patients:")
l=df_DEMOGRAPHICS.shape[0]
p1.add_run(str(l))

In [30]:
# Define income categories and corresponding labels
income_categories = [
    (0,68200,' Very Low'), 
    (68201, 100000, 'Low'),
    (100000, 140000, 'Median'),
    (140001, 180000, 'High'),
    (180001, float('inf'),'Very High')

]


# Create a new column 'INCOME_CATEGORY' with default value 'Unknown'
df_HHSFINANCIALDATA['INCOME_CATEGORY'] = 'Unknown'

# Define function to categorize median income
def categorize_income(income):
    for min_income, max_income, category in income_categories:
        if min_income <= income <= max_income:
            return category
    return 'Unknown'

# Apply the function to create income categories for each row
df_HHSFINANCIALDATA['INCOME_CATEGORY'] = df_HHSFINANCIALDATA['MEDIANINCOME'].apply(categorize_income)


In [31]:

replacements = {
    'Basophils %': 'Basophils',
    'Eosinophils %': 'Eosinophils',
    'Immunoglobulin IgE':'IgE',
    '_': ' '
}

# Implement the replacments dictionary to lab_name values 
for old, new in replacements.items():
    df_LABS['LAB_NAME'] = df_LABS['LAB_NAME'].replace(old, new, regex=True)

In [24]:
#Merge Demographocs and Financial Data 
df_FINDEMOG = pd.merge(df_HHSFINANCIALDATA, df_DEMOGRAPHICS, on='PATIENTHASHMRN', how='left')

df_FINDEMOG['MEDIANINCOME'] = df_FINDEMOG['MEDIANINCOME'].fillna(0)
df_FINDEMOG['HHSREGION'] = df_FINDEMOG['HHSREGION'].replace('0', 'Other')
df_FINDEMOG['HHSREGION'] = df_FINDEMOG['HHSREGION'].fillna('Other')
df_FINDEMOG['RACE'] = df_FINDEMOG['RACE'].fillna('Other')
df_FINDEMOG['ETHNICITY'] = df_FINDEMOG['ETHNICITY'].fillna('Other')


In [25]:
#Update RACE field by reducing the RACE categories and adding Hispanic  -- Combine Race and Ethinicity in one metric
mapping = {'White':'White', 'Black or African American':'Black', 'Other':'Other', 'Asian':'Asian', 'Unknown':'Other','Decline to Answer':'Other', 'Native Hawaiian or Other Pacific Islander':'Other','American Indian or Alaskan Native':'Other'}
df_FINDEMOG['RACE'] = df_FINDEMOG['RACE'].map(mapping).fillna(df_FINDEMOG['RACE'])
df_FINDEMOG.loc[df_FINDEMOG['ETHNICITY']== 'Hispanic or Latino or Spanish Origin', 'RACE'] = 'Hispanic'
df_FINDEMOG['RACE'] = df_FINDEMOG['RACE'].fillna('Other')


In [26]:
df_DIAGNOSES.columns
prim = df_DIAGNOSES[df_DIAGNOSES.ISPRIMARY == 1]

In [27]:
#Counter(prim.ICD10CODE)

In [28]:
asthma_dia = ['J45.20', 'J45.21', 'J45.30', 'J45.31', 'J45.32', 'J45.40', 'J45.41', 'J45.42', 'J45.50', 'J45.51', 'J45.52']#,'J45.901','J45.902','J45.909','J45.991','J82.83' ]


In [29]:
df_DIAGNOSES.columns

Index(['PATIENTHASHMRN', 'DATE_DIF', 'ENCOUNTERHASHKEY', 'CHRONIC',
       'ICD10CODE', 'ICD10DESCRIPTION', 'ISPRIMARY'],
      dtype='object')

In [30]:
df_DIAGNOSESPrimary = df_DIAGNOSES[df_DIAGNOSES.ISPRIMARY == 1]

In [31]:
df_DIAGNOSES.columns

# Step 1: Group by 'PATIENTHASHMRN' and 'DATE_DIF' and count the unique ICD10CODEs for each group
icd_counts = df_DIAGNOSESPrimary.groupby(['PATIENTHASHMRN', 'DATE_DIF'])['ICD10CODE'].nunique().reset_index()

# Step 2: Filter to find groups with more than two unique ICD10CODEs
patients_with_multiple_icd = icd_counts[icd_counts['ICD10CODE'] > 0]

# Step 3: Merge back with the original DataFrame to get the complete details of these encounters
result = pd.merge(df_DIAGNOSESPrimary, patients_with_multiple_icd[['PATIENTHASHMRN', 'DATE_DIF']], 
                  on=['PATIENTHASHMRN', 'DATE_DIF'], how='inner')

# Display the result
print(patients_with_multiple_icd)

                                           PATIENTHASHMRN  DATE_DIF  ICD10CODE
0       0002986a8d5bb31d7197d99edd2e554d3996c22bd5ffe5...     26975          1
1       0002986a8d5bb31d7197d99edd2e554d3996c22bd5ffe5...     27153          1
2       0002986a8d5bb31d7197d99edd2e554d3996c22bd5ffe5...     27166          1
3       0002986a8d5bb31d7197d99edd2e554d3996c22bd5ffe5...     27249          1
4       0002986a8d5bb31d7197d99edd2e554d3996c22bd5ffe5...     27350          1
...                                                   ...       ...        ...
499526  ffff87ad48c1cda9f6218149a9935c14f532b207f57cfd...     27922          1
499527  ffff87ad48c1cda9f6218149a9935c14f532b207f57cfd...     27936          1
499528  ffff87ad48c1cda9f6218149a9935c14f532b207f57cfd...     27971          1
499529  ffff87ad48c1cda9f6218149a9935c14f532b207f57cfd...     28508          1
499530  ffff87ad48c1cda9f6218149a9935c14f532b207f57cfd...     28611          1

[499531 rows x 3 columns]


In [32]:
df_DIAGNOSESPrimary = df_DIAGNOSESPrimary[df_DIAGNOSESPrimary.ICD10CODE.isin(asthma_dia)]

In [33]:
# Step 4: Combine ICD10CODEs on the same day for each patient as a list (joined as a string)
combined_icd = df_DIAGNOSESPrimary.groupby(['PATIENTHASHMRN', 'DATE_DIF'])['ICD10CODE'].apply(lambda x: ', '.join(x)).reset_index()
combined_icd = combined_icd.rename(columns={'ICD10CODE': 'ICD10CODE_Combined'})

# Merge this back with the original filtered DataFrame to add the combined ICD10CODE list as a new column
result_with_combined = pd.merge(result, combined_icd, on=['PATIENTHASHMRN', 'DATE_DIF'], how='inner')

# Drop duplicate columns and keep only the necessary information
result_with_combined = result_with_combined[['PATIENTHASHMRN', 'DATE_DIF', 'ICD10CODE_Combined']].drop_duplicates()

# Display the resulting DataFrame with the combined ICD10CODE list
result_with_combined.to_excel('investigate.xlsx')

In [34]:
df_DIAGNOSES = df_DIAGNOSESPrimary

In [35]:
# Define a list of asthma diagnoses ICD10 codes
asthma_dia = ['J45.20', 'J45.21', 'J45.30', 'J45.31', 'J45.32', 'J45.40', 'J45.41', 'J45.42', 'J45.50', 'J45.51', 'J45.52']#,'J45.901','J45.902','J45.909','J45.991','J82.83' ]
# Filter df_DIAGNOSES for asthma diagnoses  
visitedForASTHMA = df_DIAGNOSES[df_DIAGNOSES['ICD10CODE'].isin(asthma_dia)].PATIENTHASHMRN.unique()
# Filter to keep only patients that visited the hospotal and received an asthma diagnoses
df_FINDEMOG = df_FINDEMOG[df_FINDEMOG['PATIENTHASHMRN'].isin(visitedForASTHMA)]


#df_FINDEMOG = df_FINDEMOG[df_FINDEMOG['PATIENTHASHMRN'].isin(candidatePatientList)]


p1=document.add_paragraph(" The number of patients visited the hospital for an asthma diagnoses:")
l=df_FINDEMOG.shape[0]
p1.add_run(str(l))

In [36]:
mapping = {'Commercial':'Commercial', 'Managed Care':'Managed Care', 'Medi-Cal':'Medi-Cal', 'Medi-Cal Managed Care':'Medi-Cal','Self-Pay':'Self-Pay', 'Tricare':'Tricare','Medicare':'Medicare', 'Worker\'s Comp':'Other','Medicare HMO/PPO':'Medicare'}
df_FINDEMOG['FINANCIALCLASS'] = df_FINDEMOG['FINANCIALCLASS'].map(mapping).fillna(df_FINDEMOG['FINANCIALCLASS'])
df_FINDEMOG['FINANCIALCLASS'] = df_FINDEMOG['FINANCIALCLASS'].fillna('Other')

In [38]:
Counter(df_DIAGNOSES.ISPRIMARY)

Counter({1: 17567})

In [39]:

# Filter df_DIAGNOSES for asthma diagnoses and select relevant columns
df_DIAGASTHMA = df_DIAGNOSES[df_DIAGNOSES['ICD10CODE'].isin(asthma_dia)]

# Extract the last two characters from the ICD10CODE and convert to int as Plot_sev
df_DIAGASTHMA["ICDSeverity"] = df_DIAGASTHMA['ICD10CODE'].dropna().str[-2:].astype(int)

In [40]:
df_DIAGASTHMA.ICD10CODE

1056       J45.31
1623       J45.52
3009       J45.20
3437       J45.41
3439       J45.41
            ...  
1776051    J45.41
1776052    J45.41
1776056    J45.41
1776363    J45.41
1778149    J45.41
Name: ICD10CODE, Length: 17567, dtype: object

In [41]:
dermatologyDIAGNOSES = df_DIAGNOSES[df_DIAGNOSES['ICD10CODE'].str.startswith('L20', na=False)]
DermatologyPatients = dermatologyDIAGNOSES.PATIENTHASHMRN.unique()



nasalPolipsDIAGNOSES = df_DIAGNOSES[df_DIAGNOSES['ICD10CODE'].str.startswith('J33', na=False)]
NasalPolipsPatients = nasalPolipsDIAGNOSES.PATIENTHASHMRN.unique()


In [42]:
DermatologyPatients = list(DermatologyPatients)

In [43]:
NasalPolipsPatients = list(NasalPolipsPatients)


In [44]:
# Get unique patients with J45 and L20 conditions
j45_patients = set(df_DIAGNOSES[df_DIAGNOSES['ICD10CODE'].str.startswith('J45', na=False)]['PATIENTHASHMRN'])
l20_patients = set(df_DIAGNOSES[df_DIAGNOSES['ICD10CODE'].str.startswith('L20', na=False)]['PATIENTHASHMRN'])

# Find the intersection of patients with both J45 and L20 conditions
patients_with_both = j45_patients.intersection(l20_patients)

L45L20Diagnoses = df_DIAGNOSES[df_DIAGNOSES['ICD10CODE'].str.startswith(('L20', 'J45'), na=False)]


In [45]:
L45L20Diagnoses.columns

Index(['PATIENTHASHMRN', 'DATE_DIF', 'ENCOUNTERHASHKEY', 'CHRONIC',
       'ICD10CODE', 'ICD10DESCRIPTION', 'ISPRIMARY'],
      dtype='object')

In [46]:
#  Identify duplicates based on Patient MRN, Date and ICD10code
duplicates = df_DIAGASTHMA.duplicated(subset=['PATIENTHASHMRN', 'DATE_DIF','ICD10CODE'], keep=False)
# Filter out the duplicates
duplicate_records = df_DIAGASTHMA[duplicates]
duplicate_records_to_drop = duplicate_records[duplicate_records['CHRONIC'] == 0.0]
#Keep records with Chronic condition 
#df_DIAGASTHMA = pd.concat([df_DIAGASTHMA, duplicate_records_to_drop]).drop_duplicates(keep=False)
df_DIAGASTHMA = df_DIAGASTHMA.drop(duplicate_records_to_drop.index)



In [47]:
duplicates = df_DIAGASTHMA.duplicated(subset=['PATIENTHASHMRN', 'DATE_DIF'], keep=False)
# Filter out the duplicates
duplicate_records = df_DIAGASTHMA[duplicates]

# Sort by 'ICDSeverity' in descending order and drop duplicates
duplicate_records_sorted = duplicate_records.sort_values(by='ICDSeverity', ascending=False)
duplicate_records_kept = duplicate_records_sorted.drop_duplicates(subset=['PATIENTHASHMRN', 'DATE_DIF'])

# Remove these duplicates from the original DataFrame
df_cleaned = pd.concat([df_DIAGASTHMA.drop(duplicate_records.index), duplicate_records_kept]).sort_index()
df_DIAGASTHMA = pd.concat([df_cleaned, duplicate_records_kept]).sort_index()


In [48]:
document.add_paragraph("All diagnosis not relevant to asthma removed \n")
document.add_paragraph("Duplicate diagnosis on the same day whether the same ICD10 code or not removed \n")

#Combine diagnoses with Procedures
df_DIAGPROC = pd.merge(df_DIAGASTHMA,df_PROCEDURES, on=['PATIENTHASHMRN','DATE_DIF'],how ='left')
#include only those raws that are related to diagnoses

In [49]:
df_DIAGPROC

,PATIENTHASHMRN,DATE_DIF,ENCOUNTERHASHKEY,CHRONIC,ICD10CODE,ICD10DESCRIPTION,ISPRIMARY,ICDSeverity,CODESET,PROCEDURECODE,PROCEDURENAME,QUANTITY,TYPE
0,8f5fca5d4040ad07dc16a08928cc7b249a9ead8193c1f1...,28529,a794546888d5ad5a6469fb08c064b3a0ba6509e6508979...,0.0,J45.31,Mild persistent asthma with (acute) exacerbation,1,31,CPT(R),99213,Established patient office or other outpatient...,NaN,Level of Service
1,8f5fca5d4040ad07dc16a08928cc7b249a9ead8193c1f1...,28529,a794546888d5ad5a6469fb08c064b3a0ba6509e6508979...,0.0,J45.31,Mild persistent asthma with (acute) exacerbation,1,31,HCPCS,J7611,Albuterol non-comp con,1.0,Outgoing Claim Procedure
2,8f5fca5d4040ad07dc16a08928cc7b249a9ead8193c1f1...,28529,a794546888d5ad5a6469fb08c064b3a0ba6509e6508979...,0.0,J45.31,Mild persistent asthma with (acute) exacerbation,1,31,CPT(R),94640,Respiratory inhaled pressure or nonpressure tr...,1.0,Outgoing Claim Procedure
3,8f5fca5d4040ad07dc16a08928cc7b249a9ead8193c1f1...,28529,a794546888d5ad5a6469fb08c064b3a0ba6509e6508979...,0.0,J45.31,Mild persistent asthma with (acute) exacerbation,1,31,Custom,71046,X-ray chest 2 views with pa/ap and lateral,1.0,General Procedure
4,8f5fca5d4040ad07dc16a08928cc7b249a9ead8193c1f1...,28529,a794546888d5ad5a6469fb08c064b3a0ba6509e6508979...,0.0,J45.31,Mild persistent asthma with (acute) exacerbation,1,31,CPT(R),99203,"New patient office or other outpatient visit, ...",1.0,Outgoing Claim Procedure
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89252,43050f57d9841220a3b1b95119e787cbd4dc129530aeff...,28142,73c3a007231ef71cb6d75a0f373347f9fe7a50cd0150dc...,0.0,J45.41,Moderate persistent asthma with (acute) exacer...,1,41,CPT(R),99214,Established patient office or other outpatient...,1.0,Outgoing Claim Procedure
89253,43050f57d9841220a3b1b95119e787cbd4dc129530aeff...,28142,73c3a007231ef71cb6d75a0f373347f9fe7a50cd0150dc...,0.0,J45.41,Moderate persistent asthma with (acute) exacer...,1,41,CPT(R),99214,Established patient office or other outpatient...,NaN,Level of Service
89254,c4b9a6278fadfca12c014467c621870ba01a4a238b4b1e...,28033,be1d4508305eca4f93b92675ff17e9c457304ea79226b0...,0.0,J45.41,Moderate persistent asthma with (acute) exacer...,1,41,NaN,NaN,NaN,NaN,NaN
89255,f94907183a2709889f8e5500eb7bd15885f0742c1b1667...,27668,2d6a7b3bd79d5091dd17b0ca90fb0bf50e6671c1721880...,0.0,J45.41,Moderate persistent asthma with (acute) exacer...,1,41,CPT(R),99214,Established patient office or other outpatient...,1.0,External Claim Procedure


In [50]:
# Group by 'PATIENTHASHMRN', 'DATE_DIF', and 'ICD10CODE'
# We would like to group the procedures related to an individual diagnosis on a particular day
grouped = df_DIAGPROC.groupby(['PATIENTHASHMRN', 'DATE_DIF', 'ICD10CODE'])

# Aggregate procedure codes into a list for each diagnoses 
aggregated = grouped.agg(ProcedureList=('PROCEDURECODE', lambda x: list(x)),
                         ProcedureNames=('PROCEDURENAME', lambda x: list(x)),
                         EncounterCode=('ENCOUNTERHASHKEY', lambda x: x.iloc[0] if len(x) > 0 else None),
                         ProcedureTypes =('TYPE',  lambda x: list(x)),
                         ProcedureCount=('PROCEDURECODE', 'count')).reset_index()

aggregated.columns = ['PATIENTHASHMRN', 'DATE_DIF', 'ICD10CODE', 'ProcedureList', 'ProcedureNames','EncounterCode','ProcedureTypes','ProcedureCount']

df_DIAGPROCunique = df_DIAGPROC.drop_duplicates(subset=['PATIENTHASHMRN', 'DATE_DIF', 'ICD10CODE'])


df_DIAGPROC = pd.merge(df_DIAGPROCunique, aggregated, on=['PATIENTHASHMRN', 'DATE_DIF', 'ICD10CODE'], how='left')
# df_DIAGPROC now contains new columns with the list of procedure codes, Procedure names and Procedure types for each patient


In [51]:
df_DIAGPROC.columns

Index(['PATIENTHASHMRN', 'DATE_DIF', 'ENCOUNTERHASHKEY', 'CHRONIC',
       'ICD10CODE', 'ICD10DESCRIPTION', 'ISPRIMARY', 'ICDSeverity', 'CODESET',
       'PROCEDURECODE', 'PROCEDURENAME', 'QUANTITY', 'TYPE', 'ProcedureList',
       'ProcedureNames', 'EncounterCode', 'ProcedureTypes', 'ProcedureCount'],
      dtype='object')

In [52]:
#Drop unneeded columns since they are replicated in the lists 
df_DIAGPROC = df_DIAGPROC.drop(['PROCEDURECODE','CODESET','QUANTITY','ENCOUNTERHASHKEY','PROCEDURENAME','TYPE'], axis=1)


In [53]:
# Drop duplicates based on 'PATIENTHASHMRN', 'DATE_DIF', and 'ICD10CODE'
df_DIAGPROC= df_DIAGPROC.drop_duplicates(subset=['PATIENTHASHMRN','DATE_DIF', 'ICD10CODE'])


In [54]:
df_DIAGPROC['CHRONIC'] = df_DIAGPROC['CHRONIC'].fillna(0.0)
#df_DIAGPROC = df_DIAGPROC.dropna()


In [55]:
#locate entried with NaN values 
nan_df_DIAGPROC = df_DIAGPROC[df_DIAGPROC.isnull().any(axis=1)]


In [56]:
df_DIAGPROC = df_DIAGPROC.rename(columns={'EncounterCode': 'ENCOUNTERHASHKEY'})


In [57]:
df_DIAGPROC_FINDEMOG = pd.merge(df_DIAGPROC, df_FINDEMOG, on='PATIENTHASHMRN', how='left')



In [58]:
df_DIAGPROC_FINDEMOG_ENC = pd.merge(df_DIAGPROC_FINDEMOG, df_ENCOUNTERS, on='ENCOUNTERHASHKEY', how='inner')
df_DIAGPROC_FINDEMOG_ENCNOTE = pd.merge(df_DIAGPROC_FINDEMOG, df_ENCNOTES, on='ENCOUNTERHASHKEY', how='inner')


In [59]:
# Remove columns starting with "Unnamed"
df_DIAGPROC_FINDEMOG_ENC = df_DIAGPROC_FINDEMOG_ENC.loc[:, ~df_DIAGPROC_FINDEMOG_ENC.columns.str.startswith('Unnamed')]

# Remove columns ending with "_x"
df_DIAGPROC_FINDEMOG_ENC = df_DIAGPROC_FINDEMOG_ENC.loc[:, ~df_DIAGPROC_FINDEMOG_ENC.columns.str.endswith('_x')]

# Rename columns ending with "_y" by removing the "_y"
df_DIAGPROC_FINDEMOG_ENC.columns = df_DIAGPROC_FINDEMOG_ENC.columns.str.replace('_y$', '', regex=True)

# Display the updated DataFrame columns
df_DIAGPROC_FINDEMOG_ENC.columns

Index(['CHRONIC', 'ICD10CODE', 'ICD10DESCRIPTION', 'ISPRIMARY', 'ICDSeverity',
       'ProcedureList', 'ProcedureNames', 'ENCOUNTERHASHKEY', 'ProcedureTypes',
       'ProcedureCount', 'MEDIANINCOME', 'ASTHMAHOSPRATE', 'FINANCIALCLASS',
       'HHSREGION', 'POPULATION', 'INCOME_CATEGORY', 'AGE', 'ETHNICITY',
       'MARITALSTATUS', 'RACE', 'SEX', 'SMOKINGSTATUS', 'STATUS', 'AGE_RANGE',
       'PATIENTHASHMRN', 'DATE_DIF', 'ADMISSIONTYPE', 'DEPTLOCATION',
       'DEPTSPECIALTY', 'TYPE', 'VISITTYPE', 'ADMITDEPTSPECIALTY'],
      dtype='object')

In [61]:
df_DIAGPROC_FINDEMOG_ENCNOTE

,PATIENTHASHMRN_x,DATE_DIF_x,CHRONIC,ICD10CODE,ICD10DESCRIPTION,ISPRIMARY,ICDSeverity,ProcedureList,ProcedureNames,ENCOUNTERHASHKEY,...,DATE_DIF_y,ADMISSIONTYPE,DEPTLOCATION,DEPTSPECIALTY,TYPE,VISITTYPE,ADMITDEPTSPECIALTY,ENCDATEDIFFNO,NOTE_KEY,DEIDENTIFIED_TEXT
0,da41d397381cad64f4a0445289a32ce5310415d32d4220...,27738,0.0,J45.52,Severe persistent asthma with status asthmaticus,1,52,"[94729, PFT40, 94726, 94060, 99214, 99214, 307...","[Measurement of lung diffusing capacity, SPIRO...",f032ea10aef8698e74eabfc6f9391ed8a456507a195fbe...,...,27738,*Not Applicable,SC AMP Loc,Pulmonology,Office Visit,OFFICE VISIT,None,27738.0,32912778,Subjective Chief Complaint(s) Follow-up ...
1,da41d397381cad64f4a0445289a32ce5310415d32d4220...,27738,0.0,J45.52,Severe persistent asthma with status asthmaticus,1,52,"[94729, PFT40, 94726, 94060, 99214, 99214, 307...","[Measurement of lung diffusing capacity, SPIRO...",f032ea10aef8698e74eabfc6f9391ed8a456507a195fbe...,...,27738,*Not Applicable,SC AMP Loc,Pulmonology,Office Visit,OFFICE VISIT,None,27738.0,32912778,Subjective Chief Complaint(s) Follow-up ...
2,ac407917e23e91644223c6de3950bdfb5e630a71416084...,27173,0.0,J45.41,Moderate persistent asthma with (acute) exacer...,1,41,[nan],[nan],9ab627cb193ad1be25353b3b6394a180639f5ed3b55f51...,...,27173,*Not Applicable,SC Encinitas Loc,Family Medicine,Patient Message,*Unspecified,None,27173.0,3777868,"From: <PATIENT> To: <DOCTOR>, NP Sent: <DATE..."
3,25ceef6b66b1bb72bcda5c7b2ad9c28f8d83e10bebeec3...,27521,0.0,J45.52,Severe persistent asthma with status asthmaticus,1,52,"[99213, 99213, 3008F]",[Established patient office or other outpatien...,192f8417008d1419a1d435dbbdb97533d3b07ebd963f11...,...,27521,*Not Applicable,CO Hillcrest Loc,Family Medicine,Office Visit,OFFICE VISIT,None,27521.0,28043019,Subjective Chief Complaint(s) Cough and S...
4,da41d397381cad64f4a0445289a32ce5310415d32d4220...,27864,0.0,J45.52,Severe persistent asthma with status asthmaticus,1,52,"[LAB17, 71045, LAB10659, 90121, 71045, 71045, ...","[COMPREHENSIVE METABOLIC PANEL, X-ray portable...",5281054df8856c60c3af52d8d1aa985c00f7ff382f6922...,...,27864,*Not Applicable,SC AMP Loc,Pulmonology,Telemedicine,VIDEO VISIT,None,27864.0,36050256,Subjective Chief Complaint(s) Follow-up ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28383,f94907183a2709889f8e5500eb7bd15885f0742c1b1667...,27668,0.0,J45.41,Moderate persistent asthma with (acute) exacer...,1,41,"[99214, 99214]",[Established patient office or other outpatien...,2d6a7b3bd79d5091dd17b0ca90fb0bf50e6671c1721880...,...,27668,*Not Applicable,CO Hillcrest Loc,Family Medicine,Telemedicine,VIDEO VISIT,None,27668.0,31215874,"As discussed during your visit, your asthma me..."
28384,f94907183a2709889f8e5500eb7bd15885f0742c1b1667...,27668,0.0,J45.41,Moderate persistent asthma with (acute) exacer...,1,41,"[99214, 99214]",[Established patient office or other outpatien...,2d6a7b3bd79d5091dd17b0ca90fb0bf50e6671c1721880...,...,27668,*Not Applicable,CO Hillcrest Loc,Family Medicine,Telemedicine,VIDEO VISIT,None,27668.0,31352227,Patient would likely benefit from hormone test...
28385,f94907183a2709889f8e5500eb7bd15885f0742c1b1667...,27668,0.0,J45.41,Moderate persistent asthma with (acute) exacer...,1,41,"[99214, 99214]",[Established patient office or other outpatien...,2d6a7b3bd79d5091dd17b0ca90fb0bf50e6671c1721880...,...,27668,*Not Applicable,CO Hillcrest Loc,Family Medicine,Telemedicine,VIDEO VISIT,None,27668.0,31341840,Provider is comfortable refilling Concerta for...
28386,f94907183a2709889f8e5500eb7bd15885f0742c1b1667...,27668,0.0,J45.41,Moderate persistent asthma with (acute) exacer...,1,41,"[99214, 99214]",[Established patient office or other outpatien...,2d6a7b3bd79d5091dd17b0ca90fb0bf50e6671c1721880...,...,27668,*Not Applicable,CO Hillcrest Loc,Family Medicine,Telemedicine,VIDEO VISIT,None,27668.0,31346587,Patient reports last Pap <DATE> at outside pro...


In [ ]:
#Remove all diagnises  by departments unrelated such as orthopedy 
Counter(df_DIAGPROC_FINDEMOG_ENC['DEPTSPECIALTY'])
#checkcodes = df_DIAGPROC_FINDEMOG_ENC[df_DIAGPROC_FINDEMOG_ENC['DEPTSPECIALTY'] =='Orthopaedic Surgery']

In [ ]:
DepartmentRelevant = [
    'Family Medicine',
    'Internal Medicine',
    #'Pediatrics',
    'Allergy and Immunology',
    'Pulmonology',
    'Urgent Care',
    'Emergency Medicine']
    #'Sleep Medicine']

df_DIAGPROC_FINDEMOG_ENC = df_DIAGPROC_FINDEMOG_ENC[df_DIAGPROC_FINDEMOG_ENC['DEPTSPECIALTY'].isin(DepartmentRelevant)]

#df_DIAGPROC_FINDEMOG_ENC.loc[~df_DIAGPROC_FINDEMOG_ENC['DEPTSPECIALTY'].isin(DepartmentRelevant), 'DEPTSPECIALTY'] = 'Other'
df_DIAGPROC_FINDEMOG_ENCFiltered = df_DIAGPROC_FINDEMOG_ENC#[df_DIAGPROC_FINDEMOG_ENC['DEPTSPECIALTY'].isin(DepartmentRelevant)]

In [ ]:
df_DIAGPROC_FINDEMOG_ENC.columns


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from docx import Document
from docx.shared import Inches

# Step 1: Create a Count column if it doesn't already exist
df_summary = df_DIAGPROC_FINDEMOG_ENC.groupby(['DEPTSPECIALTY', 'ICD10CODE']).size().reset_index(name='Count')

# Step 2: Calculate the total number of patients for each ICD10 code
total_counts_by_code = df_summary.groupby('ICD10CODE')['Count'].transform('sum')

# Step 3: Calculate the percentage of patients with each ICD10 code visiting each department specialty
df_summary['Percentage'] = (df_summary['Count'] / total_counts_by_code) * 100

# Step 4: Pivot the summary table to prepare for the heatmap
heatmap_data = df_summary.pivot(index='DEPTSPECIALTY', columns='ICD10CODE', values='Count').fillna(0)
percentage_data = df_summary.pivot(index='DEPTSPECIALTY', columns='ICD10CODE', values='Percentage').fillna(0)

# Step 5: Create a formatted annotation DataFrame showing both percentage and count
annot_data = heatmap_data.astype(int).astype(str) + "\n(" + percentage_data.round(2).astype(str) + "%)"

# Step 6: Create the heatmap with combined annotations
plt.figure(figsize=(12, 8))
ax=sns.heatmap(
    heatmap_data,
    annot=annot_data,  # Custom annotations
    fmt='',  # Allows string-based annotations
    cmap='Blues',
    cbar_kws={'label': 'Number of Patients'},
    annot_kws={'fontsize': 12}  # Set annotation font size
)

# Customize the color bar label font size
cbar = ax.collections[0].colorbar  # Access the color bar
cbar.ax.yaxis.label.set_size(12)  # Set the font size of the color bar label
cbar.ax.tick_params(labelsize=12)  # Adjust font size for color bar ticks


# Customize font sizes for labels and title
plt.xlabel('ICD10Code', fontsize=12)
plt.ylabel('Department Specialty', fontsize=12)
plt.xticks(rotation=90, fontsize=12)  # Rotate x-axis labels and set font size
plt.yticks(rotation=0,fontsize=12)  # Set y-axis label font size
plt.title('Patients with Each ICD10 Code Visiting Each Department Specialty (Counts and Percentages)', fontsize=16)

# Save the plot as an image
plt.savefig('temp.png', bbox_inches='tight')

# Step 8: Add the heatmap to a Word document
#document = Document()
document.add_heading('Patient Data Analysis', level=1)
document.add_picture('temp.png', width=Inches(5))
document.add_paragraph('Patients with Each ICD10 Code Visiting Each Department Specialty (Counts and Percentages)')

# Save the Word document#
#document.save('Patient_Analysis_Report.docx')

# Step 9: Show the plot
plt.show()

In [ ]:
df_DIAGPROC_FINDEMOG_ENC.columns

In [ ]:
import pandas as pd

# Assuming df_DIAGPROC_FINDEMOG_ENCFiltered is your DataFrame
# and 'PATIENTHASHMRN_x' is the patient identifier, 'DATE_DIF_x' is the date of diagnosis, and 'ICD10CODE' is the diagnosis code.

# Step 1: Group by patient ID and date, and count the number of diagnoses
diagnosis_count = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby(['PATIENTHASHMRN', 'DATE_DIF']).size().reset_index(name='DiagnosisCount')

# Step 2: Filter for records where the DiagnosisCount is greater than 1
multiple_diagnoses = diagnosis_count[diagnosis_count['DiagnosisCount'] > 1]

# Step 3: Merge with the original DataFrame to get full details of these diagnoses
patients_with_multiple_diagnoses = df_DIAGPROC_FINDEMOG_ENCFiltered.merge(
    multiple_diagnoses[['PATIENTHASHMRN', 'DATE_DIF']],
    on=['PATIENTHASHMRN', 'DATE_DIF'],
    how='inner'
)

# Display the result
patients_with_multiple_diagnoses

In [ ]:
# Step 1: Group by 'ICD10CODE' and 'ICD10DESCRIPTION', counting unique patients
patient_counts = df_DIAGPROC_FINDEMOG_ENC.groupby(['ICD10CODE', 'ICD10DESCRIPTION'])['PATIENTHASHMRN'].count().reset_index(name='Number_of_Patients')

# Step 2: Display the resulting table
print(patient_counts)  # You can replace this with other display methods like patient_counts.head() or exporting to a file
patient_counts.to_csv('ICD10_patient_counts.csv', index=False)

document.add_paragraph("Number of Visits, ICD 10 codes and Department Specialities")
doctable(patient_counts," Department Specialities and Patients Seen",['index','ICD10 Code ','ICD10 Description', 'Number of Visits'])


In [ ]:
df_DIAGPROC_FINDEMOG_ENCFiltered.columns

In [ ]:
# Get unique patients
unique_patients = df_DIAGPROC_FINDEMOG_ENCFiltered['PATIENTHASHMRN'].unique()

# If you wish to see the number of unique patients
number_of_unique_patients = len(unique_patients)

print(f"Number of unique patients: {number_of_unique_patients}")
p=document.add_paragraph("The number of unique patients with a diagnosis by a relevant department is :")
p.add_run(str(number_of_unique_patients))
p.add_run("The patients that received an asthma related visit to the doctor --")
# If you want to see the list of unique patient IDs, you can print 'unique_patients' but be cautious with large datasets
# print(unique_patients)


In [ ]:
#copy the data back from the master
#df_DIAGPROC_FINDEMOG_ENCFiltered = df_DIAGPROC_FINDEMOG_ENCFiltered_copy

#replace ICD10 codes for Astma sub penotypes with do not care
# Define the replacement dictionary

replacement_dict = {
    'J45.20': 'J45.2x', 'J45.21': 'J45.2x',
    'J45.30': 'J45.3x', 'J45.31': 'J45.3x', 'J45.32': 'J45.3x',
    'J45.40': 'J45.4x', 'J45.41': 'J45.4x', 'J45.42': 'J45.4x',
    'J45.50': 'J45.5x', 'J45.51': 'J45.5x', 'J45.52': 'J45.5x'
}

# Apply the replacements
df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'] = df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'].replace(replacement_dict)

num_unique_patients = df_DIAGPROC_FINDEMOG_ENCFiltered['PATIENTHASHMRN'].count()
patient_visits = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby('PATIENTHASHMRN').size().reset_index(name='Visit_Count')

icd_codes_list = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby('PATIENTHASHMRN')['ICD10CODE'].apply(list).reset_index(name='ICD10CODEs')
dept_specialty_list = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby(['PATIENTHASHMRN', 'ICD10CODE'])['DEPTSPECIALTY'].apply(list).reset_index(name='DEPTSPECIALTYs')

patient_visits = pd.merge(patient_visits, icd_codes_list, on='PATIENTHASHMRN', how='left')
patient_visits_department = pd.merge(patient_visits, dept_specialty_list, on='PATIENTHASHMRN', how='left')

patient_visits_department=patient_visits_department.drop('ICD10CODE',axis=1)

#print("Summary DataFrame:")
#print(patient_visits)

# Merging both DataFrames on 'PATIENTHASHMRN'
merged_df = pd.merge(icd_codes_list, dept_specialty_list, on='PATIENTHASHMRN', how='left')

# Creating a pivot table
pivot_table = pd.pivot_table(df_DIAGPROC_FINDEMOG_ENCFiltered, index='DEPTSPECIALTY', columns='ICD10CODE', aggfunc='size', fill_value=0)

document.add_paragraph("Matrix of DEPTSPECIALTY versus ICD10CODES:")
print(pivot_table)
doctable(pivot_table," Department Specialities and Patients Seen",['Department Speciality','J45.2x', 'J45.3x', 'J45.4x', 'J45.5x'])


In [ ]:
Counter(df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'])

In [ ]:

#Drop duplicates to generate statistics  for these patients  only for demographics purposes
df_DIAGPROC_FINDEMOG_ENCFiltered_Unique= df_DIAGPROC_FINDEMOG_ENCFiltered.drop_duplicates(subset=['PATIENTHASHMRN'])

# Count the number of patients for each ethnicity
ethnicity_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique['RACE'].value_counts()

# Calculate the percentage of patients for each ethnicity
ethnicity_percentage = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique['RACE'].value_counts(normalize=True) * 100

# Create a summary DataFrame to neatly display counts and percentages
ethnicity_summary = pd.DataFrame({
    'Number of Patients': ethnicity_counts,
    'Percentage': ethnicity_percentage
})

# Optional: format the 'Percentage' column to round to 2 decimal places and add the '%' sign
ethnicity_summary['Percentage'] = ethnicity_summary['Percentage'].map('{:.2f}%'.format)

# Display the summary
print(ethnicity_summary)
doctable(ethnicity_summary,"Ethnicity Statistics",['Ethnicity','Number of Patients', 'Percentage'])
                                                                                           
                                                                                           
# Count the number of patients for each ethnicity
race_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique['RACE'].value_counts()

# Calculate the percentage of patients for each ethnicity
race_percentage = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique['RACE'].value_counts(normalize=True) * 100

# Create a summary DataFrame to neatly display counts and percentages
race_summary = pd.DataFrame({
    'Number of Patients': race_counts,
    'Percentage': race_percentage
})

# Optional: format the 'Percentage' column to round to 2 decimal places and add the '%' sign
race_summary['Percentage'] = race_summary['Percentage'].map('{:.2f}%'.format)

# Display the summary
print(race_summary)
doctable(race_summary,"Race Statistics",['Race','Number of Patients', 'Percentage'])

# Count the number of patients for each ethnicity
incomecat_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique['INCOME_CATEGORY'].value_counts()

# Calculate the percentage of patients for each ethnicity
incomecat_percentage = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique['INCOME_CATEGORY'].value_counts(normalize=True) * 100

# Create a summary DataFrame to neatly display counts and percentages
incomecat_summary = pd.DataFrame({
    'Number of Patients': incomecat_counts,
    'Percentage': incomecat_percentage
})

# Optional: format the 'Percentage' column to round to 2 decimal places and add the '%' sign
incomecat_summary['Percentage'] = incomecat_summary['Percentage'].map('{:.2f}%'.format)

# Display the summary
print(incomecat_summary)
doctable(incomecat_summary,"Income Category Statistics",['Income Category','Number of Patients', 'Percentage'])
                                                                                           

In [ ]:
# Only patients with Asthma diagnoses in  during the data collection period are considered
patientMRNforAsthmaPatients = df_FINDEMOG.PATIENTHASHMRN.unique()
df_MEDICATIONS = df_MEDICATIONS[df_MEDICATIONS['PATIENTHASHMRN'].isin(patientMRNforAsthmaPatients)]

In [ ]:
# Medication's classes assigned as Severity 
Severe_Med_list=['GLUCOCORTICOIDS', #3
                 'INTERLEUKIN-5 (IL-5) ANTAGONISTS, MAB', # 3 GSk
                 'INTERLEUKIN-5(IL-5) RECEPTOR ALPHA ANTAGONIST, MAB', #3
                 'MONOCLONAL ANTIBODIES TO IMMUNOGLOBULIN E (IGE)',#3
                 'INTERLEUKIN-4(IL-4) RECEPTOR ALPHA ANTAGONIST, MAB',#3
                 'THYMIC STROMAL LYMPHOPOIETIN (TSLP) INHIBITORS' #3
                ]

Moderate_Med_list=[
                'BETA-ADRENERGIC AND ANTICHOLINERGIC COMBO, INHALED',# 2 Moderate  does include stereoid
                'BETA-ADRENERGIC AND GLUCOCORTICOID COMBO, INHALED', # 2 Moderate
                'BETA-ADRENERGIC-ANTICHOLINERGIC-GLUCOCORT, INHALED' # 2 Moderate
                ]

Mild_Med_list=[ 'GLUCOCORTICOIDS, ORALLY INHALED', # 1 
                'GENERAL BRONCHODILATOR AGENTS',  #1 
                'BETA-ADRENERGIC AGENTS', #1
                'BETA-ADRENERGIC AGENTS, INHALED, SHORT ACTING',#1
                'ANTICHOLINERGICS, ORALLY INHALED SHORT ACTING',#1
                'BETA-ADRENERGIC AGENTS, INHALED, ULTRA-LONG ACTING', #1 
                'BETA-ADRENERGIC AGENTS, ORALLY INHALED,LONG ACTING', #1 
                'ANTICHOLINERGICS, ORALLY INHALED LONG ,ACTING'#  1 
                ]

Soft_Med_list=[
                '5-LIPOXYGENASE INHIBITORS',# X Mild  0.5 
                'LEUKOTRIENE RECEPTOR ANTAGONISTS', # X mild 0.5 
                'MAST CELL STABILIZERS, ORALLY INHALED', #0.25 
                'PHOSPHODIESTERASE-4 (PDE4) INHIBITORS' # 0.5
                ]      

def Medic_Severe(cell):
    if any(val in str(cell) for val in Severe_Med_list) == True:
        return 3.0
    elif any(val in str(cell) for val in Moderate_Med_list) == True:
        return 2.0
    elif any(val in str(cell) for val in Mild_Med_list) == True:
        return 1.0
    elif any(val in str(cell) for val in Soft_Med_list) == True:
        return 0.5
    else:
        return 0.0 
    
# Create Severity columns and note severity level of medicines
df_MEDICATIONS['MedicationSeverity']=df_MEDICATIONS['PHARMACEUTICALCLASS'].apply(Medic_Severe)

paragraph = document.add_paragraph('The patient data reported in this section is the patients that has been prescribed oral stereoids or biological medications')
paragraph.add_run(' two new variables addes to the data, medication severity is an indicator of the patients use  medications classified as class 3 which includes oral stereoid and biological medictions')
paragraph.add_run(' the second variable named as patientseverity indicates whether the patient has been prescribed two or more doses of oral stereoids within a window if  6 months')



In [ ]:
comboinhalers = df_MEDICATIONS[
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] == 'BETA-ADRENERGIC AND GLUCOCORTICOID COMBO, INHALED') |
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] =='BETA-ADRENERGIC-ANTICHOLINERGIC-GLUCOCORT, INHALED')
]

In [ ]:
Biologicals =[   'INTERLEUKIN-5 (IL-5) ANTAGONISTS, MAB', # 3 GSk
                 'INTERLEUKIN-5(IL-5) RECEPTOR ALPHA ANTAGONIST, MAB', #3
                 'MONOCLONAL ANTIBODIES TO IMMUNOGLOBULIN E (IGE)',#3
                 'INTERLEUKIN-4(IL-4) RECEPTOR ALPHA ANTAGONIST, MAB',#3
                 'THYMIC STROMAL LYMPHOPOIETIN (TSLP) INHIBITORS' #3
                ]


In [ ]:
BIOLOGICALS = df_MEDICATIONS[df_MEDICATIONS['PHARMACEUTICALCLASS'].isin(Biologicals)].MEDNAME.unique()
BIOLOGICALPresc= df_MEDICATIONS[df_MEDICATIONS['PHARMACEUTICALCLASS'].isin(Biologicals)]

In [ ]:
BIOLOGICALPresc

In [ ]:
COMBOINHALERS = list(comboinhalers.MEDNAME.unique())

In [ ]:
COMBOINHALERS

In [ ]:
COMBOINHALERS
# Convert the list to a DataFrame
df = pd.DataFrame(COMBOINHALERS, columns=['inhaledmedications'])

# Write the DataFrame to an Excel file
df.to_excel('comboinhales.xlsx', index=False)

In [ ]:
GLUCOCORTICOIDSPresc = df_MEDICATIONS[df_MEDICATIONS['PHARMACEUTICALCLASS'] == 'GLUCOCORTICOIDS']

In [ ]:
GLUCOCORTICOIDS = sorted(list(GLUCOCORTICOIDSPresc.MEDNAME.unique()))

In [ ]:
GLUCOCORTICOIDS

In [ ]:
glococorticoidsOrallyInhaled = df_MEDICATIONS[df_MEDICATIONS['PHARMACEUTICALCLASS'].str.strip().str.contains('GLUCOCORTICOIDS, ORALLY INHALED',case=False, na=False, regex=True)]
GLUCOCORTICOIDSORALLYINHALED = sorted(list(glococorticoidsOrallyInhaled.MEDNAME.unique()))



In [ ]:
# Convert the list to a DataFrame
df = pd.DataFrame(GLUCOCORTICOIDSORALLYINHALED, columns=['inhaledmedications'])

# Write the DataFrame to an Excel file
df.to_excel('inhaled.xlsx', index=False)

In [ ]:
Counter(glococorticoidsOrallyInhaled.MEDNAME)

In [ ]:
GLUCOCORTICOIDSORALLYINHALED

In [ ]:
OralInhalerContollers = df_MEDICATIONS[
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] == 'GENERAL BRONCHODILATOR AGENTS')| 
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] =='BETA-ADRENERGIC AGENTS')|
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] =='BETA-ADRENERGIC AGENTS, INHALED, SHORT ACTING')|
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] =='ANTICHOLINERGICS, ORALLY INHALED SHORT ACTING')|
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] =='BETA-ADRENERGIC AGENTS, INHALED, ULTRA-LONG ACTING')| 
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] =='BETA-ADRENERGIC AGENTS, ORALLY INHALED,LONG ACTING')| 
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] =='ANTICHOLINERGICS, ORALLY INHALED LONG ,ACTING')|
    (df_MEDICATIONS['PHARMACEUTICALCLASS'] =='BETA-ADRENERGIC AND ANTICHOLINERGIC COMBO, INHALED')
    

]
CONTROLLERINHALERS = sorted(list(OralInhalerContollers.MEDNAME.unique()))

In [ ]:
CONTROLLERINHALERS
# Convert the list to a DataFrame
df = pd.DataFrame(CONTROLLERINHALERS, columns=['inhaledmedications'])

# Write the DataFrame to an Excel file
df.to_excel('controllerinhales.xlsx', index=False)

In [ ]:
GLUCOCORTICOIDS

In [ ]:
#Filter out all medications  and remove medications with severity 0,  these are medications that are not relevant to Asthma treatment
df_MEDICATIONS = df_MEDICATIONS[df_MEDICATIONS.MedicationSeverity != 0]
# the medication database now contains only medications prescribed for Asthma

In [ ]:
df_MEDICATIONS['c']=0

def check_InhalerStatus(medication_list, grup1, grup2, grup3):
    # Convert groups to sets for efficient lookup
    grup1_set = set(grup1)
    grup2_set = set(grup2)
    grup3_set = set(grup3)

    # Check if there is a medication in GRUP 1 and a separate medication in GRUP 2
    in_grup1 = any(medication in grup1_set for medication in medication_list)
    in_grup2 = any(medication in grup2_set for medication in medication_list)

    if in_grup1 and in_grup2:
        return 1  # Return 1 if a medication is in GRUP 1 and another is in GRUP 2
    
    # Check if any medication is in GRUP 3
    in_grup3 = any(medication in grup3_set for medication in medication_list)
    if in_grup3:
        return 1  # Return 1 if a match is found in GRUP 3
    
    return 0



df_MEDICATIONS['InhalerController']=0

for mrn in df_MEDICATIONS['PATIENTHASHMRN'].unique():
    # Extract the relevant rows for this MRN
    group = df_MEDICATIONS[df_MEDICATIONS['PATIENTHASHMRN'] == mrn]
    medications = group['MEDNAME'].sort_values()

    oralInhalerStatus = check_InhalerStatus(medications,GLUCOCORTICOIDSORALLYINHALED,CONTROLLERINHALERS,COMBOINHALERS)
    
    df_MEDICATIONS.loc[df_MEDICATIONS['PATIENTHASHMRN'] == mrn, 'InhalerController'] = oralInhalerStatus
    

In [ ]:
patientsonBiologicals = df_MEDICATIONS[df_MEDICATIONS['MEDNAME'].isin(BIOLOGICALS)].PATIENTHASHMRN.unique()


In [ ]:
patientsonBiologicals

In [ ]:
#Determine the patient list who has been prescribed an orally inhaled corticosteroid -- 
#Severe asthma will be determined based on the medications precribed

# controller medications, one of which includes a medium or high dose glucosteroid inhaler
# Uncontrolled despite being prescribed these medications, which typically means more than 1 oral steroid course

df_MEDICATIONSOrallyInhaled = df_MEDICATIONS[df_MEDICATIONS['InhalerController'] == 1]
PatientsOralInhal = df_MEDICATIONSOrallyInhaled.PATIENTHASHMRN.unique()
#Patient list who has been prescribed an orally inhaled corticosteroid has been determined

glucocorticoids = ['GLUCOCORTICOIDS']

# Assign the Severe medications to dataframe 
# Patients that were prescribed  Oral Corticosteroids 
df_Severe_med = df_MEDICATIONS[df_MEDICATIONS['PHARMACEUTICALCLASS'].isin(glucocorticoids)]

#df_Severe_med=df_Severe_med[df_Severe_med['PHARMACEUTICALCLASS'].isin(Severe_Med_list)]


df_Severe_med['PatientSeverity']=0.0


df_Severe_med.sort_values(by=['PATIENTHASHMRN', 'DATE_DIF'], inplace=True)

p1 = document.add_paragraph(" Patient severity is decided based on the  number of oral steroid prescritions within 12 months.  if the patient received two or more oral stereoid prescriptons during a 6 month")
p1.add_run("period and at least 10 days apart between steroid prescriptions, the patient is classified as severe, otherwise the patient is classified as not severe based on oral steroid episodes")

p2 = document.add_paragraph("Medication Severity is decided based on the type of the medications.  All oral and injectable stereoids and biologicals are classified as severe medications")


# Function to determine if a patient is a SeverePatient
def check_severity(severity_1_dates):
    retVal = 0
    last_occurance = {}
    event_in_180_days = False
    
    # Check if there are enough dates to compare
    if len(severity_1_dates) < 2:
        return retVal  # Not enough events to compare
    
    # Iterate through the list of dates
    for i in range(len(severity_1_dates) - 1):
        date_difference = severity_1_dates.iloc[i + 1] - severity_1_dates.iloc[i]
        
        # Check if the time difference is between 10 and 365 days
        if 10 < date_difference <= 365:
            if 'OCS' in last_occurance:
                last_date_diff = severity_1_dates.iloc[i + 1] - last_occurance['OCS']
                
                # Check if an OCS event occurred within the last 365 days
                if last_date_diff < 365:
                    last_occurance['OCS'] = severity_1_dates.iloc[i + 1]
                    event_in_180_days = True
                    retVal += 1  # Increment the severity counter
            else:
                # Record the first occurrence of an OCS event
                last_occurance['OCS'] = severity_1_dates.iloc[i + 1]
    
    # Return the result based on whether events occurred in 180 days
    if event_in_180_days:
        return retVal  # Number of severe events detected
    else:
        return 0  # No severe event in the timeframe

for mrn in df_Severe_med['PATIENTHASHMRN'].unique():
    # Extract the relevant rows for this MRN
    group = df_Severe_med[df_Severe_med['PATIENTHASHMRN'] == mrn]

    # Filter for severity 3 medications and sort
    #severity_1_dates = group[group['MedicationSeverity'] == 3.0]['DATE_DIF'].sort_values()
    severity_1_dates = group['DATE_DIF'].sort_values()
   
   
    # Check for severity and update the DataFrame
    severity_status = check_severity(severity_1_dates)
    
    #print(severity_status)
    df_Severe_med.loc[df_Severe_med['PATIENTHASHMRN'] == mrn, 'PatientSeverity'] = severity_status
    
    # Set patient Severity to a binary value of 0 or 1 

df_Severe_med.loc[df_Severe_med['PatientSeverity'] == 1.0, 'PatientSeverity'] = 0.0
# If at least two dose of stereoids within 6 months
df_Severe_med.loc[df_Severe_med['PatientSeverity'] > 1.0, 'PatientSeverity'] = 1.0
df_Severe_med['PatientSeverity'] = df_Severe_med['PatientSeverity'].fillna(0)

In [ ]:
len(patientsonBiologicals)


In [ ]:
import csv
severeCandidatePatients = df_Severe_med[df_Severe_med['PatientSeverity'] == 1 ].PATIENTHASHMRN.unique()
severePatientList  = df_Severe_med[df_Severe_med['PatientSeverity'] ==1 ].PATIENTHASHMRN.unique()

# Specify the CSV file name
file_name = 'uncontrolled.csv'

# Open the file in write mode ('w') and create a csv.writer object
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write each item in the list as a separate row
    for item in severePatientList:
        writer.writerow([item])  # Note the item is wrapped in a list
        
        

# Open the file in write mode ('w') and create a csv.writer object
with open('severeCandidate.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write each item in the list as a separate row
    for item in severeCandidatePatients:
        writer.writerow([item])  # Note the item is wrapped in a list


In [ ]:
#df_FINDEMOG.loc[df_FINDEMOG['PATIENTHASHMRN'].isin(severePatientList), 'Uncontrolled'] = 1

df_FINDEMOG['SeverityCandidate']= 0
df_FINDEMOG.loc[df_FINDEMOG['PATIENTHASHMRN'].isin(severeCandidatePatients), 'SeverityCandidate'] = 1

df_FINDEMOG['onOralInhaledController']= 0
df_FINDEMOG.loc[df_FINDEMOG['PATIENTHASHMRN'].isin(PatientsOralInhal), 'onOralInhaledController'] = 1

df_FINDEMOG['Uncontrolled'] = 0
df_FINDEMOG['Uncontrolled'] = df_FINDEMOG.apply(
    lambda row: 1 if row['SeverityCandidate'] == 1 and row['onOralInhaledController'] == 1 else 0, 
    axis=1
)

df_FINDEMOG['onBiological']= 0
df_FINDEMOG.loc[df_FINDEMOG['PATIENTHASHMRN'].isin(patientsonBiologicals), 'onBiological'] = 1



In [ ]:
Counter(df_FINDEMOG.onOralInhaledController)

In [ ]:
# Filter for patients who are both Uncontrolled and onBiological
uncontrolled_on_biological_counts = df_FINDEMOG[(df_FINDEMOG['Uncontrolled'] == True) & (df_FINDEMOG['onBiological'] == True)].groupby('HHSREGION').size().reset_index(name='Number of Patients Uncontrolled & onBiological')

# Pivot to create a single-row table format with races as columns
uncontrolled_on_biological_table = uncontrolled_on_biological_counts.set_index('HHSREGION').T

# Display the resulting table
print(uncontrolled_on_biological_table)

In [ ]:
df_FINDEMOG.columns

In [ ]:
# Calculate the number of patients onBiological for each race
on_biological_counts = df_FINDEMOG[df_FINDEMOG['Uncontrolled'] == True].groupby('RACE').size().reset_index(name='Number of Patients onBiological')

# Pivot to create a single-row table format with races as columns
on_biological_table = on_biological_counts.set_index('RACE').T

# Display the resulting table
print(on_biological_table)

In [ ]:
df_FINDEMOG

In [ ]:
InhalerControlledPatients = df_FINDEMOG[df_FINDEMOG.SeverityCandidate == 1]

# Calculate the percentage of patients on biological treatment for each race
# Calculate the total population for each race
total_population = df_FINDEMOG["RACE"].value_counts(normalize=True) * 100  # Percentage of each race in the population

# Calculate the percentage on biological for each race
# Filter for biological users
biological_users = df_FINDEMOG[df_FINDEMOG["onBiological"] == 1]

# Total biological users
total_biological_users = len(biological_users)

# Percentage of each race among biological users
percentage_on_biological = biological_users["RACE"].value_counts(normalize=True) * 100


# Align indexes for calculation
total_population = total_population[percentage_on_biological.index]

# Calculate the Disparity Index
disparity_index = percentage_on_biological / total_population

# Combine results into a DataFrame
results = pd.DataFrame({
    "Race": percentage_on_biological.index,
    "Percentage on Biological": percentage_on_biological.values,
    "Percentage in Population": total_population.values,
    "Disparity Index": disparity_index.values
})

# Display the results
print(results)

In [ ]:
df_FINDEMOG.columns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Calculate Total Population Percentage by Race
total_population_percentage = df_FINDEMOG["RACE"].value_counts(normalize=True) * 100

# Filter for biological users
biological_users = df_FINDEMOG[df_FINDEMOG["onBiological"] == 1]

# Calculate Percentage on Biological for Each Race
percentage_on_biological = biological_users["RACE"].value_counts(normalize=True) * 100

# Align indexes for consistent comparison
total_population_percentage = total_population_percentage[percentage_on_biological.index]

# Calculate Disparity Index
disparity_index = percentage_on_biological / total_population_percentage

# Combine results into a DataFrame
results = pd.DataFrame({
    "Race": percentage_on_biological.index,
    "Percentage on Biological": percentage_on_biological.values,
    "Percentage in Population": total_population_percentage.values,
    "Disparity Index": disparity_index.values
})

# Plotting
fig, ax1 = plt.subplots(figsize=(10, 6))
x = np.arange(len(results["Race"]))
bar_width = 0.35

# Bar plots for percentages
bars_population = ax1.bar(x - bar_width / 2, results["Percentage in Population"], bar_width, label="Percentage in Population", color='blue')
bars_biological = ax1.bar(x + bar_width / 2, results["Percentage on Biological"], bar_width, label="Percentage on Biological", color='orange')

# Setting labels and titles
ax1.set_xlabel("")
ax1.set_ylabel("Percentage (%)", fontsize=14)
ax1.set_title("Comparison of Population Percentage, Biological Usage, and Disparity Index", fontsize=16)
ax1.set_xticks(x)
ax1.set_xticklabels(results["Race"], fontsize=14)
#ax1.legend(loc="upper left")

# Add data values above the bars
for bar in bars_population:
    ax1.annotate(f'{bar.get_height():.1f}%', 
                 xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()), 
                 xytext=(0, 5),  # Offset for text
                 textcoords="offset points", 
                 ha='center', va='bottom', fontsize=12)

for bar in bars_biological:
    ax1.annotate(f'{bar.get_height():.1f}%', 
                 xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()), 
                 xytext=(0, 5), 
                 textcoords="offset points", 
                 ha='center', va='bottom', fontsize=12)

# Secondary axis for Disparity Index
ax2 = ax1.twinx()
disparity_points = ax2.scatter(x, results["Disparity Index"], color='red', label="Disparity Index", s=100, zorder=5)
ax2.set_ylabel("Disparity Index", fontsize=14)
ax2.axhline(1, color='red', linestyle='--', linewidth=1, label="No Disparity")

# Annotate disparity index values
for i, val in enumerate(results["Disparity Index"]):
    ax2.annotate(f'{val:.2f}', 
                 (x[i], val), 
                 textcoords="offset points", 
                 xytext=(0, 5), 
                 ha='center', va='bottom', fontsize=12, color='red')

# Add combined legend
fig.legend(loc="upper right", bbox_to_anchor=(0.94, 0.9), fontsize=12, frameon=True)
# Adjust layout and show plot
plt.tight_layout()
plt.show()

# Save results to a CSV file
results.to_csv("biological_disparity_analysis.csv", index=False)

In [ ]:
InhalerControlledPatients = df_FINDEMOG[df_FINDEMOG.SeverityCandidate == 1]

UncontrolledPatients = df_FINDEMOG[(df_FINDEMOG['SeverityCandidate'] == 1) & (df_FINDEMOG['onOralInhaledController'] == 1)]

# Display the new DataFrame
print("Filtered DataFrame:")
UncontrolledPatients
InhalerPatients =  df_FINDEMOG[df_FINDEMOG['onOralInhaledController'] == 1]
InhalerPatients
UncontrolledNoInhaler =  df_FINDEMOG[(df_FINDEMOG['SeverityCandidate'] == 1) & (df_FINDEMOG['onOralInhaledController'] == 0)]
UncontrolledNoInhaler

InhalerPatients =  df_FINDEMOG[df_FINDEMOG['onOralInhaledController'] == 1]
InhalerPatients
UncontrolledNoInhaler =  df_FINDEMOG[(df_FINDEMOG['SeverityCandidate'] == 1) & (df_FINDEMOG['onOralInhaledController'] == 0)]
UncontrolledNoInhaler
uncontrolledpatientsNames = UncontrolledPatients.PATIENTHASHMRN.unique()
inhalerControlledPatientNames = InhalerPatients.PATIENTHASHMRN.unique()
stereoidNoInhalerPatientNames =  UncontrolledNoInhaler.PATIENTHASHMRN.unique()


In [ ]:
# Calculate the total population for each race
total_population = InhalerControlledPatients["FINANCIALCLASS"].value_counts(normalize=True) * 100  # Percentage of each race in the population

# Filter for patients not on biological treatment
non_biological_users = InhalerControlledPatients[InhalerControlledPatients["onBiological"] == 0]

# Total non-biological users
total_non_biological_users = len(non_biological_users)

# Percentage of each race among non-biological users
percentage_not_on_biological = non_biological_users["FINANCIALCLASS"].value_counts(normalize=True) * 100

# Align indexes for calculation
total_population = total_population[percentage_not_on_biological.index]

# Calculate the Disparity Index
disparity_index = percentage_not_on_biological / total_population

# Combine results into a DataFrame
results = pd.DataFrame({
    "Race": percentage_not_on_biological.index,
    "Percentage Not on Biological": percentage_not_on_biological.values,
    "Percentage in Population": total_population.values,
    "Disparity Index": disparity_index.values
})

# Display the results
print(results)

In [ ]:
# Calculate the total population for each race
total_population = InhalerControlledPatients["RACE"].value_counts(normalize=True) * 100  # Percentage of each race in the population

# Filter for patients NOT on biological treatment
non_biological_users = InhalerControlledPatients[InhalerControlledPatients["onBiological"] == 0]

# Total non-biological users
total_non_biological_users = len(non_biological_users)

# Percentage of each race among non-biological users
percentage_not_on_biological = non_biological_users["RACE"].value_counts(normalize=True) * 100

# Reindex total_population to match percentage_not_on_biological indices
percentage_not_on_biological = percentage_not_on_biological.reindex(total_population.index, fill_value=0)

# Calculate the Disparity Index
disparity_index = percentage_not_on_biological / total_population

# Combine results into a DataFrame
results = pd.DataFrame({
    "Race": percentage_not_on_biological.index,
    "Percentage Not on Biological": percentage_not_on_biological.values,
    "Percentage in Population": total_population.values,
    "Disparity Index": disparity_index.values
})

# Display the results
print(results)

In [ ]:
import matplotlib.pyplot as plt

# Total counts for onBiological and not onBiological
biological_counts = df_FINDEMOG[df_FINDEMOG["onBiological"] == 1]["RACE"].value_counts()
non_biological_counts = df_FINDEMOG[df_FINDEMOG["onBiological"] == 0]["RACE"].value_counts()

# Normalize to percentages
total_population = df_FINDEMOG["RACE"].value_counts(normalize=True) * 100
percentage_on_biological = biological_counts / biological_counts.sum() * 100
percentage_not_on_biological = non_biological_counts / non_biological_counts.sum() * 100

# Align for comparison
percentage_on_biological = percentage_on_biological.reindex(total_population.index, fill_value=0)
percentage_not_on_biological = percentage_not_on_biological.reindex(total_population.index, fill_value=0)

# Disparity Index calculation
disparity_index_on = percentage_on_biological / total_population
disparity_index_not_on = percentage_not_on_biological / total_population

# Combine results into a DataFrame
results = pd.DataFrame({
    "Race": total_population.index,
    "Percentage on Biological": percentage_on_biological.values,
    "Percentage Not on Biological": percentage_not_on_biological.values,
    "Percentage in Population": total_population.values,
    "Disparity Index (On Biological)": disparity_index_on.values,
    "Disparity Index (Not on Biological)": disparity_index_not_on.values
})

# Display the results
print(results)

# Visualization of Disparity Index
results.set_index("Race")[["Disparity Index (On Biological)", "Disparity Index (Not on Biological)"]].plot(kind="bar", figsize=(10, 6))
plt.axhline(1, color="red", linestyle="--", label="Parity (Disparity Index = 1)")
plt.title("Disparity Index for On and Not On Biological Treatment by Race")
plt.ylabel("Disparity Index")
plt.xlabel("Race")
plt.legend()
plt.show()

In [ ]:
Counter(df_FINDEMOG['onOralInhaledController'])


In [ ]:
Counter(df_FINDEMOG['Uncontrolled'])


In [ ]:
Counter(df_FINDEMOG['SeverityCandidate'])


In [ ]:

# Create a summary DataFrame
summary = pd.crosstab(index=[df_FINDEMOG['onOralInhaledController']],columns=df_FINDEMOG['SeverityCandidate'])

plt.figure(figsize=(10, 7))
sns.heatmap(summary, annot=True, fmt='d', cmap='YlGnBu', cbar=True)
plt.title('Heatmap of Binary Variables')
plt.xlabel('Two or more Episodes of Systemic Corticosteroids in a Year ')
plt.ylabel('On high dose inhaled corticosteroids plus a second controller')
plt.show()

In [ ]:
import numpy as np

# Step 1: Create the crosstab with counts
summary = pd.crosstab(index=df_FINDEMOG['onOralInhaledController'], columns=df_FINDEMOG['SeverityCandidate'])

# Step 2: Calculate percentages (row-wise or column-wise based on your preference)
# Here I'm calculating percentages relative to the entire table:
summary_percent = summary / summary.sum().sum() * 100  # Entire table percentage

# Optionally, you can calculate percentages row-wise or column-wise as needed:
# Row-wise percentage: summary_percent = summary.div(summary.sum(axis=1), axis=0) * 100
# Column-wise percentage: summary_percent = summary.div(summary.sum(axis=0), axis=1) * 100

# Step 3: Create annotations with counts and percentages
annotations = np.empty_like(summary).astype(str)

for i in range(summary.shape[0]):
    for j in range(summary.shape[1]):
        annotations[i, j] = f'{summary.iloc[i, j]}\n({summary_percent.iloc[i, j]:.1f}%)'

# Step 4: Plot the heatmap with both counts and percentages
plt.figure(figsize=(10, 7))
sns.heatmap(summary, annot=annotations, fmt='', cmap='YlGnBu', cbar=True)

# Add titles and labels
#plt.title('Heatmap of Binary Variables with Counts and Percentages')
plt.xlabel('Two or more Episodes of Systemic Corticosteroids in a Year')
plt.ylabel('On high dose inhaled corticosteroids plus a second controller')

# Show the plot
plt.show()

In [ ]:
from scipy.stats import norm
from scipy.stats import chi2_contingency


def populationProportions(DataFrame,field, title, fieldnames):
    # Ensure 'Uncontrolled' is numeric
    DataFrame['Uncontrolled'] = pd.to_numeric(DataFrame['Uncontrolled'], errors='coerce')

    # Group by ethnicity and count controlled/uncontrolled
    ethnicity_controlled_count = DataFrame.groupby(field)['Uncontrolled'].value_counts().unstack(fill_value=0)
    
    # Convert counts to float for division
    total_counts_per_ethnicity = DataFrame[field].value_counts().astype(float)
    ethnicity_controlled_count = ethnicity_controlled_count.astype(float)

    # Z-score for 95% confidence
    Z = norm.ppf(0.975)

    # Function to calculate confidence interval
    def calculate_ci(n, x):
        p = x / n
        SE = np.sqrt(p * (1 - p) / n)
        lower_bound = p - Z * SE
        upper_bound = p + Z * SE
        return (lower_bound * 100, upper_bound * 100)  # Convert bounds to percentages

    # Calculate confidence intervals
    ci_controlled = ethnicity_controlled_count.apply(lambda row: calculate_ci(total_counts_per_ethnicity[row.name], row[0]), axis=1)
    ci_uncontrolled = ethnicity_controlled_count.apply(lambda row: calculate_ci(total_counts_per_ethnicity[row.name], row[1]), axis=1)

    # Perform Chi-square test
    chi2, p, dof, expected = chi2_contingency(ethnicity_controlled_count)
    chi2_result = pd.DataFrame({
        'Chi-square statistic': [chi2],
        'p-value': [p]
    })

    # Create summary DataFrame with CIs included
    ethnicity_summary = pd.DataFrame({
        'Number of Patients': total_counts_per_ethnicity,
        'Controlled Percentage': ((ethnicity_controlled_count[0] / total_counts_per_ethnicity) * 100).map('{:.2f}%'.format),
        'Uncontrolled Percentage': ((ethnicity_controlled_count[1] / total_counts_per_ethnicity) * 100).map('{:.2f}%'.format),
        'CI Controlled (%)': ci_controlled.map(lambda x: f"{x[0]:.2f}% to {x[1]:.2f}%"),
        'CI Uncontrolled (%)': ci_uncontrolled.map(lambda x: f"{x[0]:.2f}% to {x[1]:.2f}%")
    })

    doctable(ethnicity_summary,title,fieldnames)

    print("Chi-square Test Results:", field)
    print(chi2_result)
    print("\nEthnicity Summary:")
    print(ethnicity_summary)

    p1 = document.add_paragraph("Chi-square Test Results:")
    p1.add_run(field)
    p1.add_run(str(chi2_result))
    
    
    
# Call the function with appropriate parameters
    
populationProportions(df_FINDEMOG,'RACE','Race and Ethnicity Statistics',['Race','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%)' ])
populationProportions(df_FINDEMOG,'INCOME_CATEGORY','Income Category Statistics',['Income Category','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%)' ])
populationProportions(df_FINDEMOG,'SEX','Gender Statistics',['Sex','Number of Patients','Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%)' ])
populationProportions(df_FINDEMOG,'AGE_RANGE','Age Statistics',['AGE_RANGE','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%)' ])
populationProportions(df_FINDEMOG,'HHSREGION','Health Service region Statistics',['HHSREGION','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%) '])
populationProportions(df_FINDEMOG,'FINANCIALCLASS','Insurance Status Statistics',['FINANCIAL CLASS','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%) '])


In [ ]:
df_LABS.columns

In [ ]:
from scipy.stats import chi2_contingency
import numpy as np
import pandas as pd

def populationProportionsByGroup(DataFrame, field, title, fieldnames):
    # Ensure 'Uncontrolled' is numeric
    DataFrame['Uncontrolled'] = pd.to_numeric(DataFrame['Uncontrolled'], errors='coerce')

    # Step 1: Create a crosstab that counts controlled (0) and uncontrolled (1) patients for each demographic group
    group_controlled_uncontrolled = pd.crosstab(DataFrame[field], DataFrame['Uncontrolled'], normalize='columns') * 100

    # Step 2: Calculate the number of patients in each group (for reporting purposes)
    total_patients_per_group = DataFrame[field].value_counts()

    # Step 3: Perform Chi-square test
    chi2, p, dof, expected = chi2_contingency(pd.crosstab(DataFrame[field], DataFrame['Uncontrolled']))

    # Step 4: Calculate Odds Ratios for each group (compared to all other groups combined)
    raw_counts = pd.crosstab(DataFrame[field], DataFrame['Uncontrolled'])  # Get raw counts for OR calculation
    odds_ratios = []
    for group in raw_counts.index:
        a = raw_counts.loc[group, 1]  # Uncontrolled in the group
        b = raw_counts.loc[group, 0]  # Controlled in the group
        c = raw_counts[1].sum() - a  # Uncontrolled in all other groups
        d = raw_counts[0].sum() - b  # Controlled in all other groups
        
        # Ensure no divisions by zero
        if b == 0 or d == 0:
            odds_ratio = np.nan  # Undefined odds ratio
        else:
            odds_ratio = (a / b) / (c / d)
        
        odds_ratios.append(odds_ratio)

    # Step 5: Create a summary DataFrame
    group_summary = pd.DataFrame({
        'Number of Patients': total_patients_per_group,
        'Percentage in Controlled (%)': group_controlled_uncontrolled[0].map('{:.2f}%'.format),
        'Percentage in Uncontrolled (%)': group_controlled_uncontrolled[1].map('{:.2f}%'.format),
        'Odds Ratio': odds_ratios
    })
    
    
    doctable(group_summary,title,fieldnames)

    # Step 6: Display the summary and the chi-square test results
    print(f"\nChi-square Test Results for {field}:")
    print(f"Chi-square Statistic: {chi2:.4f}")
    print(f"p-value: {p:.4f}\n")

    print(f"\n{title} Summary:")
    print(group_summary)
        
    p1 = document.add_paragraph("Chi-square Test Results:")
    p1.add_run(field)
    p1.add_run(str(chi2))
    

# Call the function with appropriate parameters
populationProportionsByGroup(df_FINDEMOG, 'RACE', 'Race and Ethnicity Statistics',
                             ['Race', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(df_FINDEMOG, 'AGE_RANGE', 'Age Statistics',
                             ['Age Range', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(df_FINDEMOG, 'INCOME_CATEGORY', 'Income Category Statistics',
                             ['Income Category', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(df_FINDEMOG, 'SEX', 'Gender Statistics',
                             ['Sex', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(df_FINDEMOG, 'HHSREGION', 'Health Services Region Statistics',
                             ['HHSREGION', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(df_FINDEMOG, 'FINANCIALCLASS', 'Insurance Status Statistics',
                             ['FINAL CLASS', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

In [ ]:
    
populationProportions(InhalerPatients,'RACE','Race and Ethnicity Statistics',['Race','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%)' ])
populationProportions(InhalerPatients,'INCOME_CATEGORY','Income Category Statistics',['Income Category','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%)' ])
populationProportions(InhalerPatients,'SEX','Gender Statistics',['Sex','Number of Patients','Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%)' ])
populationProportions(InhalerPatients,'AGE_RANGE','Age Statistics',['AGE_RANGE','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%)' ])
populationProportions(InhalerPatients,'HHSREGION','Health Service region Statistics',['HHSREGION','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%) '])
populationProportions(InhalerPatients,'FINANCIALCLASS','Insurance Status Statistics',['FINANCIAL CLASS','Number of Patients', 'Controlled Percentage','Uncontrolled Percentage','CI Controlled (%)',' CI Uncontrolled (%) '])


In [ ]:
# Call the function with appropriate parameters
populationProportionsByGroup(InhalerPatients, 'RACE', 'Race and Ethnicity Statistics',
                             ['Race', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(InhalerPatients, 'AGE_RANGE', 'Age Statistics',
                             ['Age Range', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(InhalerPatients, 'INCOME_CATEGORY', 'Income Category Statistics',
                             ['Income Category', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(InhalerPatients, 'SEX', 'Gender Statistics',
                             ['Sex', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(InhalerPatients, 'HHSREGION', 'Health Services Region Statistics',
                             ['HHSREGION', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

populationProportionsByGroup(InhalerPatients, 'FINANCIALCLASS', 'Insurance Status Statistics',
                             ['FINAL CLASS', 'Number of Patients', 'Percentage in Controlled', 'Percentage in Uncontrolled', 'Odds Ratio'])

In [ ]:
df_FINDEMOG

In [ ]:
df_FINDEMOG.PATIENTHASHMRN.nunique()



In [ ]:
import pandas as pd
import statsmodels.api as sm

DataFrame = df_FINDEMOG

# Initialize a DataFrame to store the results
odds_ratios = pd.DataFrame(columns=['Race', 'Odds_Ratio', 'Lower_CI', 'Upper_CI', 'Controlled_Percentage', 'Uncontrolled_Percentage'])

# Get a list of all races
all_races = DataFrame['RACE'].unique()

# Loop through each race to compute the odds ratio and percentages
for race in all_races:
    # Filter data for the current race and other races
    race_data = DataFrame[DataFrame['RACE'] == race]
    other_races_data = DataFrame[DataFrame['RACE'] != race]

    # Count the number of controlled and uncontrolled cases
    race_controlled_count = race_data['Uncontrolled'].value_counts().get(0, 0)
    race_uncontrolled_count = race_data['Uncontrolled'].value_counts().get(1, 0)
    total_race_cases = race_controlled_count + race_uncontrolled_count

    # Calculate percentages
    controlled_percentage = (race_controlled_count / total_race_cases) * 100 if total_race_cases != 0 else 0
    uncontrolled_percentage = (race_uncontrolled_count / total_race_cases) * 100 if total_race_cases != 0 else 0
    
    # Build the contingency table for odds ratio calculation
    contingency_table = pd.DataFrame({
        'Controlled': [race_controlled_count, other_races_data['Uncontrolled'].value_counts().get(0, 0)],
        'Uncontrolled': [race_uncontrolled_count, other_races_data['Uncontrolled'].value_counts().get(1, 0)]
    }, index=[race, 'Other Races'])
    
    # Calculate the odds ratio with a 95% confidence interval
    if contingency_table.shape == (2, 2):  # Ensure valid 2x2 table
        table = sm.stats.Table2x2(contingency_table.values)  # Create a 2x2 table
        odds_ratio = table.oddsratio
        lower_ci, upper_ci = table.oddsratio_confint(alpha=0.05)
    else:
        odds_ratio, lower_ci, upper_ci = 'Invalid table', 'N/A', 'N/A'
    
    # Store the results using pd.concat
    new_row = pd.DataFrame({
        'Race': [race],
        'Odds_Ratio': [odds_ratio],
        'Lower_CI': [lower_ci],
        'Upper_CI': [upper_ci],
        'Controlled_Percentage': [controlled_percentage],
        'Uncontrolled_Percentage': [uncontrolled_percentage]
    })
    odds_ratios = pd.concat([odds_ratios, new_row], ignore_index=True)

# Display the results
print(odds_ratios)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar plot for controlled and uncontrolled percentages
sns.barplot(x='Race', y='Controlled_Percentage', data=odds_ratios, color='blue', label='Controlled', ax=ax1)
sns.barplot(x='Race', y='Uncontrolled_Percentage', data=odds_ratios, color='red', alpha=0.6, label='Uncontrolled', ax=ax1)
ax1.set_ylabel('Percentage')

# Line plot for odds ratios
ax2 = ax1.twinx()
sns.lineplot(x='Race', y='Odds_Ratio', data=odds_ratios, marker='o', sort=False, ax=ax2)
ax2.set_ylabel('Odds Ratio')

fig.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
plt.title('Control Status by Race with Odds Ratios')
plt.show()


In [ ]:
def uncontrolledStats(field, title, fieldnames):
    
    # Set font size globally for the plot
    plt.rcParams.update({'font.size': 14})
    
        # Ensure the figure background is white
    #plt.rcParams['axes.facecolor'] = 'none'  # Set axis background
    #plt.rcParams['figure.facecolor'] = 'none'  # Set figure background
    race_counts = df_FINDEMOG.groupby(['Uncontrolled', field]).size()
    total_counts = race_counts.groupby('Uncontrolled').sum()
    race_percentages = (race_counts.unstack().div(total_counts, axis=0) * 100).stack().reset_index(name='Percentage')
    race_percentages.columns = ['Control Status', field, 'Percentage']

    # Pivot and calculate stats
    pivot_table = race_percentages.pivot(index=field, columns='Control Status', values='Percentage')
    pivot_table.columns = ['Controlled Percentage', 'Uncontrolled Percentage']
      
    # Here, ensure 'total' is derived from actual data
    total = df_FINDEMOG[field].count()

    # Calculate Odds for Controlled and Uncontrolled
    pivot_table['Odds_Controlled'] = pivot_table['Controlled Percentage'] / (100 - pivot_table['Controlled Percentage'])
    pivot_table['Odds_Uncontrolled'] = pivot_table['Uncontrolled Percentage'] / (100 - pivot_table['Uncontrolled Percentage'])

    # Calculate Odds Ratios for being uncontrolled vs. controlled
    pivot_table['Odds_Ratio'] = pivot_table['Odds_Uncontrolled'] / pivot_table['Odds_Controlled']

    # Convert the percentages back to counts to get the original numbers for a and b
    pivot_table['a'] = pivot_table['Controlled Percentage'] * total / 100
    pivot_table['b'] = pivot_table['Uncontrolled Percentage'] * total / 100

    # Calculate the standard error of the log of the odds ratio
    pivot_table['SE_log_OR'] = np.sqrt(1/pivot_table['a'] + 1/pivot_table['b'] + 
                                       1/(total - pivot_table['a']) + 1/(total - pivot_table['b']))

    # Calculate the confidence intervals
    pivot_table['lower_CI'] = np.exp(np.log(pivot_table['Odds_Ratio']) - 1.96 * pivot_table['SE_log_OR'])
    pivot_table['upper_CI'] = np.exp(np.log(pivot_table['Odds_Ratio']) + 1.96 * pivot_table['SE_log_OR'])

    # Output the final table with CIs
    print(pivot_table[['Controlled Percentage', 'Uncontrolled Percentage', 'Odds_Controlled', 'Odds_Uncontrolled', 'Odds_Ratio', 'lower_CI', 'upper_CI']])

    # Reset index for easier plotting
    custom_order = fieldnames
    
    if (field == 'INCOME_CATEGORY') :
        pivot_table = pivot_table.drop('Unknown', axis=0)
    else:
        if (field != 'AGE_RANGE'):
            pivot_table = pivot_table.drop('Other', axis=0)
    
    
    pivot_table = pivot_table.reindex(custom_order)

    plot_data = pivot_table.reset_index()


    fig, ax1 = plt.subplots(figsize=(7,5))

    # Bar plot for percentages
    pivot_table[['Controlled Percentage', 'Uncontrolled Percentage']].plot(kind='bar', ax=ax1)
   # Add annotations to the existing plot
    for container in ax1.containers:
        for bar in container:
            # Get bar height
            height = bar.get_height()
            if height > 0:  # Annotate only if height is greater than 0
                ax1.text(
                    bar.get_x() + bar.get_width() / 2,  # Center of the bar
                    height / 2 ,  # Slightly above the bar
                    f'{height:.1f}%',  # Format value as percentage
                    ha='center',  # Center alignment
                    va='center',  # Bottom alignment
                    fontsize=14,  # Font size
                    rotation=90  # Rotate text to vertical

                )
    ax1.set_ylabel('Percentage', fontsize=14)
#    ax1.set_title('Control Status and Odds Ratios by Race')
    ax1.tick_params(axis='x',labelsize=14, rotation=45)
    ax1.tick_params(axis='y',labelsize=14)

    ax1.set_xlabel('')  # Remove x-axis title
    ax1.legend(['Controlled Percentage', 'Uncontrolled Percentage'], loc='upper left', fontsize=12)  # Increased font size
    
    
    #ax1.grid(False)



    # Second y-axis for the odds ratios
    ax2 = ax1.twinx()
    sns.lineplot(data=plot_data, x=field, y='Odds_Ratio', marker='o', ax=ax2, color='red')
    ax2.set_ylabel('Odds Ratio', fontsize=14)

    # Add error bars manually for the odds ratios
    ax2.errorbar(x=plot_data[field], y=plot_data['Odds_Ratio'], 
                 yerr=[plot_data['Odds_Ratio'] - plot_data['lower_CI'], plot_data['upper_CI'] - plot_data['Odds_Ratio']],
                 fmt='none', c='red', capsize=6)
    
    ax2.set_xlabel('')  # Remove x-axis title
    ax2.tick_params(axis='y',labelsize=14)


    
    ax2.grid(False)
    
    plt.tight_layout()
    
    image_path = "plot.png"
    plt.tight_layout()
    plt.savefig(image_path, dpi=300)
    document.add_picture(image_path, width=Inches(7))  # Adjust width as needed

    plt.show()

uncontrolledStats('SEX', "Sex", ['Female','Male'])
uncontrolledStats('INCOME_CATEGORY', "Income Category", ['Very Low', 'Low', 'Median', 'High', 'Very High'])
#uncontrolledStats('INCOME_CATEGORY', "Income Category", ['Low', 'Median', 'High', 'Very High'])

uncontrolledStats('RACE', "Race", ['Asian','Black', 'Hispanic', 'White'])
uncontrolledStats('FINANCIALCLASS','Financial Class ', ['Commercial', 'Managed Care', 'Medi-Cal', 'Self-Pay', 'Tricare','Medicare'])
uncontrolledStats('AGE_RANGE','Age Range ', ['6-17','18-30','31-40','41-50','51-63','64+'])



In [ ]:
df_FINDEMOG.FINANCIALCLASS.unique()


In [ ]:
mapping = {'Commercial':'Commercial', 'Managed Care':'Managed Care', 'Medi-Cal':'Medi-Cal', 'Medi-Cal Managed Care':'Medi-Cal','Self-Pay':'Self-Pay', 'Tricare':'Tricare','Medicare':'Medicare', 'Worker\'s Comp':'Other','Medicare HMO/PPO':'Medicare'}
df_FINDEMOG['FINANCIALCLASS'] = df_FINDEMOG['FINANCIALCLASS'].map(mapping).fillna(df_FINDEMOG['FINANCIALCLASS'])
df_FINDEMOG['FINANCIALCLASS'] = df_FINDEMOG['FINANCIALCLASS'].fillna('Other')

In [ ]:
df_FINDEMOG.columns

In [ ]:
#df_FINDEMOG=df_FINDEMOG.drop('Unnamed: 0_y',axis=1)
#df_FINDEMOG=df_FINDEMOG.drop('Unnamed: 0_x',axis=1)

In [ ]:
# Logistic_list_dummies=['RACE','SEX','AGE_RANGE','INCOME_CATEGORY','FINANCIALCLASS','Uncontrolled']
# df_LOGISTIC_DUMM=pd.get_dummies(In[Logistic_list_dummies], drop_first=False)
# X=df_LOGISTIC_DUMM.drop('Uncontrolled',axis=1)
# Y=df_LOGISTIC_DUMM['Uncontrolled']

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.39, random_state=42)

# # Initialize the Logistic Regression model
# model = LogisticRegression(solver='liblinear')  # Using 'liblinear' solver as it's good for small datasets and binary classification.

# # Train the model
# model.fit(X_train, y_train)

# # Coefficients
# coefficients = model.coef_[0]
# intercept = model.intercept_[0]

# # Calculate the standard errors of the coefficients
# # Covariance matrix of the parameter estimates
# covariance_matrix = np.linalg.inv(np.dot(X_train.T, X_train))
# standard_errors = np.sqrt(np.diag(covariance_matrix))

# # Compute the 95% Confidence Intervals for the coefficients
# z = 1.96  # z-score for 95% CI
# lower_bounds = coefficients - z * standard_errors
# upper_bounds = coefficients + z * standard_errors

# # Compute the 95% CI for the Odds Ratios
# lower_bounds_or = np.exp(lower_bounds)
# upper_bounds_or = np.exp(upper_bounds)

# # Print results
# # print("Intercept:", intercept)
# # print("Coefficients:", coefficients)
# # print("Odds Ratios:", np.exp(coefficients))
# # print("95% CI for Coefficients:", list(zip(lower_bounds, upper_bounds)))
# # print("95% CI for Odds Ratios:", list(zip(lower_bounds_or, upper_bounds_or)))
# merged_data = pd.DataFrame({
#     'Variable':X.columns,
#     'OR': np.exp(coefficients),
#     '2.5%': lower_bounds_or,
#     '97.5%': upper_bounds_or
# })

# # Save the DataFrame to a CSV file
# merged_data

In [ ]:
# # Identify indices of NaN values in standard errors
# nan_indices = np.where(np.isnan(std_err))[0]

# # Identify corresponding variable names
# problematic_vars = X_train_reduced.columns[nan_indices]

# print(f"Variables with NaN in standard errors: {problematic_vars}")

# # Optionally, remove these variables and refit the model
# if len(problematic_vars) > 0:
#     X_train_reduced = X_train_reduced.drop(columns=problematic_vars)
#     X_train_sm_reduced = sm.add_constant(np.asarray(X_train_reduced, dtype=float))

#     # Refit the model after removing problematic variables
#     logit_model_reduced = sm.Logit(y_train_np, X_train_sm_reduced)
#     result_reduced = logit_model_reduced.fit_regularized(method='l1', maxiter=500)

#     # Recalculate statistics
#     coef_reduced = result_reduced.params
#     std_err = result_reduced.bse
#     p_values = result_reduced.pvalues
#     conf_int = result_reduced.conf_int()

#     print("Model refitted after removing problematic variables.")

#     # Ensure everything is the same length before creating the DataFrame
#     assert len(coef_reduced) == len(std_err) == len(p_values) == len(conf_int), "Mismatch in lengths!"

#     # Compile into a DataFrame
#     result_df = pd.DataFrame({
#         'Coefficient': coef_reduced,
#         'Std Error': std_err,
#         'P-Value': p_values,
#         'Conf. Interval Lower': conf_int[0],
#         'Conf. Interval Upper': conf_int[1],
#         'OR': np.exp(coef_reduced)
#     })

#     print("Final Model results after removing problematic variables:")
#     print(result_df)
# else:
#     print("No variables were found with NaN in standard errors.")


In [ ]:
# groups_dummies = ['RACE', 'SEX', 'AGE_RANGE', 'INCOME_CATEGORY', 'FINANCIALCLASS']
# references = {
#     'RACE_White': 'RACE', 
#     'SEX_Female': 'SEX', 
#     'AGE_RANGE_18-30': 'AGE_RANGE',
#     'INCOME_CATEGORY_Low': 'INCOME_CATEGORY',
#     'FINANCIALCLASS_Commercial': 'FINANCIALCLASS'
# }

# referenced_df = pd.DataFrame(columns=merged_data.columns)

# for ref, cat in references.items():
#     # Filter rows based on category
#     small_df = merged_data[merged_data['Variable'].str.contains(cat)].copy()

#     # Get reference number (OR) for the reference variable
#     ref_num = small_df.loc[small_df['Variable'] == ref, 'OR'].values[0]

#     # Normalize the OR, 2.5%, and 97.5% by the reference number
#     for col in ['OR', '2.5%', '97.5%']:
#         small_df[col] = small_df[col] / ref_num

#     # Append to the referenced_df
#     referenced_df = pd.concat([referenced_df, small_df], ignore_index=True)

# print(referenced_df)

In [ ]:
# Variable= ['Race    ',
#             'Asian', 'Black', 'Hispanic', 'Other', 'White',
#             'Sex    ',
#            'Female', 'Male', 'Other', 
#            'Age    ',
#            '18-30', '31-40', '41-50', '51-63', '6-17', '64+',
#            'Income    ',
#            'Very Low','High', 'Low', 'Median', 'Unknown', 'Very High',
#            'Insurance    ',
#            'Commercial', 'Managed Care','Medi-Cal', 'Medicare', 'Other', 'Self-Pay', 'Tricare']
# sub_group_names=['RACE','SEX','AGE','INCOME','FINANCIALCLASS']
# referenced_df = referenced_df.drop(index=referenced_df[referenced_df['Variable'].str.contains('Other')].index)


In [ ]:
# for x in sub_group_names:
#     sub_group=referenced_df[referenced_df['Variable'].str.contains(x)]
#     length_sub=len(sub_group)

#     fig, ax = plt.subplots(figsize=(5,length_sub-1 ))
#     i=0
    
#     for _, row in sub_group.iterrows():
#         if row['OR'] == 1:
#             # Draw a square if OR is 1
#             ax.plot(row['OR'], length_sub-(i+1), 's', color='blue', markersize=12)
#         else:
#             # Draw error bars with a point for other values
#             ax.errorbar(row['OR'],  length_sub-(i+1), xerr=[[row['OR'] - row['2.5%']], [row['97.5%'] - row['OR']]],
#                         fmt='o', color='red', ecolor='black', capsize=3, markersize=5)
#         i=i+1
#     ax.plot(1, length_sub, 's', color='blue', markersize=0)

#     ytic=list(sub_group['Variable'][::-1])
#     ytic_cleaned = [var.replace(x+'_', ' ') for var in ytic]
#     ytic_cleaned=ytic_cleaned+[x+':']
#       # Reverse the order for y-axis labels
#     plt.yticks(range(1+len(sub_group)), ytic_cleaned, fontsize=14)
#     plt.xticks( fontsize=14)

In [ ]:
#document=Document()

In [ ]:
def boxPlot(field1,field2, DataFrame, xlabel, ylabel,  title):
    sns.set(font_scale=1)
    plt.figure()
    sns.boxplot(x=field1, y=field2, data=DataFrame, palette='Blues')

    #plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel('')
    plt.xticks(rotation=45,ha='right',fontsize=12)  # Rotate x-axis labels for better readability
    plt.yticks(fontsize=12)  # Rotate x-axis labels for better readability
    
    plt.tight_layout()
   
    plt.savefig('temp.png', bbox_inches='tight')
    document.add_picture('temp.png')
    paragraph = document.add_paragraph(title)
    #plt.show()

In [ ]:
def violinPlot(field1,field2, DataFrame, xlabel, ylabel,  title):
    sns.set(font_scale=1)
    plt.figure(figsize=(7, 5))
    sns.violinplot(x=field1, y=field2, data=DataFrame, palette='Blues')

    #plt.title(title)
    plt.ylabel(ylabel,fontsize=14)
    plt.yticks(fontsize=14)
    #plt.xlabel('HHS Regions')
    plt.xlabel('')

    plt.xticks(rotation=45,ha='right',fontsize=14)  # Rotate x-axis labels for better readability
    plt.tight_layout()
    
    plt.savefig('temp.png', bbox_inches='tight')
    document.add_picture('temp.png', width=Inches(7))
    paragraph = document.add_paragraph(title)
    
    #plt.show()

In [ ]:
def printQuartiles(field1,field2, DataFrame, title, labels):
    quartiles = DataFrame.groupby(field1)[field2].quantile([0.25, 0.5, 0.75]).unstack()
    mean = DataFrame.groupby(field1)[field2].mean()
    maximum = DataFrame.groupby(field1)[field2].max()
    #minimum = df_FINDEMOG.groupby('HHSREGION')['MEDIANINCOME'].min()

    # Combine statistics into a single DataFrame
    count = DataFrame[field1].value_counts()

    # Combine statistics into a single DataFrame
    stats = pd.concat([quartiles, mean, maximum, count], axis=1)
    stats.columns = ['25th Quartile', 'Median', '75th Quartile', 'Mean', 'Maximum', 'Count']

    print(stats)

    doctable(stats,title, labels)

In [ ]:
df_FINDEMOGFiltered = df_FINDEMOG[df_FINDEMOG['HHSREGION'] != 'Other']
printQuartiles('HHSREGION', 'MEDIANINCOME', df_FINDEMOGFiltered, "Median Income Statistics for Each District in the Database",['District','25th Quartile', 'Median', '75th Quartile', 'Mean', 'Maximum', 'Number of Patients'])
boxPlot('HHSREGION', 'MEDIANINCOME',df_FINDEMOGFiltered, '', 'Median Income'," Median Income versus HHSREGION") 
violinPlot('HHSREGION', 'MEDIANINCOME',df_FINDEMOGFiltered, '', 'Median Income'," Median Income versus HHSREGION") 

In [ ]:
df_FINDEMOGFiltered = df_FINDEMOG[df_FINDEMOG['FINANCIALCLASS'] != 'Other']

printQuartiles('FINANCIALCLASS', 'MEDIANINCOME', df_FINDEMOGFiltered, "Median Income Statistics for Each Insurance Class in the Database",['District','25th Quartile', 'Median', '75th Quartile', 'Mean', 'Maximum', 'Number of Patients'])
boxPlot('FINANCIALCLASS', 'MEDIANINCOME',df_FINDEMOGFiltered, '', 'Median Income'," Median Income versus Insurance Status") 
violinPlot('FINANCIALCLASS', 'MEDIANINCOME',df_FINDEMOGFiltered, '', 'Median Income'," Median Income versus Insurance Status") 

In [ ]:
printQuartiles('RACE', 'MEDIANINCOME', df_FINDEMOG, "Median Income Statistics for Each District in the Database",['District','25th Quartile', 'Median', '75th Quartile', 'Mean', 'Maximum', 'Number of Patients'])
boxPlot('RACE', 'MEDIANINCOME',df_FINDEMOG, '', 'Median Income'," Median Income versus RACE") 
violinPlot('RACE', 'MEDIANINCOME',df_FINDEMOG, '', 'Median Income'," Median Income versus RACE") 

In [ ]:
printQuartiles('SEX', 'MEDIANINCOME', df_FINDEMOG, "Median Income Statistics for Each District in the Database",['District','25th Quartile', 'Median', '75th Quartile', 'Mean', 'Maximum', 'Number of Patients'])
boxPlot('SEX', 'MEDIANINCOME',df_FINDEMOG, '', 'Median Income'," Median Income versus SEX") 
violinPlot('SEX', 'MEDIANINCOME',df_FINDEMOG, '', 'Median Income'," Median Income versus SEX") 

In [ ]:
printQuartiles('FINANCIALCLASS', 'MEDIANINCOME', df_FINDEMOG, "Median Income Statistics for each Insurance in the Database",['District','25th Quartile', 'Median', '75th Quartile', 'Mean', 'Maximum', 'Number of Patients'])
boxPlot('FINANCIALCLASS', 'MEDIANINCOME',df_FINDEMOG, '', 'Median Income'," Median Income versus Insurance") 
violinPlot('FINANCIALCLASS', 'MEDIANINCOME',df_FINDEMOG, '', 'Median Income'," Median Income versus Insurance") 

In [ ]:
#document.save("Simple.docx")

In [ ]:
#df_DIAGPROC=df_DIAGPROC.drop('Unnamed: 0_y',axis=1)
#df_DIAGPROC=df_DIAGPROC.drop('Unnamed: 0_x',axis=1)


In [ ]:
#df_DIAGPROC_FINDEMOG_ENCFiltered = df_DIAGPROC_FINDEMOG_ENC[df_DIAGPROC_FINDEMOG_ENC['DEPTSPECIALTY'].isin(DepartmentRelevant)]

In [ ]:
df_DIAGPROC_FINDEMOG_ENCFiltered

In [ ]:
#make a copy of the procedures  and encounters for further analysis 
df_DIAGPROC_FINDEMOG_ENCFiltered_copy = df_DIAGPROC_FINDEMOG_ENCFiltered


In [ ]:
#copy the data back from the master
df_DIAGPROC_FINDEMOG_ENCFiltered = df_DIAGPROC_FINDEMOG_ENCFiltered_copy

#replace ICD10 codes for Astma sub penotypes with do not care
df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'] = df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'].replace({'J45.20': 'J45.2x', 'J45.21': 'J45.2x'})
df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'] = df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'].replace({'J45.30': 'J45.3x', 'J45.31': 'J45.3x', 'J45.32': 'J45.3x'})
df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'] = df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'].replace({'J45.40': 'J45.4x', 'J45.41': 'J45.4x', 'J45.42': 'J45.4x'})
df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'] = df_DIAGPROC_FINDEMOG_ENCFiltered['ICD10CODE'].replace({'J45.50': 'J45.5x', 'J45.51': 'J45.5x', 'J45.52': 'J45.5x'})


num_unique_patients = df_DIAGPROC_FINDEMOG_ENCFiltered['PATIENTHASHMRN'].nunique()
patient_visits = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby('PATIENTHASHMRN').size().reset_index(name='Visit_Count')

icd_codes_list = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby('PATIENTHASHMRN')['ICD10CODE'].apply(list).reset_index(name='ICD10CODEs')
dept_specialty_list = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby(['PATIENTHASHMRN', 'ICD10CODE'])['DEPTSPECIALTY'].apply(list).reset_index(name='DEPTSPECIALTYs')

patient_visits = pd.merge(patient_visits, icd_codes_list, on='PATIENTHASHMRN', how='left')
patient_visits_department = pd.merge(patient_visits, dept_specialty_list, on='PATIENTHASHMRN', how='left')

patient_visits_department=patient_visits_department.drop('ICD10CODE',axis=1)

#print("Summary DataFrame:")
#print(patient_visits)

# Merging both DataFrames on 'PATIENTHASHMRN'
merged_df = pd.merge(icd_codes_list, dept_specialty_list, on='PATIENTHASHMRN', how='left')

# Creating a pivot table
pivot_table = pd.pivot_table(df_DIAGPROC_FINDEMOG_ENCFiltered, index='DEPTSPECIALTY', columns='ICD10CODE', aggfunc='size', fill_value=0)

document.add_paragraph("Matrix of DEPTSPECIALTY versus ICD10CODES:")
print(pivot_table)
doctable(pivot_table," Department Specialities and Patients Seen",['Department Speciality','J45.2x', 'J45.3x', 'J45.4x', 'J45.5x'])


In [ ]:
#Filter for visits to Emergency Medicine
emergency_visits = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']
total_emergency_visits = emergency_visits.groupby('PATIENTHASHMRN').size()

# Convert the Series to a DataFrame and give the column a name
total_emergency_visits_df = total_emergency_visits.reset_index(name='Total Emergency Visits')


#Identify unique MRNs from these visits
unique_mrn_emergency = emergency_visits['PATIENTHASHMRN'].unique()

# Filter original DataFrame for these MRNs to find visits to other departments
emergency_dept_visits = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['PATIENTHASHMRN'].isin(unique_mrn_emergency)]
                                                     #& (df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] != 'Emergency Medicine')]

#Count visits to each department for these MRNs
visits_count = emergency_dept_visits.groupby(['PATIENTHASHMRN', 'DEPTSPECIALTY']).size().reset_index(name='Visits')

department_visits_count = emergency_dept_visits.groupby(['DEPTSPECIALTY']).size().reset_index(name='Visits')

# The departments and number of time visited by patients who visited emergency services
dept_visits_matrix = emergency_dept_visits.pivot_table(index='PATIENTHASHMRN', columns='DEPTSPECIALTY', aggfunc='size', fill_value=0)

dept_visits_matrix = dept_visits_matrix.merge(total_emergency_visits_df, how='left', left_index=True, right_on='PATIENTHASHMRN')

# Set PATIENTHASHMRN back as the index if it's been reset due to the merge
dept_visits_matrix.set_index('PATIENTHASHMRN', inplace=True)

# Fill NaN values that might have occurred if there are patients with no emergency visits recorded
dept_visits_matrix['Total Emergency Visits'].fillna(0, inplace=True)

# Convert the 'Total Emergency Visits' column to integers to clean up the data presentation
dept_visits_matrix['Total Emergency Visits'] = dept_visits_matrix['Total Emergency Visits'].astype(int)

dept_visits_matrix = dept_visits_matrix.sort_values(by='Total Emergency Visits', ascending=False)

# Reset the index to turn the index (PATIENTHASHMRN) into a column
dept_visits_matrix = dept_visits_matrix.reset_index()

emergency_dept_visits_matrix = pd.merge(dept_visits_matrix, df_FINDEMOG, on='PATIENTHASHMRN', how='inner')





In [ ]:
emergency_dept_visits_matrix

In [ ]:
department_visits_count.to_csv("departmentVisitCounts.csv")

In [ ]:
dept_visits_matrix.to_csv("emergencyVisit.csv")

In [ ]:
#Filter for visits to Emergency Medicine
emergency_visits = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']
total_emergency_visits = emergency_visits.groupby('PATIENTHASHMRN').size()

# Convert the Series to a DataFrame and give the column a name
total_emergency_visits_df = total_emergency_visits.reset_index(name='Total Emergency Visits')


#Identify unique MRNs from these visits
unique_mrn_emergency = emergency_visits['PATIENTHASHMRN'].unique()

# Filter original DataFrame for these MRNs to find visits to other departments
other_dept_visits = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['PATIENTHASHMRN'].isin(unique_mrn_emergency)& (df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] != 'Emergency Medicine')]

#Count visits to each department for these MRNs
visits_count = other_dept_visits.groupby(['PATIENTHASHMRN', 'DEPTSPECIALTY']).size().reset_index(name='Visits')

detailed_visits_count = other_dept_visits.groupby(['PATIENTHASHMRN', 'DEPTSPECIALTY']).size().reset_index(name='Visits')

# The departments and number of time visited by patients who visited emergency services
dept_visits_matrix = other_dept_visits.pivot_table(index='PATIENTHASHMRN', columns='DEPTSPECIALTY', aggfunc='size', fill_value=0)

dept_visits_matrix = dept_visits_matrix.merge(total_emergency_visits_df, how='left', left_index=True, right_on='PATIENTHASHMRN')

# Set PATIENTHASHMRN back as the index if it's been reset due to the merge
dept_visits_matrix.set_index('PATIENTHASHMRN', inplace=True)

# Fill NaN values that might have occurred if there are patients with no emergency visits recorded
dept_visits_matrix['Total Emergency Visits'].fillna(0, inplace=True)

# Convert the 'Total Emergency Visits' column to integers to clean up the data presentation
dept_visits_matrix['Total Emergency Visits'] = dept_visits_matrix['Total Emergency Visits'].astype(int)

dept_visits_matrix = dept_visits_matrix.sort_values(by='Total Emergency Visits', ascending=False)

# Reset the index to turn the index (PATIENTHASHMRN) into a column
dept_visits_matrix = dept_visits_matrix.reset_index()

other_dept_visits_matrix = pd.merge(dept_visits_matrix, df_FINDEMOG, on='PATIENTHASHMRN', how='inner')


In [ ]:
other_dept_visits_matrix

In [ ]:
tempList = other_dept_visits_matrix.PATIENTHASHMRN
Onlyemergency_dept_visits_matrix = emergency_dept_visits_matrix[~emergency_dept_visits_matrix['PATIENTHASHMRN'].isin(tempList)]
                                                                

In [ ]:
other_dept_visits_matrix.to_csv('otherdepartmentVisits.csv')
emergency_dept_visits_matrix.to_csv('EmergencyDepartmentVisits.csv')
Onlyemergency_dept_visits_matrix.to_csv('OnlyEmergencyDepartmentVisits.csv')

p1 = document.add_paragraph("The total number of patients seen by emergency department : ")
p1.add_run(str(len(emergency_dept_visits_matrix)))

p2 = document.add_paragraph("The number of patients seen by only emergency department.  These patients only accessed medical services through emergency Services :")
p2.add_run(str(len(Onlyemergency_dept_visits_matrix)))

p3 = document.add_paragraph("The number of patients seen by  emergency department but also by other departments :")
p3.add_run(str(len(other_dept_visits_matrix)))


In [ ]:
Onlyemergency_dept_visits_matrix.columns

In [ ]:
document = Document()
# Group by FINANCIALCLASS and Total Emergency Visits, then count the patients
sns.set(font_scale=1.1)    
financialclass_category_order =['Medi-Cal Managed Care','Medi-Cal','Commercial','Self-Pay','Medicare HMO/PPO','Medicare','Tricare','Managed Care','Worker\'s Comp','Other']

patient_counts = Onlyemergency_dept_visits_matrix.groupby(['FINANCIALCLASS', 'Total Emergency Visits']).size().reset_index(name='Patient_Count')
patient_counts['FINANCIALCLASS'] = pd.Categorical(patient_counts['FINANCIALCLASS'], categories=financialclass_category_order, ordered=True)

#bubble_plot = sns.scatterplot(data=patient_counts, y='Total Emergency Visits', x='FINANCIALCLASS', size='Patient_Count', sizes=(30, 500), alpha=0.8, legend=False)
patient_matrix = patient_counts.pivot(index='FINANCIALCLASS', columns='Total Emergency Visits', values='Patient_Count').fillna(0)

## Calculate column-wise percentages
column_totals = patient_matrix.sum(axis=0)  # Sum each column
percent_matrix = patient_matrix.div(column_totals, axis=1) * 100  # Calculate percentage for each cell

# Create annotation text showing both counts and percentages
annotations = patient_matrix.applymap(int).astype(str) + "\n(" + percent_matrix.round(1).astype(str) + "%)"

plt.figure(figsize=(6, 4))
sns.heatmap(
    patient_matrix,
    annot=annotations,
    fmt="",
    cmap='YlGnBu',
    linewidths=0.5,
    cbar_kws={'label': 'Patient Count'},
    annot_kws={"size": 10}  # Adjust annotation font size
)

#plt.title('Total Emergency Visits vs. Financial Class - Only for patients visiting emergency service')
plt.ylabel('Financial Class (Insurance)')
#plt.xlabel('Financial Class')
plt.xlabel('Total of  Emergency Visits')
plt.grid(True)  # Add grid for better readability

# Adjusting y-axis labels for better readability if necessary
plt.xticks(rotation=45, ha='right')
plt.savefig('temp.png', bbox_inches='tight')


document.add_picture('temp.png', width=Inches(6))
document.add_paragraph("Figure XX:Financial Class versus Total Emergency Visits (For Patients visited only Emergency Services)")

plt.show()
# Pivot the patient_counts DataFrame to create the matrix
patient_matrix = patient_counts.pivot(index='FINANCIALCLASS', columns='Total Emergency Visits', values='Patient_Count').fillna(0)
doctable(patient_matrix, "Financial Class versus Total Emergency Visits (For Patients visited only Emergency Services) ",['Financial Class','1','2', '3', '4', '5', '6', '8', '9', '12', '14'])


# Display the resulting matrix
print(patient_matrix)

In [ ]:
patient_matrix

In [ ]:
# Group by FINANCIALCLASS and Total Emergency Visits, then count the patients
sns.set(font_scale=1.1)   
income_category_order = [' Very Low', 'Low', 'Median','High', 'Very High', 'Unknown']
#income_category_order = [ 'Low', 'Median','High', 'Very High', 'Unknown']

patient_counts = Onlyemergency_dept_visits_matrix.groupby(['INCOME_CATEGORY', 'Total Emergency Visits']).size().reset_index(name='Patient_Count')

# Convert INCOME_CATEGORY to a categorical type with a defined order
patient_counts['INCOME_CATEGORY'] = pd.Categorical(patient_counts['INCOME_CATEGORY'], categories=income_category_order, ordered=True)


#bubble_plot = sns.scatterplot(data=patient_counts, y='Total Emergency Visits', x='INCOME_CATEGORY', size='Patient_Count', sizes=(30, 500), alpha=0.8, legend=False)

# Create the pivot table for the heatmap
patient_matrix = patient_counts.pivot(index='INCOME_CATEGORY', columns='Total Emergency Visits', values='Patient_Count').fillna(0)

## Calculate column-wise percentages
column_totals = patient_matrix.sum(axis=0)  # Sum each column
percent_matrix = patient_matrix.div(column_totals, axis=1) * 100  # Calculate percentage for each cell

# Create annotation text showing both counts and percentages
annotations = patient_matrix.applymap(int).astype(str) + "\n(" + percent_matrix.round(1).astype(str) + "%)"

plt.figure(figsize=(6, 4))
sns.heatmap(
    patient_matrix,
    annot=annotations,
    fmt="",
    cmap='YlGnBu',
    linewidths=0.5,
    cbar_kws={'label': 'Patient Count'},
    annot_kws={"size": 10}  # Adjust annotation font size
)
#plt.title('Total Emergency Visits vs. Income Category')
plt.xlabel('Total Emergency Visits')
#plt.xlabel('Income Category')
plt.ylabel('Income Category')

plt.grid(True)  # Add grid for better readability

# Adjusting y-axis labels for better readability if necessary
plt.xticks(rotation=45, ha='right')
plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
document.add_paragraph("Figure XX:Income Category versus Total Emergency Visits (For Patients visited only Emergency Services)")

plt.show()

# Pivot the patient_counts DataFrame to create the matrix
patient_matrix = patient_counts.pivot(index='INCOME_CATEGORY', columns='Total Emergency Visits', values='Patient_Count').fillna(0)
doctable(patient_matrix, "Income Category versus Total Emergency Visits (For Patients visited only Emergency Services)",['Income Category','1','2', '3', '4', '5', '6', '8', '9', '12', '14'])

# Display the resulting matrix
print(patient_matrix)

In [ ]:
Onlyemergency_dept_visits_matrix.columns

In [ ]:
Onlyemergency_dept_visits_matrix

In [ ]:
#patient_counts_check = patientRecordswithOnlyEmergencyVisit.groupby(['PATIENTHASHMRN', 'Total Emergency Visits']).size().reset_index(name='Patient_Count')


In [ ]:
#patientRecordswithOnlyEmergencyVisit.columns

In [ ]:
# Group by 'PATIENTHASHMRN' and 'ICD10CODE', then count the size of each group
patientswithOnlyEmergencyVisit = Onlyemergency_dept_visits_matrix.PATIENTHASHMRN
patientRecordswithOnlyEmergencyVisit = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['PATIENTHASHMRN'].isin(patientswithOnlyEmergencyVisit)]
patientRecordswithOnlyEmergencyVisit = patientRecordswithOnlyEmergencyVisit.merge(Onlyemergency_dept_visits_matrix[['PATIENTHASHMRN', 'Total Emergency Visits']], on='PATIENTHASHMRN', how='left')


# Define the priority order for ICD10Code
priority_order = ['J45.5x', 'J45.4x', 'J45.3x', 'J45.2x']

# Create a mapping dictionary to assign a rank to each ICD10Code
priority_map = {code: rank for rank, code in enumerate(priority_order)}

# Assign a priority rank to each record based on ICD10Code
patientRecordswithOnlyEmergencyVisit['PriorityRank'] = patientRecordswithOnlyEmergencyVisit['ICD10CODE'].map(priority_map)

# For records where ICD10Code is not in the priority list, assign a high rank
patientRecordswithOnlyEmergencyVisit['PriorityRank'] = patientRecordswithOnlyEmergencyVisit['PriorityRank'].fillna(len(priority_order))

# Sort the DataFrame by patient ID and PriorityRank, and keep the first record for each patient
deduplicated_df = (
    patientRecordswithOnlyEmergencyVisit.sort_values(by=['PATIENTHASHMRN', 'PriorityRank'])
    .drop_duplicates(subset=['PATIENTHASHMRN'], keep='first')
    .drop(columns=['PriorityRank'])  # Remove the PriorityRank column if not needed
)


patientRecordswithOnlyEmergencyVisit = deduplicated_df

patientRecordswithOnlyEmergencyVisit.to_excel('yikil.xlsx')


# Group by FINANCIALCLASS and Total Emergency Visits, then count the patients
sns.set(font_scale=1.1)   

ICD10_category_order = ['J45.5x','J45.4x','J45.3x','J45.2x']

# Group by FINANCIALCLASS and Total Emergency Visits, then count the patients
sns.set(font_scale=1.1)   
#income_category_order = [ 'Low', 'Median','High', 'Very High', 'Unknown']

patient_counts = patientRecordswithOnlyEmergencyVisit.groupby(['ICD10CODE', 'Total Emergency Visits']).size().reset_index(name='Patient_Count')
# Convert INCOME_CATEGORY to a categorical type with a defined order
patient_counts['ICD10CODE'] = pd.Categorical(patient_counts['ICD10CODE'], categories=ICD10_category_order, ordered=True)


#bubble_plot = sns.scatterplot(data=patient_counts, y='Total Emergency Visits', x='INCOME_CATEGORY', size='Patient_Count', sizes=(30, 500), alpha=0.8, legend=False)

# Create the pivot table for the heatmap
patient_matrix = patient_counts.pivot(index='ICD10CODE', columns='Total Emergency Visits', values='Patient_Count').fillna(0)

## Calculate column-wise percentages
column_totals = patient_matrix.sum(axis=0)  # Sum each column
percent_matrix = patient_matrix.div(column_totals, axis=1) * 100  # Calculate percentage for each cell

# Create annotation text showing both counts and percentages
annotations = patient_matrix.applymap(int).astype(str) + "\n(" + percent_matrix.round(1).astype(str) + "%)"

plt.figure(figsize=(6, 4))
sns.heatmap(
    patient_matrix,
    annot=annotations,
    fmt="",
    cmap='YlGnBu',
    linewidths=0.5,
    cbar_kws={'label': 'Patient Count'},
    annot_kws={"size": 10}  # Adjust annotation font size
)
#plt.title('Total Emergency Visits vs. Income Category')
plt.xlabel('Total Emergency Visits')
#plt.xlabel('Income Category')
plt.ylabel('ICD10 Code')

plt.grid(True)  # Add grid for better readability

# Adjusting y-axis labels for better readability if necessary
plt.xticks(rotation=45, ha='right')
plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
document.add_paragraph("Figure XX:ICD10 Code for Diagnoses versus Total Emergency Visits (For Patients visited only Emergency Services)")

plt.show()

# Pivot the patient_counts DataFrame to create the matrix
patient_matrix = patient_counts.pivot(index='ICD10CODE', columns='Total Emergency Visits', values='Patient_Count').fillna(0)
doctable(patient_matrix, "Figure XX:ICD10 Code for Diagnosesversus Total Emergency Visits (For Patients visited only Emergency Services)",['Income Category','1','2', '3', '4', '5', '6', '8', '9', '12', '14'])

# Display the resulting matrix
print(patient_matrix)




In [ ]:
Onlyemergency_dept_visits_matrix

In [ ]:
# Group by FINANCIALCLASS and Total Emergency Visits, then count the patients
sns.set(font_scale=1.1)   

HHSRegion_category_order = ['South', 'Central', 'East','North Coastal', 'North Central', 'North Inland','Other']

# Group by FINANCIALCLASS and Total Emergency Visits, then count the patients
sns.set(font_scale=1)   
#income_category_order = [ 'Low', 'Median','High', 'Very High', 'Unknown']

patient_counts = Onlyemergency_dept_visits_matrix.groupby(['HHSREGION', 'Total Emergency Visits']).size().reset_index(name='Patient_Count')
# Convert INCOME_CATEGORY to a categorical type with a defined order
patient_counts['HHSREGION'] = pd.Categorical(patient_counts['HHSREGION'], categories=HHSRegion_category_order, ordered=True)


#bubble_plot = sns.scatterplot(data=patient_counts, y='Total Emergency Visits', x='INCOME_CATEGORY', size='Patient_Count', sizes=(30, 500), alpha=0.8, legend=False)

# Create the pivot table for the heatmap
patient_matrix = patient_counts.pivot(index='HHSREGION', columns='Total Emergency Visits', values='Patient_Count').fillna(0)

## Calculate column-wise percentages
column_totals = patient_matrix.sum(axis=0)  # Sum each column
percent_matrix = patient_matrix.div(column_totals, axis=1) * 100  # Calculate percentage for each cell

# Create annotation text showing both counts and percentages
annotations = patient_matrix.applymap(int).astype(str) + "\n(" + percent_matrix.round(1).astype(str) + "%)"

plt.figure(figsize=(6, 4))
sns.heatmap(
    patient_matrix,
    annot=annotations,
    fmt="",
    cmap='YlGnBu',
    linewidths=0.5,
    cbar_kws={'label': 'Patient Count'},
    annot_kws={"size": 10}  # Adjust annotation font size
)
#plt.title('Total Emergency Visits vs. Income Category')
plt.xlabel('Total Emergency Visits')
#plt.xlabel('Income Category')
plt.ylabel('Health Services Region')

plt.grid(True)  # Add grid for better readability

# Adjusting y-axis labels for better readability if necessary
plt.xticks(rotation=45, ha='right')
plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
document.add_paragraph("Figure XX:Health Services Region  versus Total Emergency Visits (For Patients visited only Emergency Services)")

plt.show()

# Pivot the patient_counts DataFrame to create the matrix
patient_matrix = patient_counts.pivot(index='HHSREGION', columns='Total Emergency Visits', values='Patient_Count').fillna(0)
doctable(patient_matrix, "Health Services Region  versus Total Emergency Visits (For Patients visited only Emergency Services)",['Income Category','1','2', '3', '4', '5', '6', '8', '9', '12', '14'])

# Display the resulting matrix
print(patient_matrix)










In [ ]:
patientRecordswithOnlyEmergencyVisit.columns

In [ ]:
#emergency_visits_df = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']
emergency_visits_df =patientRecordswithOnlyEmergencyVisit

# Step 2 and 3: Group by Ethnicity and Race, and count visits
Emergencyvisits_counts = emergency_visits_df.groupby(['RACE']).size().reset_index(name='Visits')

# Step 4: Calculate total emergency department visits
total_emergency_visits = Emergencyvisits_counts['Visits'].sum()

# Step 5: Calculate percentages
Emergencyvisits_counts['Percentage'] = (Emergencyvisits_counts['Visits'] / total_emergency_visits) * 100



# Step 1 and 2: Group by Race and count entries
visit_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.groupby('RACE').size().reset_index(name='Count')

# Step 3: Calculate total entries in the dataset
total_entries = visit_counts['Count'].sum()

# Step 4: Calculate percentages
visit_counts['Percentage'] = (visit_counts['Count'] / total_entries) * 100

doctable(Emergencyvisits_counts, "Race stats of patients who visited only emergency for access to medical care",['index','Race','Count','Percentage'])
doctable(visit_counts, "Race stats of patients",['index','Race','Count','Percentage'])


print(Emergencyvisits_counts)
print(visit_counts)



In [ ]:
#emergency_visits_df = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']
emergency_visits_df =patientRecordswithOnlyEmergencyVisit

# Step 2 and 3: Group by Ethnicity and Race, and count visits
Emergencyvisits_counts = emergency_visits_df.groupby(['ETHNICITY']).size().reset_index(name='Visits')

# Step 4: Calculate total emergency department visits
total_emergency_visits = Emergencyvisits_counts['Visits'].sum()

# Step 5: Calculate percentages
Emergencyvisits_counts['Percentage'] = (Emergencyvisits_counts['Visits'] / total_emergency_visits) * 100



# Step 1 and 2: Group by Race and count entries
visit_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.groupby('ETHNICITY').size().reset_index(name='Count')

# Step 3: Calculate total entries in the dataset
total_entries = visit_counts['Count'].sum()

# Step 4: Calculate percentages
visit_counts['Percentage'] = (visit_counts['Count'] / total_entries) * 100

doctable(Emergencyvisits_counts, "Ethnicity stats of patients who visited only emergency for access to medical care",['index','Ethnicity','Count','Percentage'])
doctable(visit_counts, "Ethnicity stats of patients",['index','Ethnicity','Count','Percentage'])


print(Emergencyvisits_counts)
print(visit_counts)



In [ ]:
#emergency_visits_df = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']
emergency_visits_df =patientRecordswithOnlyEmergencyVisit

# Step 2 and 3: Group by Ethnicity and Race, and count visits
Emergencyvisits_counts = emergency_visits_df.groupby(['INCOME_CATEGORY']).size().reset_index(name='Visits')

# Step 4: Calculate total emergency department visits
total_emergency_visits = Emergencyvisits_counts['Visits'].sum()

# Step 5: Calculate percentages
Emergencyvisits_counts['Percentage'] = (Emergencyvisits_counts['Visits'] / total_emergency_visits) * 100



# Step 1 and 2: Group by Race and count entries
visit_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.groupby('INCOME_CATEGORY').size().reset_index(name='Count')

# Step 3: Calculate total entries in the dataset
total_entries = visit_counts['Count'].sum()

# Step 4: Calculate percentages
visit_counts['Percentage'] = (visit_counts['Count'] / total_entries) * 100

doctable(Emergencyvisits_counts, "INCOME_CATEGORY stats of patients who visited only emergency for access to medical care",['index','INCOME_CATEGORY','Count','Percentage'])
doctable(visit_counts, "INCOME_CATEGORY stats of patients",['index','INCOME_CATEGORY','Count','Percentage'])


print(Emergencyvisits_counts)
print(visit_counts)



In [ ]:

patient_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.groupby(['DEPTLOCATION', 'INCOME_CATEGORY']).size().reset_index(name='PatientCount')

# Assuming 'df_DIAGPROC_FINDEMOG_ENCFilter' is your filtered DataFrame

# Step 1: Aggregate the data to count the number of patients for each combination of HHSREGION and INCOME_CATEGORY
# This step remains the same.

# Step 2: Correctly pivot the data using pivot_table method
pivot_table = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.pivot_table(index='DEPTLOCATION', columns='INCOME_CATEGORY', aggfunc='size', fill_value=0)

# Step 3: Create the heatmap with the corrected pivot table
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=True, fmt="d", cmap="YlGnBu", linewidths=1,
            annot_kws={"size": 8})  # Adjusting font size here
#plt.title('Patient Counts by Income Category and Facility Used', fontsize=8)
plt.ylabel('Facility Region', fontsize=6)
#plt.xlabel('Income Category', fontsize=6)
plt.xlabel('')

plt.xticks(fontsize=8)  # Adjusting x-axis label font sizes
plt.yticks(fontsize=8)  # Adjusting y-axis label font sizes
plt.tight_layout()
plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
document.add_paragraph("Figure XX:Patient Counts by Income Category and  Health Facility Used  ")

plt.show()


In [ ]:
sns.set_context("paper", font_scale=0.8)
# Step 1: Aggregate the data to count the number of patients for each combination of HHSREGION and INCOME_CATEGORY
patient_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.groupby(['HHSREGION', 'INCOME_CATEGORY']).size().reset_index(name='PatientCount')

# First, create a pivot table with counts
pivot_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.pivot_table(index='HHSREGION', columns='INCOME_CATEGORY', aggfunc='size', fill_value=0)

# Calculate column totals
column_totals = pivot_counts.sum(axis=0)

# Convert counts to percentages of each column total
pivot_percentages_column = (pivot_counts.div(column_totals, axis=1)) * 100

# Plotting
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_percentages_column, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=.5)
#plt.title('Percentage of Patients within Each Income Category by HHS Region', fontweight='bold', fontsize=8)
plt.ylabel('HHS Region', fontsize=8)
#plt.xlabel('Income Category',fontsize=8)
plt.xlabel('')

plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
document.add_paragraph("Figure XX:Percentage of Patients within Each Income Category by  HHSREGION    ")

plt.show()

doctable(pivot_counts, "HHS Region stats of patients",['HHSREGION',' reg1', 'reg2', 'reg3', 'reg4', 'reg5','reg6'])


In [ ]:
sns.set_context("paper", font_scale=0.8)
# Step 1: Aggregate the data to count the number of patients for each combination of HHSREGION and INCOME_CATEGORY
patient_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.groupby(['ETHNICITY', 'HHSREGION']).size().reset_index(name='PatientCount')

# First, create a pivot table with counts
pivot_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.pivot_table(index='HHSREGION', columns='ETHNICITY', aggfunc='size', fill_value=0)

# Calculate column totals
column_totals = pivot_counts.sum(axis=0)

# Convert counts to percentages of each column total
pivot_percentages_column = (pivot_counts.div(column_totals, axis=1)) * 100

# Plotting
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_percentages_column, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=.5)
#plt.title('Percentage of Patients in each HHSRegion versus Ethnicity', fontweight='bold', fontsize=8)
plt.ylabel('HHS Region', fontsize=8)
#plt.xlabel('RACE',fontsize=8)
plt.xlabel('')

plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
document.add_paragraph("Figure XX:Percentage of Patients in each HHSRegion versus Ethnicity   ")

plt.show()

In [ ]:
sns.set_context("paper", font_scale=0.8)
# Step 1: Aggregate the data to count the number of patients for each combination of HHSREGION and INCOME_CATEGORY
patient_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.groupby(['HHSREGION','RACE']).size().reset_index(name='PatientCount')

# First, create a pivot table with counts
pivot_counts = df_DIAGPROC_FINDEMOG_ENCFiltered_Unique.pivot_table(index='HHSREGION', columns='RACE', aggfunc='size', fill_value=0)

# Calculate column totals
column_totals = pivot_counts.sum(axis=0)

# Convert counts to percentages of each column total
pivot_percentages_column = (pivot_counts.div(column_totals, axis=1)) * 100

# Plotting
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_percentages_column, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=.5)
#plt.title('Percentage of Patients in each HHSRegion versus Race', fontweight='bold', fontsize=8)
plt.ylabel('HHS Region', fontsize=8)
#plt.xlabel('RACE',fontsize=8)
plt.xlabel('')

plt.savefig('temp.png', bbox_inches='tight')


document.add_picture('temp.png', width=Inches(6))
document.add_paragraph("Figure XX:Percentage of Patients in each HHSRegion versus Race   ")

plt.show()

In [ ]:
fsize = 14
width = 7
height = 5

#Calucate Odds Radio and Disparity Index
df_DIAGPROC_FINDEMOG_ENCFiltered['IsEmergencyVisit'] = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine'

def calculate_index_of_disparity(Category, df):

    # Step 1: Filter the data for emergency visits
    emergency_visits = df[df['IsEmergencyVisit'] == 1]

    # Step 2: Calculate the total number of emergency visits for each race
    visits_by_race = emergency_visits.groupby(Category).agg(
        EDVisits=pd.NamedAgg(column='IsEmergencyVisit', aggfunc='sum')
    ).reset_index()

    # Step 3: Calculate the proportion of emergency visits for each race
    total_ed_visits = visits_by_race['EDVisits'].sum()
    visits_by_race['EDVisitProportion'] = visits_by_race['EDVisits'] / total_ed_visits

    # Step 4: Calculate the overall average proportion of emergency visits
    overall_average = visits_by_race['EDVisitProportion'].mean()

    # Step 5: Calculate the absolute deviation from the average for each race
    visits_by_race['AbsoluteDeviation'] = abs((visits_by_race['EDVisitProportion'] - overall_average) / overall_average)

    # Step 6: Calculate the Index of Disparity
    index_of_disparity = (visits_by_race['AbsoluteDeviation'].mean()) * 100

    return index_of_disparity


def EmergencyVisitsPerCategory(Category):
    # Step 2: Calculate Total and Emergency Visits by Ethnicity
    visits_by_Category = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby(Category).agg(
        TotalVisits=pd.NamedAgg(column='IsEmergencyVisit', aggfunc='count'),
        EDVisits=pd.NamedAgg(column='IsEmergencyVisit', aggfunc='sum')
    ).reset_index()

    # Step 3: Calculate Odds for Each Race
    visits_by_Category['NonEDVisits'] = visits_by_Category['TotalVisits'] - visits_by_Category['EDVisits']
    visits_by_Category['Odds'] = visits_by_Category['EDVisits'] / visits_by_Category['NonEDVisits']

    # Step 4: Calculate Total Population and ED Visits Proportion for Disparity Index
    total_population = visits_by_Category['TotalVisits'].sum()
    total_ed_visits = visits_by_Category['EDVisits'].sum()

    visits_by_Category['PopulationProportion'] = visits_by_Category['TotalVisits'] / total_population
    visits_by_Category['EDVisitsProportion'] = visits_by_Category['EDVisits'] / total_ed_visits

    # Step 5: Calculate Disparity Index for Each Race
    visits_by_Category['DisparityIndex'] = visits_by_Category['EDVisitsProportion'] / visits_by_Category['PopulationProportion']

    # Note: Calculating Odds Ratio (OR) for each race against all others directly is less straightforward
    # because it involves comparing each race to the aggregation of all others.
    # For simplicity, we calculate Odds and Disparity Index, which already provide significant insights.
    visits_by_Category['AbsoluteDifference'] = abs(visits_by_Category['EDVisitsProportion'] - visits_by_Category['PopulationProportion'])
        
    # Step 4: Sum the absolute differences and divide by 2
    index_of_dissimilarity = (visits_by_Category['AbsoluteDifference'].sum() / 2) * 100  # Convert to percentage
      
    
    # Step 5: Calculate Disparity Index for Each Race
    
    
    index_of_disparity = calculate_index_of_disparity(Category, df_DIAGPROC_FINDEMOG_ENCFiltered)
    
    paragraph = document.add_paragraph('index of dissimilarity :')
    paragraph.add_run(f"{index_of_dissimilarity:.4f}%")
    paragraph = document.add_paragraph('index of disparity :')
    paragraph.add_run(f"{index_of_disparity:.4f}%")

    print('index of dissimilarity :',index_of_dissimilarity, '\n index of disparity :',index_of_disparity, )

    
    # Display the final DataFrame
    visits_by_Category[[Category, 'Odds', 'DisparityIndex']]

    visits_by_Category = visits_by_Category.sort_values(by='PopulationProportion', ascending=True)
    
    # Step 2: Compute cumulative proportions
    visits_by_Category['CumulativePopulation'] = visits_by_Category['PopulationProportion'].cumsum()
    visits_by_Category['CumulativeEDVisits'] = visits_by_Category['EDVisitsProportion'].cumsum()

    concentration_curve_x = visits_by_Category['CumulativePopulation']
    concentration_curve_y = visits_by_Category['CumulativeEDVisits']
    
    print(visits_by_Category)

    # Step 4: Plot the Concentration Curve
    plt.figure(figsize=(width,height))
    plt.plot(concentration_curve_x, concentration_curve_y, label='Concentration Curve', marker='o', linestyle='-')
    plt.plot([0, 1], [0, 1], color='k', linestyle='--', label='Line of Equality')  # Diagonal line representing perfect equality
    #plt.xlabel('Cumulative Proportion of Population')
    plt.xlabel('')
    plt.grid(False)  # Completely turn off the grid
    plt.ylabel('Emergency Visit Cumulative %',fontsize=fsize)
    #plt.title('Concentration Curve for Emergency Visits by '+ Category)
    plt.legend(fontsize=fsize)
    plt.grid(True)
    plt.xticks(fontsize=fsize)
    plt.yticks(fontsize=fsize)
    #plt.savefig('temp.png')    

    plt.savefig('temp.png', bbox_inches='tight')
    plt.show()

    document.add_picture('temp.png', width=Inches(width))
    
    paragraph =document.add_paragraph("Figure XX:Concentration Curve for Emergency Visits by ")
    paragraph.add_run(Category)

    visits_by_Category['PercentageEmergencyVisits'] = (visits_by_Category['EDVisits'] / visits_by_Category['TotalVisits']) * 100

     # Calculate the maximum Disparity Index
    maximum_disparity = visits_by_Category['DisparityIndex'].max()

    # Print the data to be plotted
    print("Visit by category")
    print(visits_by_Category)

    # Plotting
    fig, ax1 = plt.subplots(figsize=(width,height))

    # Plot the bar chart for Percentage of Emergency Visits
    sns.barplot(x=Category, y='PercentageEmergencyVisits', data=visits_by_Category, palette='coolwarm', ax=ax1)

    # Set axis labels and increase font size
    ax1.set_ylabel('Emergency Visits Percentage', fontsize=fsize)
    ax1.set_xlabel('')
    ax1.tick_params(axis='both', which='major', labelsize=fsize)
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')

    # Add percentage labels on top of the bars
    for p in ax1.patches:
        ax1.annotate(f'{p.get_height():.1f}%', 
                     (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha='center', va='center', fontsize=fsize, color='black', 
                     xytext=(0, 5), textcoords='offset points')

    # Create a secondary axis for the Disparity Index
    ax2 = ax1.twinx()
    ax2.set_ylabel('Disparity Index', fontsize=fsize, color='blue')
    ax2.set_ylim(0, maximum_disparity + 1)  # Set the y-axis limit to maximum_disparity + 2
    ax2.tick_params(axis='y', labelsize=fsize)  # Change y-ticks font size for the primary axis


    # Plot Disparity Index as points
    ax2.scatter(visits_by_Category[Category], visits_by_Category['DisparityIndex'], color='blue', s=6, zorder=5, label='Disparity Index')

    # Add labels to each point for Disparity Index
    for i, (x, y) in enumerate(zip(visits_by_Category[Category], visits_by_Category['DisparityIndex'])):
        ax2.annotate(f'{y:.2f}', (i, y), textcoords="offset points", xytext=(10, 0),  fontsize=fsize, ha='left', color='blue')

    # Adjust layout and show the plot
    plt.grid(False)  # Completely turn off the grid
    plt.tight_layout()
    plt.legend(loc="upper right", fontsize=fsize)  # Add legend for Disparity Index points
    plt.savefig('temp.png', bbox_inches='tight')

    document.add_picture('temp.png', width=Inches(width))
    paragraph=document.add_paragraph("Figure XX: Percentage of  Emergency Visits by ")
    paragraph.add_run(Category)    
    plt.show()


        # Plotting
    fig, ax1 = plt.subplots(figsize=(width,height))

    # Plot the bar chart for Percentage of Emergency Visits
    sns.barplot(x=Category, y='DisparityIndex', data=visits_by_Category, palette='coolwarm', ax=ax1)

    # Set axis labels and increase font size
    ax1.set_ylabel('Disparity Index', fontsize=fsize)
    ax1.set_xlabel('')
    ax1.tick_params(axis='both', which='major', labelsize=fsize)
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')

    # Add percentage labels on top of the bars
    for p in ax1.patches:
        ax1.annotate(f'{p.get_height():.1f}', 
                     (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha='center', va='center', fontsize=fsize, color='black', 
                     xytext=(0, 5), textcoords='offset points')

    # Adjust layout and show the plot
    plt.grid(False)  # Completely turn off the grid
    plt.tight_layout()
    plt.savefig('temp.png', bbox_inches='tight')

    document.add_picture('temp.png', width=Inches(width))
    paragraph=document.add_paragraph("Figure XX: Percentage of  Emergency Visits by ")
    paragraph.add_run(Category)    
    plt.show()

    
    

    # Step 4: Calculate Gini Coefficient
    def gini_coefficient(x, y):
        n = len(x)
        area_below_curve = np.trapz(y, x)
        area_perfect_equality = 0.5  # Area under the diagonal line
        gini = (area_perfect_equality - area_below_curve) / area_perfect_equality
        return gini

    gini = gini_coefficient(visits_by_Category['CumulativeEDVisits'], visits_by_Category['CumulativePopulation'], )
    paragraph=document.add_paragraph('Gini Coefficient :')
    paragraph.add_run(str(gini))
    print('Gini Coefficient :',gini)

    return(visits_by_Category)

visits_by_race = EmergencyVisitsPerCategory('RACE')
visits_by_race = visits_by_race.set_index('RACE')
visits_by_race = visits_by_race.drop(['CumulativePopulation', 'CumulativeEDVisits'], axis=1)
doctable(visits_by_race,"Emergency Visit By Race",['RACE', 'TotalVisits', 'EDVisits', 'NonEDVisits', 'Odds','PopulationProportion', 'EDVisitsProportion', 'DisparityIndex','PercentageEmergencyVisits','AbsoluteDifference'])

visit_by_HHSREGION = EmergencyVisitsPerCategory('HHSREGION')
visit_by_HHSREGION =visit_by_HHSREGION .set_index('HHSREGION')
visit_by_HHSREGION = visit_by_HHSREGION.drop(['CumulativePopulation', 'CumulativeEDVisits'], axis=1)

doctable(visit_by_HHSREGION,"Emergency Visits By REGION",['HHSREGION', 'TotalVisits', 'EDVisits', 'NonEDVisits', 'Odds','PopulationProportion', 'EDVisitsProportion', 'DisparityIndex','PercentageEmergencyVisits','AbsoluteDifference'])


visit_by_FINANCIALCLASS = EmergencyVisitsPerCategory('FINANCIALCLASS')
visit_by_FINANCIALCLASS = visit_by_FINANCIALCLASS.set_index('FINANCIALCLASS')
visit_by_FINANCIALCLASS = visit_by_FINANCIALCLASS.drop(['CumulativePopulation', 'CumulativeEDVisits'], axis=1)

doctable(visit_by_FINANCIALCLASS,"Emergency Visits By Financial Clas",['FINANCIALCLASS', 'TotalVisits', 'EDVisits', 'NonEDVisits', 'Odds','PopulationProportion', 'EDVisitsProportion', 'DisparityIndex','PercentageEmergencyVisits','AbsoluteDifference'])

visit_by_INCOME_CATEGORY = EmergencyVisitsPerCategory('INCOME_CATEGORY')
visit_by_INCOME_CATEGORY=visit_by_INCOME_CATEGORY.set_index('INCOME_CATEGORY')
visit_by_INCOME_CATEGORY = visit_by_INCOME_CATEGORY.drop(['CumulativePopulation', 'CumulativeEDVisits'], axis=1)

doctable(visit_by_INCOME_CATEGORY,"Emergency Visits By Income Category",['INCOME_CATEGORY', 'TotalVisits', 'EDVisits', 'NonEDVisits', 'Odds','PopulationProportion', 'EDVisitsProportion', 'DisparityIndex','PercentageEmergencyVisits','AbsoluteDifference'])

visit_by_AGE_RANGE = EmergencyVisitsPerCategory('AGE_RANGE')
visit_by_AGE_RANGE = visit_by_AGE_RANGE.set_index('AGE_RANGE')
visit_by_AGE_RANGE = visit_by_AGE_RANGE.drop(['CumulativePopulation', 'CumulativeEDVisits'], axis=1)

doctable(visit_by_AGE_RANGE,"Emergency Visits By Age Range",['AGE_RANGE', 'TotalVisits', 'EDVisits', 'NonEDVisits', 'Odds','PopulationProportion', 'EDVisitsProportion', 'DisparityIndex','PercentageEmergencyVisits','AbsoluteDifference'])

visit_by_ETHNICITY = EmergencyVisitsPerCategory('ETHNICITY')
visit_by_ETHNICITY = visit_by_ETHNICITY.set_index('ETHNICITY')
visit_by_ETHNICITY = visit_by_ETHNICITY.drop(['CumulativePopulation', 'CumulativeEDVisits'], axis=1)

doctable(visit_by_ETHNICITY,"Emergency Visits By ETHNICITY",['ETHNICITY', 'TotalVisits', 'EDVisits', 'NonEDVisits', 'Odds','PopulationProportion', 'EDVisitsProportion', 'DisparityIndex','PercentageEmergencyVisits','AbsoluteDifference'])



In [ ]:
visit_by_ETHNICITY

In [ ]:
df_DIAGPROC_FINDEMOG_ENCFiltered.columns

In [ ]:
pivot_table = df_DIAGPROC_FINDEMOG_ENCFiltered.pivot_table(index='DEPTSPECIALTY', columns='TYPE', aggfunc='size', fill_value=0)

# Plotting the heatmap
plt.figure(figsize=(12, 8))  # Adjust the figure size as needed
sns.heatmap(pivot_table, annot=True, fmt="d", cmap="YlGnBu", linewidths=.5)

# Set labels and title
plt.xlabel('TYPE')
plt.ylabel('DEPTSPECIALTY')
plt.title('Heatmap of TYPE against DEPTSPECIALTY')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
fsize = 14
width = 7
height = 5
#Calucate Odds Radio and Disparity Index
specialistList =['Allergy and Immunology','Pulmonology']
visitType = ['Hospital Encounter']


df_DIAGPROC_FINDEMOG_ENCFiltered['IsspecialistVisit'] = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].isin(specialistList)
df_DIAGPROC_FINDEMOG_ENCFiltered['TYPE'] = df_DIAGPROC_FINDEMOG_ENCFiltered['TYPE'].isin(visitType)

def calculate_index_of_disparity(Category, df):

    # Step 1: Filter the data for emergency visits
    emergency_visits = df[df['IsspecialistVisit'] == 1]

    # Step 2: Calculate the total number of emergency visits for each race
    visits_by_race = emergency_visits.groupby(Category).agg(
        SDVisits=pd.NamedAgg(column='IsspecialistVisit', aggfunc='sum')
    ).reset_index()

    # Step 3: Calculate the proportion of emergency visits for each race
    total_ed_visits = visits_by_race['SDVisits'].sum()
    visits_by_race['SDVisitProportion'] = visits_by_race['SDVisits'] / total_ed_visits

    # Step 4: Calculate the overall average proportion of emergency visits
    overall_average = visits_by_race['SDVisitProportion'].mean()

    # Step 5: Calculate the absolute deviation from the average for each race
    visits_by_race['AbsoluteDeviation'] = abs((visits_by_race['SDVisitProportion'] - overall_average) / overall_average)

    # Step 6: Calculate the Index of Disparity
    index_of_disparity = (visits_by_race['AbsoluteDeviation'].mean()) * 100

    return index_of_disparity



def specialistVisitsPerCategory(Category):
    # Step 2: Calculate Total and specialist Visits by Ethnicity
    visits_by_Category = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby(Category).agg(
        TotalVisits=pd.NamedAgg(column='IsspecialistVisit', aggfunc='count'),
        SDVisit=pd.NamedAgg(column='IsspecialistVisit', aggfunc='sum')
    ).reset_index()

    # Step 3: Calculate Odds for Each Race
    visits_by_Category['NonSDVisit'] = visits_by_Category['TotalVisits'] - visits_by_Category['SDVisit']
    visits_by_Category['Odds'] = visits_by_Category['SDVisit'] / visits_by_Category['NonSDVisit']

    # Step 4: Calculate Total Population and ED Visits Proportion for Disparity Index
    total_population = visits_by_Category['TotalVisits'].sum()
    total_sd_visits = visits_by_Category['SDVisit'].sum()

    visits_by_Category['PopulationProportion'] = visits_by_Category['TotalVisits'] / total_population
    visits_by_Category['SDVisitProportion'] = visits_by_Category['SDVisit'] / total_sd_visits

    visits_by_Category['AbsoluteDifference'] = abs(visits_by_Category['SDVisitProportion'] - visits_by_Category['PopulationProportion'])
        
    # Step 4: Sum the absolute differences and divide by 2
    index_of_dissimilarity = (visits_by_Category['AbsoluteDifference'].sum() / 2) * 100  # Convert to percentage
      
    
    # Step 5: Calculate Disparity Index for Each Race
    visits_by_Category['DisparityIndex'] = visits_by_Category['SDVisitProportion'] / visits_by_Category['PopulationProportion']
    
    
    index_of_disparity = calculate_index_of_disparity(Category, df_DIAGPROC_FINDEMOG_ENCFiltered)
    
#     paragraph = document.add_paragraph('index of dissimilarity :')
#     paragraph.add_run(f"{index_of_dissimilarity:.4f}%")
#     paragraph = document.add_paragraph('index of disparity :')
#     paragraph.add_run(f"{index_of_disparity:.4f}%")

#     print('index of dissimilarity :',index_of_dissimilarity, '\n index of disparity :',index_of_disparity, )


    # Note: Calculating Odds Ratio (OR) for each race against all others directly is less straightforward
    # because it involves comparing each race to the aggregation of all others.
    # For simplicity, we calculate Odds and Disparity Index, which already provide significant insights.

    # Display the final DataFrame
    visits_by_Category[[Category, 'Odds', 'DisparityIndex']]
    
    print(visits_by_Category)
    visits_by_Category = visits_by_Category.sort_values(by='PopulationProportion', ascending=True)
    print(visits_by_Category)


    # Step 2: Compute cumulative proportions
    visits_by_Category['CumulativePopulation'] = visits_by_Category['PopulationProportion'].cumsum()
    visits_by_Category['CumulativeSDVisit'] = visits_by_Category['SDVisitProportion'].cumsum()

    concentration_curve_x = visits_by_Category['CumulativePopulation']
    concentration_curve_y = visits_by_Category['CumulativeSDVisit']

    print(visits_by_Category)

    
     # Step 4: Plot the Concentration Curve
    plt.figure(figsize=(width, height))
    plt.plot(concentration_curve_x, concentration_curve_y, label='Concentration Curve', marker='o', linestyle='-')
    plt.plot([0, 1], [0, 1], color='k', linestyle='--', label='Line of Equality')  # Diagonal line representing perfect equality
    #plt.xlabel('Cumulative Proportion of Population')
    plt.xlabel('')

    plt.ylabel('Specialist Consults Cumulative %',fontsize=12)
    #plt.title('Concentration Curve for Emergency Visits by '+ Category)
    plt.legend(fontsize=fsize)
    plt.grid(True)
    plt.xticks(fontsize=fsize)
    plt.yticks(fontsize=fsize)
    #plt.savefig('temp.png')
    plt.savefig('temp.png', bbox_inches='tight')
    document.add_picture('temp.png', width=Inches(width))
    paragraph = document.add_paragraph("Figure XX:Concentration Curve for specialist Consults by ")
    paragraph.add_run(Category)
    plt.show()

    visits_by_Category['PercentagespecialistVisits'] = (visits_by_Category['SDVisit'] / visits_by_Category['TotalVisits']) * 100

    
    
     # Calculate the maximum Disparity Index
    maximum_disparity = visits_by_Category['DisparityIndex'].max()

    # Print the data to be plotted
    print("Visit by category")
    print(visits_by_Category)

    # Plotting
    fig, ax1 = plt.subplots(figsize=(width, height))

    # Plot the bar chart for Percentage of Emergency Visits
    sns.barplot(x=Category, y='PercentagespecialistVisits', data=visits_by_Category, palette='coolwarm', ax=ax1)

    # Set axis labels and increase font size
    ax1.set_ylabel('Specialist Consults Percentage', fontsize=fsize)
    ax1.set_xlabel('')
    ax1.tick_params(axis='both', which='major', labelsize=fsize)
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')

    # Add percentage labels on top of the bars
    for p in ax1.patches:
        ax1.annotate(f'{p.get_height():.1f}%', 
                     (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha='center', va='center', fontsize=fsize, color='black', 
                     xytext=(0, 5), textcoords='offset points')

    # Create a secondary axis for the Disparity Index
    ax2 = ax1.twinx()
    ax2.set_ylabel('Disparity Index', fontsize=fsize, color='blue')
    ax2.set_ylim(0, maximum_disparity + 1)  # Set the y-axis limit to maximum_disparity + 2
     # Create a secondary axis for the Disparity Index
    ax2.tick_params(axis='y', labelsize=fsize)  # Change y-ticks font size for the primary axis


    # Plot Disparity Index as points
    ax2.scatter(visits_by_Category[Category], visits_by_Category['DisparityIndex'], color='blue', s=5, zorder=5, label='Disparity Index')

    # Add labels to each point for Disparity Index
    for i, (x, y) in enumerate(zip(visits_by_Category[Category], visits_by_Category['DisparityIndex'])):
        ax2.annotate(f'{y:.2f}', (i, y), textcoords="offset points", xytext=(10, 0),  fontsize=fsize, ha='left', color='blue')

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.legend(loc="upper left", fontsize=fsize)  # Add legend for Disparity Index points
    plt.savefig('temp.png', bbox_inches='tight')

    document.add_picture('temp.png', width=Inches(width))
    paragraph=document.add_paragraph("Figure XX: Percentage of  Specialist Consults by ")
    paragraph.add_run(Category)    
    plt.show()


        # Plotting
    fig, ax1 = plt.subplots(figsize=(width, height))

    # Plot the bar chart for Percentage of Emergency Visits
    sns.barplot(x=Category, y='DisparityIndex', data=visits_by_Category, palette='coolwarm', ax=ax1)

    # Set axis labels and increase font size
    ax1.set_ylabel('Disparity Index', fontsize=fsize)
    ax1.set_xlabel('')
    ax1.tick_params(axis='both', which='major', labelsize=fsize)
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')

    # Add percentage labels on top of the bars
    for p in ax1.patches:
        ax1.annotate(f'{p.get_height():.1f}', 
                     (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha='center', va='center', fontsize=fsize, color='black', 
                     xytext=(0, 5), textcoords='offset points')

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.savefig('temp.png', bbox_inches='tight')
    document.add_picture('temp.png', width=Inches(width))    
    paragraph = document.add_paragraph("Figure XX: Percentage of  specialist Visits by ")
    paragraph.add_run(Category)
    plt.show()



    def gini_coefficient(x, y):
        """
        Calculate the Gini coefficient based on the cumulative distributions 
        of specialist visits (y) and population (x).

        Parameters:
        - x: Cumulative distribution of the population
        - y: Cumulative distribution of the visits

        Returns:
        - Gini coefficient as a value between 0 and 1
        """
        n = len(x)
        if n < 2:
            return 0  # Gini coefficient is zero for datasets with fewer than two points

        # Ensure the data is sorted by x (population proportion)
        sorted_pairs = sorted(zip(x, y))
        x_sorted, y_sorted = zip(*sorted_pairs)

        # Calculate the Gini coefficient using the trapezoidal rule for numerical integration
        area_below_curve = np.trapz(y_sorted, x_sorted)
        area_perfect_equality = 0.5  # This represents the area under the line of equality (diagonal)

        # Calculate Gini coefficient
        gini = (area_perfect_equality - area_below_curve) / area_perfect_equality
        return gini
    
    
    gini = gini_coefficient(visits_by_Category['CumulativeSDVisit'],visits_by_Category['CumulativePopulation'])
    
    paragraph = document.add_paragraph("Gini Coefficient: ") 
    paragraph.add_run(f"{gini:.4f}%")
    print('Gini coefficient :',gini)
    
    

    paragraph = document.add_paragraph('index of dissimilarity :')
    paragraph.add_run(f"{index_of_dissimilarity:.4f}%")
    paragraph = document.add_paragraph('index of disparity :')
    paragraph.add_run(f"{index_of_disparity:.4f}%")

    print('index of dissimilarity :',index_of_dissimilarity, '\n index of disparity :',index_of_disparity, )


    return(visits_by_Category)


visits_by_race = specialistVisitsPerCategory('RACE')
visits_by_race = visits_by_race.drop(['CumulativePopulation', 'CumulativeSDVisit'], axis=1)
visits_by_race = visits_by_race.set_index('RACE')
doctable(visits_by_race,"Specialist Visits By Race",['RACE', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','AbsoluteDifference'])

visit_by_HHSREGION = specialistVisitsPerCategory('HHSREGION')
visit_by_HHSREGION = visit_by_HHSREGION.drop(['CumulativePopulation', 'CumulativeSDVisit'], axis=1)
visit_by_HHSREGION =visit_by_HHSREGION .set_index('HHSREGION')
doctable(visit_by_HHSREGION,"Specialist Visits By Health Facility Region",['HHSREGION', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','AbsoluteDifference'])


visit_by_FINANCIALCLASS = specialistVisitsPerCategory('FINANCIALCLASS')
visit_by_FINANCIALCLASS = visit_by_FINANCIALCLASS.drop(['CumulativePopulation', 'CumulativeSDVisit'], axis=1)
visit_by_FINANCIALCLASS = visit_by_FINANCIALCLASS.set_index('FINANCIALCLASS')
doctable(visit_by_FINANCIALCLASS,"Specialist Visits By Finaincial Class",['FINANCIALCLASS', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','AbsoluteDifference'])

visit_by_INCOME_CATEGORY = specialistVisitsPerCategory('INCOME_CATEGORY')
visit_by_INCOME_CATEGORY = visit_by_INCOME_CATEGORY.drop(['CumulativePopulation', 'CumulativeSDVisit'], axis=1)
visit_by_INCOME_CATEGORY=visit_by_INCOME_CATEGORY.set_index('INCOME_CATEGORY')
doctable(visit_by_INCOME_CATEGORY,"Specialist Visits By Income Category",['INCOME_CATEGORY', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','AbsoluteDifference'])

visit_by_AGE_RANGE = specialistVisitsPerCategory('AGE_RANGE')
visit_by_AGE_RANGE = visit_by_AGE_RANGE.drop(['CumulativePopulation', 'CumulativeSDVisit'], axis=1)
visit_by_AGE_RANGE = visit_by_AGE_RANGE.set_index('AGE_RANGE')
doctable(visit_by_AGE_RANGE,"Specialist Visits By Age Range",['AGE_RANGE', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','AbsoluteDifference'])

visit_by_DEPTLOCATION = specialistVisitsPerCategory('DEPTLOCATION')
visit_by_DEPTLOCATION = visit_by_DEPTLOCATION.drop(['CumulativePopulation', 'CumulativeSDVisit'], axis=1)
visit_by_DEPTLOCATION = visit_by_DEPTLOCATION.set_index('DEPTLOCATION')
doctable(visit_by_DEPTLOCATION,"Specialist Visits By Health Department Location",['DEPTLOCATION', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','AbsoluteDifference'])


visit_by_ETHNICITY = specialistVisitsPerCategory('ETHNICITY')
visit_by_ETHNICITY = visit_by_ETHNICITY.drop(['CumulativePopulation', 'CumulativeSDVisit'], axis=1)
visit_by_ETHNICITY = visit_by_ETHNICITY.set_index('ETHNICITY')
doctable(visit_by_DEPTLOCATION,"Specialist Visits By Health Department Location",['ETHNICITY', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','AbsoluteDifference'])



In [ ]:
#Calucate Odds Radio and Disparity Index
specialistList =['Allergy and Immunology','Pulmonology']
visitType = ['Hospital Encounter']


df_DIAGPROC_FINDEMOG_ENCFiltered['IsspecialistVisit'] = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].isin(specialistList)
df_DIAGPROC_FINDEMOG_ENCFiltered['TYPE'] = df_DIAGPROC_FINDEMOG_ENCFiltered['TYPE'].isin(visitType)

def calculate_index_of_disparity_nonspecialist(Category, df):
    """
    Calculate the Index of Disparity for non-specialist visits (IsspecialistVisit == 0).
    """
    # Step 1: Filter the data for non-specialist visits
    nonspecialist_visits = df[df['IsspecialistVisit'] == 0]

    # Step 2: Calculate the total number of non-specialist visits for each category
    visits_by_category = nonspecialist_visits.groupby(Category).agg(
        NonSDVisits=pd.NamedAgg(column='IsspecialistVisit', aggfunc='count')
    ).reset_index()

    # Step 3: Calculate the proportion of non-specialist visits for each category
    total_nonsd_visits = visits_by_category['NonSDVisits'].sum()
    visits_by_category['NonSDVisitProportion'] = visits_by_category['NonSDVisits'] / total_nonsd_visits

    # Step 4: Calculate the overall average proportion of non-specialist visits
    overall_average = visits_by_category['NonSDVisitProportion'].mean()

    # Step 5: Calculate the absolute deviation from the average for each category
    visits_by_category['AbsoluteDeviation'] = abs(
        (visits_by_category['NonSDVisitProportion'] - overall_average) / overall_average
    )

    # Step 6: Calculate the Index of Disparity
    index_of_disparity = (visits_by_category['AbsoluteDeviation'].mean()) * 100

    return index_of_disparity


def nonSpecialistVisitsPerCategory(Category):
    # Step 1: Calculate Total and Non-specialist Visits by Category
    visits_by_Category = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby(Category).agg(
        TotalVisits=pd.NamedAgg(column='IsspecialistVisit', aggfunc='count'),
        NonSDVisit=pd.NamedAgg(column='IsspecialistVisit', aggfunc=lambda x: (x == 0).sum())
    ).reset_index()

    # Step 2: Calculate SDVisits (specialist visits)
    visits_by_Category['SDVisit'] = visits_by_Category['TotalVisits'] - visits_by_Category['NonSDVisit']

    # Step 3: Calculate Odds for Non-specialist Visits
    visits_by_Category['Odds'] = visits_by_Category['NonSDVisit'] / visits_by_Category['SDVisit']

    # Step 4: Calculate Total Population and Non-specialist Visit Proportion
    total_population = visits_by_Category['TotalVisits'].sum()
    total_non_sd_visits = visits_by_Category['NonSDVisit'].sum()

    visits_by_Category['PopulationProportion'] = visits_by_Category['TotalVisits'] / total_population
    visits_by_Category['NonSDVisitProportion'] = visits_by_Category['NonSDVisit'] / total_non_sd_visits

    visits_by_Category['AbsoluteDifference'] = abs(visits_by_Category['NonSDVisitProportion'] - visits_by_Category['PopulationProportion'])
    
    # Step 5: Calculate Index of Dissimilarity
    index_of_dissimilarity = (visits_by_Category['AbsoluteDifference'].sum() / 2) * 100  # Convert to percentage

    index_of_disparity = calculate_index_of_disparity_nonspecialist(Category, df_DIAGPROC_FINDEMOG_ENCFiltered)
    

    # Step 6: Calculate Disparity Index
    visits_by_Category['DisparityIndex'] = visits_by_Category['NonSDVisitProportion'] / visits_by_Category['PopulationProportion']

    # Step 7: Compute cumulative proportions for the concentration curve
    visits_by_Category = visits_by_Category.sort_values(by='PopulationProportion', ascending=True)
    visits_by_Category['CumulativePopulation'] = visits_by_Category['PopulationProportion'].cumsum()
    visits_by_Category['CumulativeNonSDVisit'] = visits_by_Category['NonSDVisitProportion'].cumsum()

    # Display the DataFrame
    print(visits_by_Category)

    # Plot the Concentration Curve
    plt.figure(figsize=(8, 6))
    plt.plot(visits_by_Category['CumulativePopulation'], visits_by_Category['CumulativeNonSDVisit'], 
             label='Concentration Curve', marker='o', linestyle='-')
    plt.plot([0, 1], [0, 1], color='k', linestyle='--', label='Line of Equality')
    plt.xlabel('Cumulative Proportion of Population', fontsize=14)
    plt.ylabel('Cumulative Proportion of Specialist Non-Visits', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('temp.png', bbox_inches='tight')
    document.add_picture('temp.png', width=Inches(6))    
    paragraph = document.add_paragraph("Figure XX: Specialist Non-Visit Concentration Curve ")
    paragraph.add_run(Category)
    plt.show()
    
    
    
    
    
    visits_by_Category['PercentagespecialistVisits'] = (visits_by_Category['NonSDVisit'] / visits_by_Category['TotalVisits']) * 100

         # Calculate the maximum Disparity Index
    maximum_disparity = visits_by_Category['DisparityIndex'].max()

    # Print the data to be plotted
    print("Visit by category")
    print(visits_by_Category)

    # Plotting
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot the bar chart for Percentage of Emergency Visits
    sns.barplot(x=Category, y='PercentagespecialistVisits', data=visits_by_Category, palette='coolwarm', ax=ax1)

    # Set axis labels and increase font size
    ax1.set_ylabel('Percentage of Specialist Non-Visits', fontsize=14)
    ax1.set_xlabel('')
    ax1.tick_params(axis='both', which='major', labelsize=12)
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')

    # Add percentage labels on top of the bars
    for p in ax1.patches:
        ax1.annotate(f'{p.get_height():.1f}%', 
                     (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha='center', va='center', fontsize=14, color='black', 
                     xytext=(0, 5), textcoords='offset points')

    # Create a secondary axis for the Disparity Index
    ax2 = ax1.twinx()
    ax2.set_ylabel('Disparity Index', fontsize=14, color='blue')
    ax2.set_ylim(0, maximum_disparity + 1)  # Set the y-axis limit to maximum_disparity + 2

    # Plot Disparity Index as points
    ax2.scatter(visits_by_Category[Category], visits_by_Category['DisparityIndex'], color='blue', s=100, zorder=5, label='Disparity Index')

    # Add labels to each point for Disparity Index
    for i, (x, y) in enumerate(zip(visits_by_Category[Category], visits_by_Category['DisparityIndex'])):
        ax2.annotate(f'{y:.2f}', (i, y), textcoords="offset points", xytext=(10, 0),  fontsize=14, ha='left', color='blue')

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.legend(loc="upper right", fontsize=14)  # Add legend for Disparity Index points
    plt.savefig('temp.png', bbox_inches='tight')

    document.add_picture('temp.png', width=Inches(6))
    paragraph=document.add_paragraph("Figure XX: Percentage of  Emergency Visits by ")
    paragraph.add_run(Category)    
    plt.show()


        # Plotting
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot the bar chart for Percentage of Emergency Visits
    sns.barplot(x=Category, y='DisparityIndex', data=visits_by_Category, palette='coolwarm', ax=ax1)

    # Set axis labels and increase font size
    ax1.set_ylabel('Disparity Index', fontsize=14)
    ax1.set_xlabel('')
    ax1.tick_params(axis='both', which='major', labelsize=12)
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')

    # Add percentage labels on top of the bars
    for p in ax1.patches:
        ax1.annotate(f'{p.get_height():.1f}', 
                     (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha='center', va='center', fontsize=14, color='black', 
                     xytext=(0, 5), textcoords='offset points')

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.savefig('temp.png', bbox_inches='tight')
    document.add_picture('temp.png', width=Inches(6))    
    paragraph = document.add_paragraph("Figure XX: Percentage of  specialist Visits by ")
    paragraph.add_run(Category)
    plt.show()


    
    def gini_coefficient(x, y):
        """
        Calculate the Gini coefficient based on the cumulative distributions 
        of specialist visits (y) and population (x).

        Parameters:
        - x: Cumulative distribution of the population
        - y: Cumulative distribution of the visits

        Returns:
        - Gini coefficient as a value between 0 and 1
        """
        n = len(x)
        if n < 2:
            return 0  # Gini coefficient is zero for datasets with fewer than two points

        # Ensure the data is sorted by x (population proportion)
        sorted_pairs = sorted(zip(x, y))
        x_sorted, y_sorted = zip(*sorted_pairs)

        # Calculate the Gini coefficient using the trapezoidal rule for numerical integration
        area_below_curve = np.trapz(y_sorted, x_sorted)
        area_perfect_equality = 0.5  # This represents the area under the line of equality (diagonal)

        # Calculate Gini coefficient
        gini = (area_perfect_equality - area_below_curve) / area_perfect_equality
        return gini


    gini = gini_coefficient(visits_by_Category['CumulativeNonSDVisit'],visits_by_Category['CumulativePopulation'])

    paragraph = document.add_paragraph("Gini Coefficient: ") 
    paragraph.add_run(f"{gini:.4f}%")
    print('Gini coefficient :',gini)

    
    paragraph = document.add_paragraph('index of dissimilarity :')
    paragraph.add_run(f"{index_of_dissimilarity:.4f}%")
    paragraph = document.add_paragraph('index of disparity :')
    paragraph.add_run(f"{index_of_disparity:.4f}%")

    print('index of dissimilarity :',index_of_dissimilarity, '\n index of disparity :',index_of_disparity, )


    # Add percentages for non-specialist visits
    visits_by_Category['PercentageNonSpecialistVisits'] = (visits_by_Category['NonSDVisit'] / visits_by_Category['TotalVisits']) * 100

    # Return the final DataFrame
    return visits_by_Category



visits_by_race = nonSpecialistVisitsPerCategory('RACE')
visits_by_race = visits_by_race.drop(['CumulativePopulation', 'CumulativeNonSDVisit'], axis=1)
visits_by_race = visits_by_race.set_index('RACE')
print(visits_by_race)
doctable(visits_by_race,"Specialist Visits By Race",['RACE', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','NonSDVisitProportion','AbsoluteDifference','PercentageNonSpecialistVisits'])

visit_by_HHSREGION = nonSpecialistVisitsPerCategory('HHSREGION')
visit_by_HHSREGION = visit_by_HHSREGION.drop(['CumulativePopulation', 'CumulativeNonSDVisit'], axis=1)
visit_by_HHSREGION =visit_by_HHSREGION .set_index('HHSREGION')
doctable(visit_by_HHSREGION,"Specialist Visits By Health Facility Region",['HHSREGION','TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','NonSDVisitProportion','AbsoluteDifference','PercentageNonSpecialistVisits'])


visit_by_FINANCIALCLASS = nonSpecialistVisitsPerCategory('FINANCIALCLASS')
visit_by_FINANCIALCLASS = visit_by_FINANCIALCLASS.drop(['CumulativePopulation', 'CumulativeNonSDVisit'], axis=1)
visit_by_FINANCIALCLASS = visit_by_FINANCIALCLASS.set_index('FINANCIALCLASS')
doctable(visit_by_FINANCIALCLASS,"Specialist Visits By Finaincial Class",['FINANCIALCLASS', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','NonSDVisitProportion','AbsoluteDifference','PercentageNonSpecialistVisits'])

visit_by_INCOME_CATEGORY = nonSpecialistVisitsPerCategory('INCOME_CATEGORY')
visit_by_INCOME_CATEGORY = visit_by_INCOME_CATEGORY.drop(['CumulativePopulation', 'CumulativeNonSDVisit'], axis=1)
visit_by_INCOME_CATEGORY=visit_by_INCOME_CATEGORY.set_index('INCOME_CATEGORY')
doctable(visit_by_INCOME_CATEGORY,"Specialist Visits By Income Category",['INCOME_CATEGORY', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','NonSDVisitProportion','AbsoluteDifference','PercentageNonSpecialistVisits'])

visit_by_AGE_RANGE = nonSpecialistVisitsPerCategory('AGE_RANGE')
visit_by_AGE_RANGE = visit_by_AGE_RANGE.drop(['CumulativePopulation', 'CumulativeNonSDVisit'], axis=1)
visit_by_AGE_RANGE = visit_by_AGE_RANGE.set_index('AGE_RANGE')
doctable(visit_by_AGE_RANGE,"Specialist Visits By Age Range",['AGE_RANGE', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','NonSDVisitProportion','AbsoluteDifference','PercentageNonSpecialistVisits'])

visit_by_DEPTLOCATION = nonSpecialistVisitsPerCategory('DEPTLOCATION')
visit_by_DEPTLOCATION = visit_by_DEPTLOCATION.drop(['CumulativePopulation', 'CumulativeNonSDVisit'], axis=1)
visit_by_DEPTLOCATION = visit_by_DEPTLOCATION.set_index('DEPTLOCATION')
doctable(visit_by_DEPTLOCATION,"Specialist Visits By Health Department Location",['DEPTLOCATION', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','NonSDVisitProportion','AbsoluteDifference','PercentageNonSpecialistVisits'])


visit_by_ETHNICITY = nonSpecialistVisitsPerCategory('ETHNICITY')
visit_by_ETHNICITY = visit_by_ETHNICITY.drop(['CumulativePopulation', 'CumulativeNonSDVisit'], axis=1)
visit_by_ETHNICITY = visit_by_ETHNICITY.set_index('ETHNICITY')
doctable(visit_by_DEPTLOCATION,"Specialist Visits By Health Department Location",['ETHNICITY', 'TotalVisits', 'SDVisits', 'NonSDVisits', 'Odds','PopulationProportion', 'SDVisitsProportion', 'DisparityIndex','PercentageSpecialistVisits','NonSDVisitProportion','AbsoluteDifference','PercentageNonSpecialistVisits'])


In [ ]:
visit_by_ETHNICITY


In [ ]:
emergency_visits_df = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']

# Step 2 and 3: Group by Ethnicity and Race, and count visits
visits_count = emergency_visits_df.groupby(['DEPTLOCATION']).size().reset_index(name='Visits')

# Step 4: Calculate total emergency department visits
total_emergency_visits = visits_count['Visits'].sum()

# Step 5: Calculate percentages
visits_count['Percentage'] = (visits_count['Visits'] / total_emergency_visits) * 100

In [ ]:
visits_count=visits_count.set_index(['DEPTLOCATION'])
doctable(visits_count,"Hospital used for Emergency Visits (%) ",['DEPTLOCATION','Visits','Percentage'])
                                                                 


In [ ]:
emergency_visits_df = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']

# Step 2 and 3: Group by Ethnicity and Race, and count visits
visits_count = emergency_visits_df.groupby(['ICD10CODE']).size().reset_index(name='Visits')

# Step 4: Calculate total emergency department visits
total_emergency_visits = visits_count['Visits'].sum()

# Step 5: Calculate percentages
visits_count['Percentage'] = (visits_count['Visits'] / total_emergency_visits) * 100

In [ ]:
visits_count=visits_count.set_index(['ICD10CODE'])
doctable(visits_count,"Diagnosis Percentages for Patients Visiting Emergency Department (%) ",['ICD10CODE','Visits','Percentage'])


In [ ]:
emergency_visits_df = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']

# Step 2 and 3: Group by Ethnicity and Race, and count visits
visits_count = emergency_visits_df.groupby(['SEX']).size().reset_index(name='Visits')

# Step 4: Calculate total emergency department visits
total_emergency_visits = visits_count['Visits'].sum()

# Step 5: Calculate percentages
visits_count['Percentage'] = (visits_count['Visits'] / total_emergency_visits) * 100

In [ ]:
visits_count=visits_count.set_index(['SEX'])

doctable(visits_count,"Gender of  Patients Visiting Emergency Department (%) ",['SEX','Visits','Percentage'])


In [ ]:
emergency_visits_df = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']

# Step 2 and 3: Group by Ethnicity and Race, and count visits
visits_count = emergency_visits_df.groupby(['AGE_RANGE']).size().reset_index(name='Visits')

# Step 4: Calculate total emergency department visits
total_emergency_visits = visits_count['Visits'].sum()

# Step 5: Calculate percentages
visits_count['Percentage'] = (visits_count['Visits'] / total_emergency_visits) * 100

In [ ]:
df_DIAGPROC_FINDEMOG_ENCFiltered

In [ ]:
visits_count=visits_count.set_index(['AGE_RANGE'])

doctable(visits_count,"Age Group of  Patients Visiting Emergency Department (%) ",['AGE_RANGE','Visits','Percentage'])


In [ ]:
#patient_visits_department_filtered = patient_visits_department[patient_visits_department['Visit_Count'] > 1]

In [ ]:
Counter(patient_visits['Visit_Count'])

In [ ]:
#let us get all procedure codes 
# Flatten the list of lists into a single list
all_procedure_codes = [code for sublist in df_DIAGPROC_FINDEMOG_ENCFiltered['ProcedureList'] for code in sublist]

# Get unique procedure codes
unique_procedure_codes = list(set(all_procedure_codes))

In [ ]:
# import csv

# # Open a file in write mode
# with open('cptcodesused.csv', 'w', newline='') as file:
#     writer = csv.writer(file)

#     # Writing the data
#     for item in unique_procedure_codes:
#         writer.writerow([item])

In [ ]:
flat_codes = [code for sublist in df_DIAGPROC_FINDEMOG_ENCFiltered['ProcedureList'] for code in sublist]
flat_names = [name for sublist in df_DIAGPROC_FINDEMOG_ENCFiltered['ProcedureNames'] for name in sublist]

# Step 2: Create a dictionary that maps codes to names
code_name_dict = dict(zip(flat_codes, flat_names))

# Step 3: Get unique codes and corresponding names
unique_codes = list(set(flat_codes))
unique_code_name_pairs = {code: code_name_dict[code] for code in unique_codes}

# Convert to a list of tuples if needed (code, name)
unique_code_name_list = list(unique_code_name_pairs.items())

# Now you have a unique list of (code, name) pairs
print(unique_code_name_list)

CPTCodes = pd.DataFrame.from_dict(unique_code_name_list)


In [ ]:
#CPTCodes.to_csv('cptcodesused.csv')

In [ ]:
#cptCodeCostDictionary = dict(zip(cptCodeData['CPTCode'], cptCodeData['Cost']))


In [ ]:
cptCodeData = pd.read_excel('CostTables/CPTCodesAsthma.xlsx')
cptCodeData['CPTCode'] = cptCodeData['CPTCode'].astype(str)
cptCodeCostDictionary = dict(zip(cptCodeData['CPTCode'], cptCodeData['Cost']))


# Define the list of procedure codes you are interested in
target_procedure_codes = cptCodeData['CPTCode'].tolist()

# Function to check if any of the target codes are in the procedure list
def contains_any_target_code(procedure_list):
    return any(code in procedure_list for code in target_procedure_codes)

# Filter the DataFrame
df_DIAGPROC_FINDEMOG_ENCFiltered = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['ProcedureList'].apply(contains_any_target_code)]

In [ ]:
# Function to calculate cost for each encounter
def calculate_cost(procedures):
    return sum(cptCodeCostDictionary[proc] for proc in procedures if proc in cptCodeCostDictionary)

# Apply the function to each row and create a new column
df_DIAGPROC_FINDEMOG_ENCFiltered['Cost'] = df_DIAGPROC_FINDEMOG_ENCFiltered['ProcedureList'].apply(calculate_cost)

# Display the updated dataframe
print(df_DIAGPROC_FINDEMOG_ENCFiltered)

In [ ]:
Counter(df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'])

In [ ]:
df_DIAGPROC_FINDEMOG_ENCFiltered.columns

In [ ]:
# deptLocations = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTLOCATION'].unique()
# # Open a file in write mode
# with open('departmentLocations.csv', 'w', newline='') as file:
#     writer = csv.writer(file)

#     # Writing the data
#     for item in deptLocations:
#         writer.writerow([item])

In [ ]:
# Generate a list of column names that start with 'unknown'
columns_to_drop = [col for col in df_DIAGPROC_FINDEMOG_ENCFiltered.columns if col.startswith('Unnamed')]
# Drop these columns from the DataFrame
df_DIAGPROC_FINDEMOG_ENCFiltered = df_DIAGPROC_FINDEMOG_ENCFiltered.drop(columns=columns_to_drop,errors='ignore')

df_DIAGPROC_FINDEMOG_ENCFiltered.rename(columns={'PATIENTHASHMRN_x': 'PATIENTHASHMRN'}, inplace=True, errors='ignore')
df_DIAGPROC_FINDEMOG_ENCFiltered.rename(columns={'DATE_DIF_x': 'DATE_DIF'}, inplace=True, errors='ignore')

df_DIAGPROC_FINDEMOG_ENCFiltered.drop(columns=['PATIENTHASHMRN_y','DATE_DIF_y','ADMITDEPTSPECIALTY'], inplace=True, errors='ignore')



In [ ]:
#BIOLOGICALS = list(BIOLOGICALS)
#glucocorticoid_medications = df_DIAGMED[df_DIAGMED['PHARMACEUTICALCLASS'] == 'GLUCOCORTICOIDS']


In [ ]:
#Counter(df_DIAGMED.PHARMACEUTICALCLASS)

In [ ]:
#glucocorticoid_medications

In [ ]:
from scipy import stats
from scikit_posthocs import posthoc_dunn

# Group by ethnicity and aggregate procedure counts
grouped_data = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby('DEPTSPECIALTY')['Cost'].agg(['mean', 'median', 'sum', 'count', 'std'])

plt.figure(figsize=(10, 6))

df_DIAGPROC_FINDEMOG_ENCFiltered['LogCost'] = np.log1p(df_DIAGPROC_FINDEMOG_ENCFiltered['Cost'])  # Use log1p to avoid log(0) issues
#sns.boxplot(x='DEPTSPECIALTY', y='LogCost', data=df_DIAGPROC_FINDEMOG_ENCFiltered)
sns.violinplot(x='DEPTSPECIALTY', y='LogCost', data=df_DIAGPROC_FINDEMOG_ENCFiltered, palette='Blues')

#plt.title('Department Speciality versus Cost of Treatment')
plt.xticks(rotation=45, ha='right')
plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
paragraph = document.add_paragraph("Figure XX:Department Speciality versus Estimated Cost of Treatment ")
plt.show()


# Statistical Test: ANOVA or Kruskal-Wallis
# Adjust the test based on your data distribution
anova_result = stats.f_oneway(*[group['Cost'].values for name, group in df_DIAGPROC_FINDEMOG_ENCFiltered.groupby('DEPTSPECIALTY')])
# Print ANOVA results

p1 = document.add_paragraph("ANOVA test result:")
p1.add_run(str(anova_result))
print(anova_result)

kruskal_result = stats.kruskal(*[group['Cost'].values for name, group in df_DIAGPROC_FINDEMOG_ENCFiltered.groupby('DEPTSPECIALTY')])
p1 = document.add_paragraph("Kruskal-Wallis test result:")
p1.add_run(str(kruskal_result))
print(kruskal_result)


# pairwise mann whitney test with p value adjustment 
# False discovery rate 


# Perform Dunn's post-hoc test with Bonferroni correction
posthoc_result = posthoc_dunn(
    df_DIAGPROC_FINDEMOG_ENCFiltered, 
    val_col='Cost', 
    group_col='DEPTSPECIALTY', 
    p_adjust='bonferroni'
)

print(posthoc_result)

In [ ]:
# Create a violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(x='DEPTSPECIALTY', y='Cost', data=df_DIAGPROC_FINDEMOG_ENCFiltered, inner=None, palette='muted')

# Calculate and plot percentiles for each category
categories = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].unique()
for category in categories:
    # Filter the data for the current category
    data = df_DIAGPROC_FINDEMOG_ENCFiltered.loc[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == category, 'Cost']
    # Calculate the percentiles
    p25, median, p75 = np.percentile(data, [25, 50, 75])
    # Plot the percentiles on the violin plot
    category_index = np.where(categories == category)[0][0]
    plt.scatter([category_index] * 3, [p25, median, p75], color='black', s=50, marker='o', label='Percentiles' if category_index == 0 else "")

# Add labels and legend
plt.title('Violin Plot with Percentile Indicators')
plt.xlabel('Category')
plt.ylabel('Value')
plt.legend(['25th, 50th, 75th Percentiles'])

plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
paragraph = document.add_paragraph("Figure XX:  Department Speciality versus Estimated Cost of Treatment ")
plt.show()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from docx import Document
from docx.shared import Inches

# Create the violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(x='DEPTSPECIALTY', y='Cost', data=df_DIAGPROC_FINDEMOG_ENCFiltered, inner=None, palette='muted')

# Set a symlog scale for the y-axis
plt.yscale('symlog', linthresh=3000)  # linthresh defines the threshold for the linear scale

# Calculate and plot percentiles
categories = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].unique()
for category in categories:
    # Filter data
    data = df_DIAGPROC_FINDEMOG_ENCFiltered.loc[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == category, 'Cost']
    # Percentiles
    p25, median, p75 = np.percentile(data, [25, 50, 75])
    # Plot percentiles
    category_index = np.where(categories == category)[0][0]
    plt.scatter([category_index + 0.1] * 3, [p25, median, p75], color='black', s=50, marker='o')

# Add labels and legend
plt.title('Violin Plot with Percentile Indicators (SymLog Scale)')
plt.xlabel('Department Specialty')
plt.ylabel('Cost (Logarithmic for Large Values)')
plt.xticks(rotation=45)
plt.legend(['25th, 50th, 75th Percentiles'])

# Save plot as image
plt.savefig('temp.png', bbox_inches='tight')

# Add plot to Word document
document.add_heading('Analysis of Costs by Department Specialty', level=1)
document.add_picture('temp.png', width=Inches(6))
document.add_paragraph("Figure XX: Department Specialty versus Estimated Cost of Treatment")


plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from docx import Document
from docx.shared import Inches

# Calculate the threshold dynamically based on the data
all_costs = df_DIAGPROC_FINDEMOG_ENCFiltered['Cost']
linthresh = np.percentile(all_costs, 90)  # Use the 90th percentile as the threshold

# Create the violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(x='DEPTSPECIALTY', y='Cost', data=df_DIAGPROC_FINDEMOG_ENCFiltered, inner=None, palette='muted')

# Set a symlog scale with the dynamically determined threshold
plt.yscale('symlog', linthresh=linthresh)

# Calculate and plot percentiles for each category
categories = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].unique()
for category in categories:
    # Filter data
    data = df_DIAGPROC_FINDEMOG_ENCFiltered.loc[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == category, 'Cost']
    # Percentiles
    p25, median, p75 = np.percentile(data, [25, 50, 75])
    # Plot percentiles
    category_index = np.where(categories == category)[0][0]
    plt.scatter([category_index + 0.1] * 3, [p25, median, p75], color='black', s=50, marker='o')

# Add labels and legend
plt.title('Violin Plot with Percentile Indicators (Auto-SymLog Scale)')
plt.xlabel('Department Specialty')
plt.ylabel('Cost (Logarithmic for Large Values)')
plt.xticks(rotation=45)
plt.legend(['25th, 50th, 75th Percentiles'])

# Save plot as image
plt.savefig('temp.png', bbox_inches='tight')

# Add plot to Word document
document.add_heading('Analysis of Costs by Department Specialty', level=1)
document.add_picture('temp.png', width=Inches(6))
document.add_paragraph(f"Figure XX: Department Specialty versus Estimated Cost of Treatment. Linear scale threshold is dynamically set at the {linthresh:.2f} value.")

# Save document
#document.save('Analysis_Report.docx')

plt.show()

In [ ]:
# Create a violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(x='DEPTSPECIALTY', y='LogCost', data=df_DIAGPROC_FINDEMOG_ENCFiltered, inner=None, palette='muted')

# Calculate and plot percentiles for each category
categories = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].unique()
for category in categories:
    # Filter the data for the current category
    data = df_DIAGPROC_FINDEMOG_ENCFiltered.loc[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == category, 'LogCost']
    # Calculate the percentiles
    p25, median, p75 = np.percentile(data, [25, 50, 75])
    # Plot the percentiles on the violin plot
    category_index = np.where(categories == category)[0][0]
    plt.scatter([category_index] * 3, [p25, median, p75], color='black', s=50, marker='o', label='Percentiles' if category_index == 0 else "")

# Add labels and legend
plt.title('Violin Plot with Percentile Indicators')
plt.xlabel('Department Speciality')
plt.ylabel('Log Cost')
plt.legend(['25th, 50th, 75th Percentiles'])

plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
paragraph = document.add_paragraph("Figure XX:  Department Speciality versus Estimated Cost of Treatment ")
plt.show()

plt.show()

In [ ]:
Counter(df_DIAGPROC_FINDEMOG_ENCFiltered.DEPTSPECIALTY)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(12, 7))

if df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].isnull().any():
    print("Missing data detected for some departments!")
    
# Plot the boxplot
ax = sns.boxplot(
    x='DEPTSPECIALTY',
    y='Cost',
    data=df_DIAGPROC_FINDEMOG_ENCFiltered,
    palette='muted',
    showfliers=False  # Removes the outlier points
)


# Set symlog scale for the y-axis
all_costs = df_DIAGPROC_FINDEMOG_ENCFiltered['Cost']
linthresh = np.percentile(all_costs, 90)  # Automatically determine the linthresh
plt.yscale('symlog', linthresh=linthresh)

# Extract the medians for each category from the boxplot
medians = [line.get_ydata()[0] for line in ax.lines[4::6]]  # Median is every 6th line starting at 4th
print(medians)
# Calculate relative costs compared to the minimum median
min_median = min(medians)  # Minimum median
relative_costs = [median / min_median for median in medians]  # Relative to min median

# Annotate each box with the relative cost
for i, (median, relative_cost) in enumerate(zip(medians, relative_costs)):
    ax.text(
        i,  # X-position (category index)
        median,  # Y-position (median value)
        f'{relative_cost:.2f}x Min',  # Annotation text
        ha='center', va='center', fontsize=10, color='black', rotation=90  # Rotate text vertically
    )

# Customize the plot
plt.title('Boxplot of Cost by Department Specialty (Relative to Min Median)', fontsize=16)
plt.xlabel('Department Specialty', fontsize=14)
plt.ylabel('Cost (SymLog Scale)', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.tick_params(axis='y', which='both', labelsize=12)

# Add horizontal gridlines
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Save the plot (optional)
plt.tight_layout()
plt.savefig('boxplot_relative_to_min_median.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Prepare the data for Matplotlib
categories = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].unique()
data = [df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == category]['Cost'].values for category in categories]

# Create the boxplot
fig, ax = plt.subplots(figsize=(12, 7))
box = ax.boxplot(
    data,
    labels=categories,
    patch_artist=True,  # Use filled boxes
    showfliers=False,   # Exclude outliers
    boxprops=dict(facecolor='lightblue', color='black'),  # Customize box color
    medianprops=dict(color='red', linewidth=2)  # Customize median line
)

# Set symlog scale for the y-axis
all_costs = df_DIAGPROC_FINDEMOG_ENCFiltered['Cost']
linthresh = np.percentile(all_costs, 90)
ax.set_yscale('symlog', linthresh=linthresh)

# Add annotations for medians and relative costs
medians = [np.median(d) for d in data]
min_median = max(min(medians), 1e-3)  # Avoid zero median
relative_costs = [median / min_median for median in medians]

for i, (median, relative_cost) in enumerate(zip(medians, relative_costs)):
    ax.text(
        i + 1, median,  # X-position (1-indexed for Matplotlib's boxplot)
        f'{relative_cost:.2f}x Min',
        ha='center', va='center', fontsize=10, color='black', rotation=90
    )

# Customize the plot
ax.set_title('Boxplot of Cost by Department Specialty (Relative to Min Median)', fontsize=16)
ax.set_xlabel('Department Specialty', fontsize=14)
ax.set_ylabel('Cost (SymLog Scale)', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('boxplot_matplotlib.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
ddd = df_DIAGPROC_FINDEMOG_ENCFiltered[df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'] == 'Emergency Medicine']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(12, 7))

# Check for missing data
if df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].isnull().any():
    print("Missing data detected for some departments!")

# Plot the boxplot
ax = sns.boxplot(
    x='DEPTSPECIALTY',
    y='Cost',
    data=df_DIAGPROC_FINDEMOG_ENCFiltered,
    palette='muted',
    showfliers=False  # Removes the outlier points
)

# Set symlog scale for the y-axis
all_costs = df_DIAGPROC_FINDEMOG_ENCFiltered['Cost']
linthresh = np.percentile(all_costs, 90)  # Automatically determine the linthresh
plt.yscale('symlog', linthresh=linthresh)

# Extract medians and 75th percentiles (Q3) for each category
categories = df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'].unique()
medians = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby('DEPTSPECIALTY')['Cost'].median().reindex(categories).values
q3_values = df_DIAGPROC_FINDEMOG_ENCFiltered.groupby('DEPTSPECIALTY')['Cost'].quantile(0.75).reindex(categories).values

print("Extracted Medians (50th Percentile):", medians)
print("Extracted 75th Percentiles:", q3_values)

# Calculate relative costs compared to the minimum median
min_median = max(min(medians), 1e-3)  # Avoid division by zero
relative_costs_median = [median / min_median for median in medians]
relative_costs_q3 = [q3 / min_median for q3 in q3_values]

print("Relative Costs (50th Percentile):", relative_costs_median)
print("Relative Costs (75th Percentile):", relative_costs_q3)

# Annotate each box with the relative costs of the 50th and 75th percentiles
for i, (median, relative_cost_median, q3, relative_cost_q3) in enumerate(zip(medians, relative_costs_median, q3_values, relative_costs_q3)):
    # Annotate the median (50th percentile)
    ax.text(
        i, median,  # X-position and Y-position
        f'50%: {relative_cost_median:.2f}x Min',  # Annotation text
        ha='center', va='bottom', fontsize=12, color='black'
    )
    # Annotate the 75th percentile (Q3)
    ax.text(
        i, q3,  # X-position and Y-position
        f'75%: {relative_cost_q3:.2f}x Min',  # Annotation text
        ha='center', va='bottom', fontsize=12, color='blue'
    )

# Customize the plot
#plt.title('Boxplot of Cost by Department Specialty (Relative Cost of 50% and 75%)', fontsize=16)
plt.xlabel('')
plt.ylabel('Cost (SymLog Scale)', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.tick_params(axis='y', which='both', labelsize=12)

# Add horizontal gridlines
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Save the plot (optional)
plt.tight_layout()
plt.savefig('boxplot_relative_cost_50_75_percentiles.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
ddd.Cost.median()

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

df_DIAGPROC_FINDEMOG_ENCFiltered['Cost'] = pd.to_numeric(df_DIAGPROC_FINDEMOG_ENCFiltered['Cost'], errors='coerce')
df_DIAGPROC_FINDEMOG_ENCFiltered.dropna(subset=['Cost'], inplace=True)
df_DIAGPROC_FINDEMOG_ENCFiltered.dropna(subset=['DEPTSPECIALTY'], inplace=True)


tukey_results = pairwise_tukeyhsd(endog=df_DIAGPROC_FINDEMOG_ENCFiltered['Cost'], groups=df_DIAGPROC_FINDEMOG_ENCFiltered['DEPTSPECIALTY'], alpha=0.05)
print(tukey_results)
# or convert to a DataFrame
tukey_df = pd.DataFrame(data=tukey_results._results_table.data[1:], columns=tukey_results._results_table.data[0])
tukey_df.to_excel("tukey.xlsx")

In [ ]:

#df_DIAGPROC_sorted = df_DIAGPROC.sort_values(by='ProcedureCount', ascending=True)  # or False for descending order


In [ ]:
# Only patients with Asthma diagnoses in  during the data collection period are considered
patientMRNforAsthmaPatients = df_FINDEMOG.PATIENTHASHMRN.unique()
df_MEDICATIONS = df_MEDICATIONS[df_MEDICATIONS['PATIENTHASHMRN'].isin(patientMRNforAsthmaPatients)]

In [ ]:
# Medication's classes assigned as Severity 
Severe_Med_list=['GLUCOCORTICOIDS', #3
                 'INTERLEUKIN-5 (IL-5) ANTAGONISTS, MAB', # 3 GSk
                 'INTERLEUKIN-5(IL-5) RECEPTOR ALPHA ANTAGONIST, MAB', #3
                 'MONOCLONAL ANTIBODIES TO IMMUNOGLOBULIN E (IGE)',#3
                 'INTERLEUKIN-4(IL-4) RECEPTOR ALPHA ANTAGONIST, MAB',#3
                 'THYMIC STROMAL LYMPHOPOIETIN (TSLP) INHIBITORS' #3
                ]

Moderate_Med_list=[
                'BETA-ADRENERGIC AND ANTICHOLINERGIC COMBO, INHALED',# 2 Moderate
                'BETA-ADRENERGIC AND GLUCOCORTICOID COMBO, INHALED', # 2 Moderate
                'BETA-ADRENERGIC-ANTICHOLINERGIC-GLUCOCORT, INHALED' # 2 Moderate
                ]

Mild_Med_list=[ 'GLUCOCORTICOIDS, ORALLY INHALED', # 1
                'GENERAL BRONCHODILATOR AGENTS',  #1 
                'BETA-ADRENERGIC AGENTS', #1
                'BETA-ADRENERGIC AGENTS, INHALED, SHORT ACTING',#1
                'ANTICHOLINERGICS, ORALLY INHALED SHORT ACTING',#1
                'BETA-ADRENERGIC AGENTS, INHALED, ULTRA-LONG ACTING', #1 
                'BETA-ADRENERGIC AGENTS, ORALLY INHALED,LONG ACTING', #1 
                'ANTICHOLINERGICS, ORALLY INHALED LONG ACTING',#  1 
                ]

Soft_Med_list=[
                '5-LIPOXYGENASE INHIBITORS',# X Mild  0.5 
                'LEUKOTRIENE RECEPTOR ANTAGONISTS', # X mild 0.5 
                'MAST CELL STABILIZERS, ORALLY INHALED', #0.25 
                'PHOSPHODIESTERASE-4 (PDE4) INHIBITORS' # 0.5
                ]      

def Medic_Severe(cell):
    if any(val in str(cell) for val in Severe_Med_list) == True:
        return 3.0
    elif any(val in str(cell) for val in Moderate_Med_list) == True:
        return 2.0
    elif any(val in str(cell) for val in Mild_Med_list) == True:
        return 1.0
    elif any(val in str(cell) for val in Soft_Med_list) == True:
        return 0.5
    else:
        return 0.0 
    
# Create Severity columns and note severity level of medicines
df_MEDICATIONS['MedicationSeverity']=df_MEDICATIONS['PHARMACEUTICALCLASS'].apply(Medic_Severe)

paragraph = document.add_paragraph('The patient data reported in this section is the patients that has been prescribed oral stereoids or biological medications')
paragraph.add_run(' two new variables addes to the data, medication severity is an indicator of the patients use  medications classified as class 3 which includes oral stereoid and biological medictions')
paragraph.add_run(' the second variable named as patientseverity indicates whether the patient has been prescribed two or more doses of oral stereoids within a window if  6 months')



In [ ]:
#Filter out all medications  and remove medications with severity 0,  these are medications that are not relevant to Asthma treatment
df_MEDICATIONS = df_MEDICATIONS[df_MEDICATIONS.MedicationSeverity != 0]
# the medication database now contains only medications prescribed for Asthma

In [ ]:
#Determine the patient list who has been prescribed an orally inhaled corticosteroid -- 
#Severe asthma will be determined based on the medications precribed
OrallyInhaledCorticoSteroid = ['BETA-ADRENERGIC AND GLUCOCORTICOID COMBO, INHALED','GLUCOCORTICOIDS, ORALLY INHALED']
df_MEDICATIONSOrallyInhaled = df_MEDICATIONS[df_MEDICATIONS['PHARMACEUTICALCLASS'].isin(OrallyInhaledCorticoSteroid)]
PatientsOralInhal = df_MEDICATIONSOrallyInhaled.PATIENTHASHMRN.unique()
#Patient list who has been prescribed an orally inhaled corticosteroid has been determined

# Assign the Severe medications to dataframe 
# Patients that were prescribed Orally Inhaled Corticosteroids  
#df_Severe_med = df_MEDICATIONS[df_MEDICATIONS['PATIENTHASHMRN'].isin(PatientsOralInhal)]

#df_Severe_med=df_Severe_med[df_Severe_med['PHARMACEUTICALCLASS'].isin(Severe_Med_list)]

#df_Severe_med now contains only the patient data that has been prescribed an orally inhaled cortiocosteroid and 
# an oral corticosteroid or biological medication 


# df_Severe_med['PatientSeverity']=0.0


# df_Severe_med.sort_values(by=['PATIENTHASHMRN', 'DATE_DIF'], inplace=True)

# p1 = document.add_paragraph(" Patient severity is decided based on the  number of oral steroid prescritions within 6 months.  if the patient received two or more oral stereoid prescriptons during a 6 month")
# p1.add_run("period and at least 10 days apart between steroid prescriptions, the patient is classified as severe, otherwise the patient is classified as not severe based on oral steroid episodes")

# p2 = document.add_paragraph("Medication Severity is decided based on the type of the medications.  All oral and injectable stereoids and biologicals are classified as severe medications")


# # Function to determine if a patient is a SeverePatient
# def check_severity(severity_1_dates):
#     retVal = 0
#     lastoccurance = {}
#     eventin180Days = False

#     for i in range(len(severity_1_dates) - 1):
#         datedifference = severity_1_dates.iloc[i + 1]- severity_1_dates.iloc[i]
#         if 0<datedifference<=180 :
#             if 'OCS' in lastoccurance:
#                 last_date_diff = lastoccurance['OCS']
#                 time_diff = severity_1_dates.iloc[i + 1] - last_date_diff
#                 if time_diff < 180 :
#                     lastoccurance['OCS'] = severity_1_dates.iloc[i + 1]
#                     eventin180Days = True
#                     retVal = retVal +1
#             else:
#                 lastoccurance['OCS'] = severity_1_dates.iloc[i + 1]

#         #else:
#             #lastoccurance = {}
#             #eventin180Days = False


#     if eventin180Days:
#         #print("Event A occurs more than once within 180 days", retVal)
#         #retVal = 4.0
#         return retVal
#     else:
#         #print("Event A does not occur more than once within 180 days")
#         retVal = 0

#     return retVal

# for mrn in df_Severe_med['PATIENTHASHMRN'].unique():
#     # Extract the relevant rows for this MRN
#     group = df_Severe_med[df_Severe_med['PATIENTHASHMRN'] == mrn]

#     # Filter for severity 3 medications and sort
#     #severity_1_dates = group[group['MedicationSeverity'] == 3.0]['DATE_DIF'].sort_values()
#     severity_1_dates = group['DATE_DIF'].sort_values()


#     # Check for severity and update the DataFrame
#     severity_status = check_severity(severity_1_dates)

#     #print(severity_status)
#     df_Severe_med.loc[df_Severe_med['PATIENTHASHMRN'] == mrn, 'PatientSeverity'] = severity_status

#     # Set patient Severity to a binary value of 0 or 1 

# df_Severe_med.loc[df_Severe_med['PatientSeverity'] == 1.0, 'PatientSeverity'] = 0.0
# #df_Severe_med.loc[df_Severe_med['PatientSeverity'] == 4.0, 'PatientSeverity'] = 1.0
# df_Severe_med['PatientSeverity'] = df_Severe_med['PatientSeverity'].fillna(0)

In [ ]:
df_Severe_med.PATIENTHASHMRN.nunique()

In [ ]:
df_MEDICATIONS.PATIENTHASHMRN.nunique()

In [ ]:
df_MEDICATIONSOrallyInhaled.PATIENTHASHMRN.nunique()

In [ ]:
Counter(df_Severe_med['PatientSeverity'])

In [ ]:
severeCandidatePatients = df_Severe_med[df_Severe_med['PatientSeverity'] >= 0 ].PATIENTHASHMRN.unique()
severePatientList  = df_Severe_med[df_Severe_med['PatientSeverity'] >= 1 ].PATIENTHASHMRN.unique()

# Specify the CSV file name
file_name = 'uncontrolled.csv'

# Open the file in write mode ('w') and create a csv.writer object
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write each item in the list as a separate row
    for item in severePatientList:
        writer.writerow([item])  # Note the item is wrapped in a list
        
        

# Open the file in write mode ('w') and create a csv.writer object
with open('severeCandidate.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write each item in the list as a separate row
    for item in severeCandidatePatients:
        writer.writerow([item])  # Note the item is wrapped in a list


In [ ]:
#add a new veriable to separate glucocortisoids from biological medications 
paragraph = document.add_paragraph(" We  added a new variable named as medication class to separate glucocortisoids from biological medications")
                                   
df_Severe_med['medicationClass']='GLUCOCORTICOIDS'
df_Severe_med.loc[df_Severe_med['PHARMACEUTICALCLASS'] != 'GLUCOCORTICOIDS', 'medicationClass'] = 'Biological'


In [ ]:
#drop the same medication to teh same patient on the same day
df_Severe_med = df_Severe_med.drop_duplicates(subset=['PATIENTHASHMRN', 'DATE_DIF','MEDNAME'])

#some patients have the same biological medication ordered multiple times. 

# Filter for rows where medicationClass is 'biological'
biological_df = df_Severe_med[df_Severe_med['medicationClass'] == 'Biological']
df_without_biological = df_Severe_med[df_Severe_med['medicationClass'] != 'Biological']

#Drop multiple records of glucocortisoid medications
df_without_biological = df_without_biological.drop_duplicates(subset=['PATIENTHASHMRN'])


# Drop duplicates based on PATIENTHASHMRN and MEDNAME of the biological medication
# keep='first' will keep the first occurrence of the duplicate records

# Sort the DataFrame
sorted_biological_df = biological_df.sort_values(by=['PATIENTHASHMRN', 'MEDNAME', 'DATE_DIF'], ascending=True)

# Drop duplicates based on PATIENTHASHMRN and MEDNAME, keeping the first occurrence
unique_sorted_biological_df = sorted_biological_df.drop_duplicates(subset=['PATIENTHASHMRN', 'MEDNAME'], keep='first')

#combine the two dataframes contaning 
df_Severe_med = pd.concat([unique_sorted_biological_df,df_without_biological])
df_Severe_med = df_Severe_med.sort_values(by=['PATIENTHASHMRN', 'DATE_DIF'], ascending=True)

# unique_sorted_biological_df now contains unique records for patients with a medicationClass of 'biological'
# based on PATIENTHASHMRN and MEDNAME

In [ ]:
df_MEDICATIONS.columns

In [ ]:
Counter(df_Severe_med['medicationClass'])

In [ ]:
# Merge Severe Medications and Demographics
df_med_demo_finan=pd.merge(df_MEDICATIONS,df_FINDEMOG,on="PATIENTHASHMRN",how='left')
df_Severemed_demo_finan=pd.merge(df_Severe_med,df_FINDEMOG,on="PATIENTHASHMRN",how='left')


In [ ]:
#df_Severemed_demo_finan['SevereMedicationClass'] = df_Severemed_demo_finan['PHARMACEUTICALCLASS'].apply(lambda x: 'GLUCOCORTICOIDS' if x == 'GLUCOCORTICOIDS' else 'Biological')
df_Severemed_demo_finan['SevereMedicationClass'] = df_Severemed_demo_finan['PHARMACEUTICALCLASS'].apply(lambda x: 0 if x == 'GLUCOCORTICOIDS' else 1)

# Generate a list of column names that start with 'unknown'
columns_to_drop = [col for col in df_Severemed_demo_finan.columns if col.startswith('Unnamed')]
# Drop these columns from the DataFrame
df_Severemed_demo_finan = df_Severemed_demo_finan.drop(columns=columns_to_drop,errors='ignore')
df_med_demo_finan = df_med_demo_finan.drop(columns=columns_to_drop,errors='ignore')

df_SevereMedDemoFinanEncounter = pd.merge(df_Severemed_demo_finan, df_ENCOUNTERS,on=['PATIENTHASHMRN','DATE_DIF'])
df_MedDemoFinanEncounter = pd.merge(df_med_demo_finan, df_ENCOUNTERS,on=['PATIENTHASHMRN','DATE_DIF'])


In [ ]:
Counter(df_SevereMedDemoFinanEncounter['PatientSeverity'])

In [ ]:
df_SevereMedDemoFinanEncounter.columns

In [ ]:
df_Severemed_demo_finan = df_Severemed_demo_finan.drop_duplicates(subset=['PATIENTHASHMRN', 'DATE_DIF','MEDNAME'])

Counter(df_Severemed_demo_finan['RACE'])

In [ ]:
nan_df = df_SevereMedDemoFinanEncounter[df_SevereMedDemoFinanEncounter['ETHNICITY'].isna()]
nan_df=nan_df.drop_duplicates('PATIENTHASHMRN')


In [ ]:
demoList = df_DEMOGRAPHICS.PATIENTHASHMRN
demoList

In [ ]:
df_severeBiological =df_Severemed_demo_finan[df_Severemed_demo_finan['SevereMedicationClass'] ==1]
df_severeGlucocirtisoids =df_Severemed_demo_finan[df_Severemed_demo_finan['SevereMedicationClass'] ==0]

df_severeBiological = df_severeBiological.drop_duplicates(subset=['PATIENTHASHMRN','MEDNAME'])
df_severeBiologicalUnique = df_severeBiological.drop_duplicates(subset=['PATIENTHASHMRN']) # This is for demographics purposes
severeBiologicalPatientMRN = df_severeBiological['PATIENTHASHMRN'].unique()
# Some Patients are given more than one Biological Medication 


In [ ]:
print('Glucocortisoids :',Counter(df_severeGlucocirtisoids['PatientSeverity']))
#df_severeBiological = df_severeBiological.drop_duplicates(subset=['PATIENTHASHMRN'])
print('Biologicals :', Counter(df_severeBiological['PatientSeverity']))


In [ ]:
visits_count = df_severeBiological.groupby(['PATIENTHASHMRN','GENERICNAME']).size().reset_index(name='Visits')
pivot_df = visits_count.pivot(index='PATIENTHASHMRN', columns='GENERICNAME', values='Visits')

pivot_df = pivot_df.fillna(0)


In [ ]:
hasta =df_MEDICATIONS[df_MEDICATIONS['PATIENTHASHMRN'].str.contains('028e0f500cd727359d6e758a75d3f84bc3ef663ebfe34f')]

In [ ]:
pivot_df.to_csv('biologicalsused.csv')

In [ ]:
# Identifying patients with both medication severity using both glucocortisoids and biological medications
patients_with_both_severities = df_SevereMedDemoFinanEncounter.groupby('PATIENTHASHMRN')['SevereMedicationClass'].nunique()

patients_with_both_severities = patients_with_both_severities[patients_with_both_severities == 2].index

# Filtering out rows where severity is 0 -- removing patients with glucocortisoids and biologicals 

df_SevereMedDemoFinanEncounter#[~((df_SevereMedDemoFinanEncounter['PATIENTHASHMRN'].isin(patients_with_both_severities)) & (df_SevereMedDemoFinanEncounter['SevereMedicationClass'] == 0))]


#Severe medications for patients with Asthma 
df_medicationsWithAsthmaDiagnoses = pd.merge(df_SevereMedDemoFinanEncounter, df_DIAGASTHMA, on=['PATIENTHASHMRN','DATE_DIF'], how='left')
#Mediations with asthma 
df_MedDemoFinanEncounterDiag = pd.merge(df_MedDemoFinanEncounter, df_DIAGASTHMA, on=['PATIENTHASHMRN','DATE_DIF'], how='left')
# Drop duplicates to reduce class imbalance the filtered DataFrame 
#df_filtered = df_filtered.drop_duplicates(subset='PATIENTHASHMRN', keep='first')


In [ ]:
len(df_medicationsWithAsthmaDiagnoses)

In [ ]:
df_medicationsWithAsthmaDiagnoses


In [ ]:
PediatricPatients = df_SevereMedDemoFinanEncounter[df_SevereMedDemoFinanEncounter.DEPTSPECIALTY == 'Pediatrics']

In [ ]:
PediatricPatients

In [ ]:
DepartmentRelevant = [
    'Family Medicine',
    'Internal Medicine',
    #'Pediatrics',
    'Allergy and Immunology',
    'Pulmonology',
    'Urgent Care',
    'Emergency Medicine',
    #'Sleep Medicine',
    'Urgent Care']

Departments_Exluded = [
    '*Not Applicable','Radiology','Orthopaedic Surgery', '*Unspecified',
    'Admitting/Central Scheduling', 'Urogynecology',
    'Rheumatology','Physical Therapy', 'General Surgery', 
    'Obstetrics','Executive Health', 'Interventional Radiology',
    'Occupational Medicine', 'Pre-Admission Testing','Bariatrics', 
    'Vascular Surgery', 'Urology',
    'Perinatology', 'SHPS', 'Radiation Oncology', 'Nephrology',
    'Genetics', 'Pharmacy', 'Optometry', 'Gynecologic Oncology',
    'Audiology', 'Transplant', 'Home Health Services',
    'Home Based Care', 'Palliative Medicine',
    'Wound Care', 'Intensive Care', 'Endocrinology',
    'Dermatology',
    'Pediatrics',
    'Sleep Medicine'
]

#UNCOMMENT to REDUCE DEPTSPECIALITY
df_SevereMedDemoFinanEncounterAll = df_SevereMedDemoFinanEncounter
df_SevereMedDemoFinanEncounter = df_SevereMedDemoFinanEncounter[~df_SevereMedDemoFinanEncounter['DEPTSPECIALTY'].isin(Departments_Exluded)]

#df_medicationsWithAsthmaDiagnoses = df_medicationsWithAsthmaDiagnoses[~df_medicationsWithAsthmaDiagnoses['DEPTSPECIALTY'].isin(ç)]
df_SevereMedDemoFinanEncounter.loc[~df_SevereMedDemoFinanEncounter['DEPTSPECIALTY'].isin(DepartmentRelevant), 'DEPTSPECIALTY'] = "Other"

#df_medicationsWithAsthmaDiagnoses = df_medicationsWithAsthmaDiagnoses.drop_duplicates(subset=['PATIENTHASHMRN', 'DATE_DIF','MEDNAME'])
#df_SevereMedDemoFinanEncounter = df_SevereMedDemoFinanEncounter[df_SevereMedDemoFinanEncounter['DEPTSPECIALTY'].isin(DepartmentRelevant)]

#Drop medications on the same day 
df_SevereMedDemoFinanEncounter = df_SevereMedDemoFinanEncounter.drop_duplicates(subset=['PATIENTHASHMRN', 'DATE_DIF','MEDNAME'])
duplicates = df_SevereMedDemoFinanEncounter.duplicated(subset=['PATIENTHASHMRN', 'DATE_DIF', 'MEDNAME'], keep=False)
df_duplicates = df_SevereMedDemoFinanEncounter[duplicates]
df_duplicates.sort_values(by=['PATIENTHASHMRN','DATE_DIF', 'MEDNAME'], inplace=True)



In [ ]:
df_MedDemoFinanEncounter = df_MedDemoFinanEncounter[~df_MedDemoFinanEncounter['DEPTSPECIALTY'].isin(Departments_Exluded)]
df_MedDemoFinanEncounter.loc[~df_MedDemoFinanEncounter['DEPTSPECIALTY'].isin(DepartmentRelevant), 'DEPTSPECIALTY'] = "Other"
df_MedDemoFinanEncounter = df_MedDemoFinanEncounter.drop_duplicates(subset=['PATIENTHASHMRN', 'DATE_DIF','MEDNAME'])




In [ ]:
#YUSUF
Counter(df_MedDemoFinanEncounter.PHARMACEUTICALCLASS)

In [ ]:
Counter(df_MedDemoFinanEncounter['DEPTSPECIALTY'])

In [ ]:
df_SevereMedDemoFinanEncounter.to_csv('medicationswithSevereAsthmaDiag.csv')


In [ ]:
df_duplicates.to_csv("duplicateSevereMedications.csv")

In [ ]:
Biological_counts = df_severeBiologicalUnique.groupby(['RACE']).size().reset_index(name='Counts')

# Step 4: Calculate total emergency department visits
total_biological_counts = Biological_counts['Counts'].sum()

# Step 5: Calculate percentages
Biological_counts['Percentage'] = (Biological_counts['Counts'] / total_biological_counts) * 100


df_Severemed_demo_finanUnique = df_Severemed_demo_finan.drop_duplicates(subset=['PATIENTHASHMRN'])

# Step 1 and 2: Group by Race and count entries
visit_counts = df_Severemed_demo_finanUnique.groupby('RACE').size().reset_index(name='Count')

# Step 3: Calculate total entries in the dataset
total_entries = visit_counts['Count'].sum()

# Step 4: Calculate percentages
visit_counts['Percentage'] = (visit_counts['Count'] / total_entries) * 100

doctable(Biological_counts, "Ethnicity stats of patients who has been prescribed Biological Medications",['index','Ethnicity','Count','Percentage'])
doctable(visit_counts, "Ethnicity stats of Patients classified as Severe patients",['index','Ethnicity','Count','Percentage'])


print(Biological_counts)
print(visit_counts)

In [ ]:
Biological_counts = df_severeBiologicalUnique.groupby(['RACE']).size().reset_index(name='Counts')

# Step 4: Calculate total emergency department visits
total_biological_counts = Biological_counts['Counts'].sum()

# Step 5: Calculate percentages
Biological_counts['Percentage'] = (Biological_counts['Counts'] / total_biological_counts) * 100


df_Severemed_demo_finanUnique = df_Severemed_demo_finan.drop_duplicates(subset=['PATIENTHASHMRN'])

# Step 1 and 2: Group by Race and count entries
visit_counts = df_Severemed_demo_finanUnique.groupby('RACE').size().reset_index(name='Count')

# Step 3: Calculate total entries in the dataset
total_entries = visit_counts['Count'].sum()

# Step 4: Calculate percentages
visit_counts['Percentage'] = (visit_counts['Count'] / total_entries) * 100

doctable(Biological_counts, "Race stats of patients who has been prescribed Biological Medications",['index','Race','Count','Percentage'])
doctable(visit_counts, "Race stats of Patients classified as Severe patients",['index','Race','Count','Percentage'])


print(Biological_counts)
print(visit_counts)

In [ ]:
Biological_counts = df_severeBiologicalUnique.groupby(['INCOME_CATEGORY']).size().reset_index(name='Counts')

# Step 4: Calculate total emergency department visits
total_biological_counts = Biological_counts['Counts'].sum()

# Step 5: Calculate percentages
Biological_counts['Percentage'] = (Biological_counts['Counts'] / total_biological_counts) * 100


df_Severemed_demo_finanUnique = df_Severemed_demo_finan.drop_duplicates(subset=['PATIENTHASHMRN'])

# Step 1 and 2: Group by Race and count entries
visit_counts = df_Severemed_demo_finanUnique.groupby('INCOME_CATEGORY').size().reset_index(name='Count')

# Step 3: Calculate total entries in the dataset
total_entries = visit_counts['Count'].sum()

# Step 4: Calculate percentages
visit_counts['Percentage'] = (visit_counts['Count'] / total_entries) * 100

doctable(Biological_counts, "Race stats of patients who has been prescribed Biological Medications",['index','Income Category','Count','Percentage'])
doctable(visit_counts, "Race stats of Patients classified as Severe patients",['index','Income Category','Count','Percentage'])


print(Biological_counts)
print(visit_counts)

In [ ]:
df_severeBiologicalUnique

In [ ]:
Counter(df_SevereMedDemoFinanEncounter['SevereMedicationClass'])

In [ ]:
#test if medicationClass and RACE has a relationship
from scipy.stats import chi2_contingency

# Assuming df is your DataFrame containing the data
contingency_table = pd.crosstab(df_SevereMedDemoFinanEncounter['PatientSeverity'], df_SevereMedDemoFinanEncounter['medicationClass'])
print(contingency_table)
contingency_table_df = contingency_table.reset_index()
contingency_table_df.columns.name = None

# Performing chi-square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table_df)

# Printing the results

paragraph = document.add_paragraph('Test if Patient severity and medication class  has a relationship ')
paragraph.add_run('\n')
paragraph.add_run('We run chi  square test  ')
doctable(contingency_table, "Contingency Table_df",['PatientSeverity','Biological','GLUCOCORTICOIDS'])
print(contingency_table)
paragraph.add_run("Chi-square statistic:")
paragraph.add_run(str(chi2))
paragraph.add_run("P-value:")
paragraph.add_run(str(p_value))


In [ ]:
# Calculate total count
total_count = np.sum(contingency_table)

# Calculate percentage for each cell
cell_percentages = (contingency_table / total_count) * 100


print("Cell Percentages:\n", cell_percentages)


In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_SevereMedDemoFinanEncounter['PatientSeverity'], df_SevereMedDemoFinanEncounter['SevereMedicationClass'])

# Perform the Chi-Squared Test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table)
contingency_table_df = contingency_table.reset_index()
contingency_table_df.columns.name = None



paragraph = document.add_paragraph('Test if Patient severity and Severe medication class  has a relationship ')
paragraph.add_run('\n')
paragraph.add_run('We run chi  square test  ')
doctable(contingency_table, "Contingency Table_df",['PatientSeverity','Non-Severe Medication','Severe Medication'])
paragraph.add_run("Chi-square statistic:")
paragraph.add_run(str(chi2))
paragraph.add_run("P-value:")
paragraph.add_run(str(p_value))

# Interpret the results
if p_value < 0.05:
    paragraph.add_run("\nSignificant association between patient severity and Severe Medication Class.")
else:
    paragraph.add_run("\nNo significant association between patient severity and Severe Medication Class.")

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_SevereMedDemoFinanEncounter['PatientSeverity'], df_SevereMedDemoFinanEncounter['RACE'])

# Perform the Chi-Squared Test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table)

contingency_table_df = contingency_table.reset_index()
contingency_table_df.columns.name = None


paragraph = document.add_paragraph('Test if Patient severity and Ethnicity   has a relationship ')
paragraph.add_run('\n')
paragraph.add_run('We run chi  square test  ')
doctable(contingency_table_df, "Contingency Table_df",['index','PatientSeverity','*Unspecified', 'Decline to Answer','Hispanic or Latino or Spanish Origin','Not Hispanic, Latino/a, or Spanish origin', 'Unknown'])
paragraph.add_run("Chi-square statistic:")
paragraph.add_run(str(chi2))
paragraph.add_run("P-value:")
paragraph.add_run(str(p_value))
paragraph.add_run("DOF:")
paragraph.add_run(str(dof))

# Interpret the results
if p_value < 0.05:
    paragraph.add_run("\nSignificant association between patient severity and Ethnicity")
else:
    paragraph.add_run("\nNo significant association between patient severity and Ethnicity")

In [ ]:
# Calculate total count
total_count = np.sum(contingency_table)

# Calculate percentage for each cell
cell_percentages = (contingency_table / total_count) * 100

print("Cell Percentages:\n", cell_percentages)


In [ ]:
contingency_table_df.columns

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_SevereMedDemoFinanEncounter['PatientSeverity'], df_SevereMedDemoFinanEncounter['INCOME_CATEGORY'])

# Perform the Chi-Squared Test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table)

contingency_table_df = contingency_table.reset_index()
contingency_table_df.columns.name = None


paragraph = document.add_paragraph('Test if Patient severity and Income category   has a relationship ')
paragraph.add_run('\n')
paragraph.add_run('We run chi  square test  ')
doctable(contingency_table_df, "Contingency Table_df",['index','PatientSeverity','Very low','High', 'Low', 'Median', 'Unknown','Very High'])
paragraph.add_run("Chi-square statistic:")
paragraph.add_run(str(chi2))
paragraph.add_run("P-value:")
paragraph.add_run(str(p_value))
paragraph.add_run("DOF:")
paragraph.add_run(str(dof))

# Interpret the results
if p_value < 0.05:
    paragraph.add_run("\nSignificant association between patient severity and Income Category")
else:
    paragraph.add_run("\nNo significant association between patient severity and Income Category")

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_SevereMedDemoFinanEncounter['medicationClass'], df_SevereMedDemoFinanEncounter['INCOME_CATEGORY'])

# Perform the Chi-Squared Test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table)
contingency_table_df = contingency_table.reset_index()
contingency_table_df.columns.name = None


paragraph = document.add_paragraph('Test if Medication Class and Income category   has a relationship ')
paragraph.add_run('\n')
paragraph.add_run('We run chi  square test  ')
doctable(contingency_table_df, "Contingency Table_df",['index','Medication Class', ' Very Low', 'High', 'Low', 'Median', 'Unknown',
       'Very High'])
paragraph.add_run("\nChi-square statistic:")
paragraph.add_run(str(chi2))
paragraph.add_run("\nP-value:")
paragraph.add_run(str(p_value))
paragraph.add_run("\nDegrees of Freedom:")
paragraph.add_run(str(dof))


# Interpret the results
if p_value < 0.05:
    paragraph.add_run("\nSignificant association between Medication Class and Income Category")
else:
    paragraph.add_run("\nNo significant association between Medication Class and Income Category")

In [ ]:
contingency_table.columns

In [ ]:
# Calculate total count
total_count = np.sum(contingency_table)

# Calculate percentage for each cell
cell_percentages = (contingency_table / total_count) * 100

print("Cell Percentages:\n", cell_percentages)


In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_SevereMedDemoFinanEncounter['medicationClass'], df_SevereMedDemoFinanEncounter['HHSREGION'])

# Perform the Chi-Squared Test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table)

contingency_table_df = contingency_table.reset_index()
contingency_table_df.columns.name = None


paragraph = document.add_paragraph('Test if Medication Class and HHSREGION has a relationship ')
paragraph.add_run('\n')
paragraph.add_run('We run chi  square test  ')
doctable(contingency_table_df, "Contingency Table_df",['index','medicationClass','CENTRAL', 'EAST', 'NORTH CENTRAL', 'NORTH COASTAL', 'NORTH INLAND','SOUTH','UNKNOWN'])
paragraph.add_run("\nChi-square statistic:")
paragraph.add_run(str(chi2))
paragraph.add_run("\nP-value:")
paragraph.add_run(str(p_value))
paragraph.add_run("\nDegrees of Freedom:")
paragraph.add_run(str(dof))


# Interpret the results
if p_value < 0.05:
    paragraph.add_run("\nSignificant association between Medication Class and HHSREGION")
else:
    paragraph.add_run("\nNo significant association between Medication Class and HHSREGION")

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_SevereMedDemoFinanEncounter['medicationClass'], df_SevereMedDemoFinanEncounter['DEPTSPECIALTY'])

# Perform the Chi-Squared Test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table)

contingency_table_df = contingency_table.reset_index()
contingency_table_df.columns.name = None


paragraph = document.add_paragraph('Test if Medication Class and DEPTSPECIALTY has a relationship ')
paragraph.add_run('\n')
paragraph.add_run('We run chi  square test  ')
doctable(contingency_table_df, "Contingency Table_df",['index','medicationClass','Allergy and Immunology','Dermatology','Emergency Medicine','Family Medicine','Internal Medicine','Other','Pediatrics','Pulmonology','Sleep Medicine','Urgent Care'])
paragraph.add_run("\nChi-square statistic:")
paragraph.add_run(str(chi2))
paragraph.add_run("\nP-value:")
paragraph.add_run(str(p_value))
paragraph.add_run("\nDegrees of Freedom:")
paragraph.add_run(str(dof))


# Interpret the results
if p_value < 0.05:
    paragraph.add_run("\nSignificant association between Medication Class and DEPTSPECIALTY")
else:
    paragraph.add_run("\nNo significant association between Medication Class and DEPTSPECIALTY")

In [ ]:
contingency_table_df

In [ ]:
# Group by 'PATIENTHASHMRN', 'DATE_DIF', and 'ICD10CODE'
# We would like to group the procedures related to an individual diagnosis on a particular day
#grouped = df_SevereMedDemoFinanEncounter.groupby(['PATIENTHASHMRN', 'DATE_DIF'])
grouped = df_MedDemoFinanEncounter.groupby(['PATIENTHASHMRN', 'DATE_DIF'])

# Aggregate procedure codes into a list for each diagnoses 
aggregated = grouped.agg(
                         MedList =('MEDNAME',  lambda x: list(x)),
                         MedCount=('MEDNAME', 'count')).reset_index()

aggregated.columns = ['PATIENTHASHMRN', 'DATE_DIF', 'MedList', 'MedCount']

#df_DIAGMED = pd.merge(df_SevereMedDemoFinanEncounter, aggregated, on=['PATIENTHASHMRN', 'DATE_DIF'], how='left')
df_DIAGMED = pd.merge(df_MedDemoFinanEncounter, aggregated, on=['PATIENTHASHMRN', 'DATE_DIF'], how='left')

# df_DIAGPROC now contains new columns with the list of procedure codes, Procedure names and Procedure types for each patient

In [ ]:
dd= df_MedDemoFinanEncounter[df_MedDemoFinanEncounter.DATE_DIF == 28007]


In [ ]:
dd.to_excel('review.xlsx')

In [ ]:
medCostData = pd.read_excel('CostTables/medicationCosts.xlsx')


In [ ]:
#medications = df_SevereMedDemoFinanEncounterFiltered.drop_duplicates(subset=['MEDNAME'])
#medicationList = medications['MEDNAME']
#medicationList.to_csv('medicationCosts.csv')

In [ ]:
medCostData['MEDNAME'] = medCostData['MEDNAME'].astype(str)
MedCostDictionary = dict(zip(medCostData['MEDNAME'], medCostData['RetailCost']))


In [ ]:
df_DIAGMED.columns

In [ ]:
df_DIAGMED.columns

In [ ]:
import joypy 

# Function to calculate cost for each encounter
def calculate_cost(medications):
    return sum(MedCostDictionary[med] for med in medications if med in MedCostDictionary)

# Apply the function to each row and create a new column
df_DIAGMED['Cost'] = df_DIAGMED['MedList'].apply(calculate_cost)

#Display the updated dataframe
#print(df_DIAGMED)


from scipy import stats
# Group by ethnicity and aggregate procedure counts
quartiles = df_DIAGMED.groupby('DEPTSPECIALTY')['Cost'].quantile([0.25, 0.5, 0.75]).unstack()
mean = df_DIAGMED.groupby('DEPTSPECIALTY')['Cost'].mean()
maximum = df_DIAGMED.groupby('DEPTSPECIALTY')['Cost'].max()

count = df_DIAGMED['DEPTSPECIALTY'].value_counts()


stats = pd.concat([quartiles, mean, maximum, count], axis=1)
stats.columns = ['25th Quartile', 'Median', '75th Quartile', 'Mean', 'Maximum', 'Count']


print(stats)

doctable(stats,"Medication Cost  Statistics for Each Speciality  in the Database",['District','25th Quartile', 'Median', '75th Quartile', 'Mean', 'Maximum', 'Number of Patients'])


grouped_data = df_DIAGMED.groupby('DEPTSPECIALTY')['Cost'].agg(['mean', 'median', 'sum', 'count', 'std'])

# Visualization: Box Plot
plt.figure(figsize=(10, 6))
df_DIAGMED['LogCost'] = np.log1p(df_DIAGMED['Cost'])  # Use log1p to avoid log(0) issues
sns.violinplot(x='DEPTSPECIALTY', y='LogCost', data = df_DIAGMED, palette='Blues')
#sns.boxplot(x='DEPTSPECIALTY', y='LogCost', data=df_DIAGMED)
#joypy.joyplot(df_DIAGMED, by='DEPTSPECIALTY', column='LogCost')

#plt.title('Department Speciality versus Cost of Treatment')
plt.xticks(rotation=45, ha='right')
plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
paragraph = document.add_paragraph("Figure XX:Department Speciality versus Estimated Cost of Treatment ")
plt.show()

print(grouped_data)



In [ ]:
# Create a violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(x='DEPTSPECIALTY', y='LogCost', data=df_DIAGMED, inner=None, palette='muted')

# Calculate and plot percentiles for each category
categories = df_DIAGMED['DEPTSPECIALTY'].unique()
for category in categories:
    # Filter the data for the current category
    data = df_DIAGMED.loc[df_DIAGMED['DEPTSPECIALTY'] == category, 'LogCost']
    # Calculate the percentiles
    p25, median, p75 = np.percentile(data, [25, 50, 75])
    # Plot the percentiles on the violin plot
    category_index = np.where(categories == category)[0][0]
    plt.scatter([category_index] * 3, [p25, median, p75], color='black', s=50, marker='o', label='Percentiles' if category_index == 0 else "")

# Add labels and legend
#plt.title('Violin Plot with Percentile Indicators')
plt.xlabel('Department Speciality')
plt.ylabel('Log Cost')
plt.legend(['25th, 50th, 75th Percentiles'])

plt.savefig('temp.png', bbox_inches='tight')

document.add_picture('temp.png', width=Inches(6))
paragraph = document.add_paragraph("Figure XX:Medication Cost Department Speciality versus Estimated Cost of Treatment ")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from docx import Document
from docx.shared import Inches
from matplotlib.ticker import FuncFormatter


# Initialize document
document = Document()
df_DIAGMED = df_DIAGMED[df_DIAGMED['Cost'] != 0]
df_DIAGMED = df_DIAGMED[df_DIAGMED['DEPTSPECIALTY'] != 'Other']

# Create a boxplot
plt.figure(figsize=(7, 5))
ax = sns.boxplot(
    x='DEPTSPECIALTY',
    y='Cost',
    data=df_DIAGMED,
    palette='muted',
    showfliers=False  # Remove outlier points for clarity
)

# Set symlog scale for the y-axis
all_costs = df_DIAGPROC_FINDEMOG_ENCFiltered['Cost']
linthresh = np.percentile(all_costs, 90)  # Automatically determine the linthresh
plt.yscale('symlog', linthresh=linthresh)

# Force y-axis to start at zero
plt.ylim(bottom=0)

# Set y-axis ticks to integers
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x)}'))


# Extract categories
categories = df_DIAGMED['DEPTSPECIALTY'].unique()

# Calculate medians (50th percentile) and Q3 (75th percentile) for each category
medians = df_DIAGMED.groupby('DEPTSPECIALTY')['Cost'].median().reindex(categories).values
q3_values = df_DIAGMED.groupby('DEPTSPECIALTY')['Cost'].quantile(0.75).reindex(categories).values

# Debugging: Print medians and Q3
print("Extracted Medians (50th Percentile):", medians)
print("Extracted Q3 (75th Percentiles):", q3_values)

# Calculate the relative costs based on the minimum median
min_median = max(min(medians), 1e-3)  # Avoid division by zero
relative_costs_median = [median / min_median for median in medians]
relative_costs_q3 = [q3 / min_median for q3 in q3_values]

# Debugging: Print relative costs
print("Relative Costs (50th Percentile):", relative_costs_median)
print("Relative Costs (75th Percentile):", relative_costs_q3)

# Annotate each box with relative costs
for i, (median, relative_cost_median, q3, relative_cost_q3) in enumerate(zip(medians, relative_costs_median, q3_values, relative_costs_q3)):
    # Annotate the 50th percentile (median)
    ax.text(
        i, median,
        f'50%: {relative_cost_median:.2f}x Min',
        ha='center', va='bottom', fontsize=10, color='black'
    )
    # Annotate the 75th percentile (Q3)
    ax.text(
        i, q3,
        f'75%: {relative_cost_q3:.2f}x Min',
        ha='center', va='bottom', fontsize=10, color='blue'
    )

# Customize the plot
#plt.title('Boxplot of LogCost by Department Specialty (Relative Cost of 50% and 75%)', fontsize=16)
plt.xlabel('')
plt.ylabel('Cost (SymLog Scale)', fontsize=12)
plt.xticks(rotation=45, fontsize=12)
plt.tick_params(axis='y', which='both', labelsize=12)
#plt.yticks([])  # Removes the yticks

# Add horizontal gridlines
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Save the figure
plt.tight_layout()
plt.savefig('boxplot_relative_costs.png', dpi=300, bbox_inches='tight')

# Add the plot to the Word document
document.add_picture('boxplot_relative_costs.png', width=Inches(7))
paragraph = document.add_paragraph("Figure XX: Boxplot of LogCost by Department Specialty with Relative Cost Annotations (50% and 75%).")

# Save the Word document
document.save('Boxplot_Relative_Costs_Report.docx')

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

df_DIAGMED['Cost'] = df_DIAGMED['Cost'].clip(upper=df_DIAGMED['Cost'].quantile(0.95))  # Cap at the 95th percentile

# Create the ridge plot
joypy.joyplot(df_DIAGMED, by='DEPTSPECIALTY', column='Cost', figsize=(10, 6), colormap=plt.cm.Blues)

# Limit the x-axis range to focus on a reasonable range of cost values
# Customizing the plot
plt.xticks(rotation=45, ha='right')
plt.xlabel('Cost')
plt.ylabel('Department Specialty')
plt.title('Department Specialty versus Cost of Treatment')

# Save and show
plt.savefig('temp.png', bbox_inches='tight')
plt.show()

In [ ]:
document.save("Demographics.docx")

In [ ]:
Counter(df_DIAGMED.PHARMACEUTICALCLASS)

In [ ]:
#CODE VERIFIED UP TO HERE

In [ ]:
df_DIAGMED.loc[df_DIAGMED['MEDNAME'].isin(BIOLOGICALS), 'PHARMACEUTICALCLASS'] = 'BIOLOGICAL'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter for PHARMACEUTICALCLASS values 'Biological' and 'Glucocorticoids'
filtered_df = df_DIAGMED[df_DIAGMED['PHARMACEUTICALCLASS'].isin(['BIOLOGICAL', 'GLUCOCORTICOIDS'])]

# Count the medications for each department specialty and pharmaceutical class
counts = filtered_df.groupby(['DEPTSPECIALTY', 'PHARMACEUTICALCLASS']).size().reset_index(name='Count')

# Calculate the total medications prescribed in each department specialty
total_counts = counts.groupby('DEPTSPECIALTY')['Count'].transform('sum')

# Calculate the percentage of medications prescribed for each class in each department specialty
counts['Percentage'] = (counts['Count'] / total_counts) * 100

# Pivot the table to make it easier for plotting
pivot_counts = counts.pivot(index='DEPTSPECIALTY', columns='PHARMACEUTICALCLASS', values='Percentage').fillna(0)

# Plot the data
pivot_counts.plot(kind='bar', figsize=(12, 6), stacked=True, color=['#1f77b4', '#ff7f0e'])

# Customize the plot
plt.title('Percentage of Medications Prescribed by Pharmaceutical Class', fontsize=16)
plt.xlabel('Department Specialty', fontsize=14)
plt.ylabel('Percentage of Medications (%)', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.legend(title='Pharmaceutical Class', fontsize=12)
plt.tight_layout()

# Save and show the plot
plt.savefig('percentage_medications_prescribed.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter for PHARMACEUTICALCLASS values 'Biological' and 'Glucocorticoids'
filtered_df = df_DIAGMED[df_DIAGMED['PHARMACEUTICALCLASS'].isin(['BIOLOGICAL', 'GLUCOCORTICOIDS'])]

# Count the medications for each department specialty and pharmaceutical class
counts = filtered_df.groupby(['DEPTSPECIALTY', 'PHARMACEUTICALCLASS']).size().reset_index(name='Count')

# Calculate the total medications prescribed in each pharmaceutical class
total_counts = counts.groupby('PHARMACEUTICALCLASS')['Count'].transform('sum')

# Calculate the percentage of medications prescribed for each class in each department specialty
counts['Percentage'] = (counts['Count'] / total_counts) * 100

# Split the data for plotting and sort by percentage
glucocorticoids_data = counts[counts['PHARMACEUTICALCLASS'] == 'GLUCOCORTICOIDS'].sort_values(by='Percentage', ascending=False)
biologicals_data = counts[counts['PHARMACEUTICALCLASS'] == 'BIOLOGICAL'].sort_values(by='Percentage', ascending=False)

# Plot percentage of Glucocorticoids prescribed
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='DEPTSPECIALTY', y='Percentage', data=glucocorticoids_data, palette='Blues_d')

# Add data labels
for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}%', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', fontsize=10)

plt.title('Percentage of Glucocorticoids Prescribed by Department Specialty', fontsize=16)
plt.xlabel('Department Specialty', fontsize=14)
plt.ylabel('Percentage of Medications (%)', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.tight_layout()
plt.savefig('glucocorticoids_percentage.png', dpi=300)
plt.show()

# Plot percentage of Biologicals prescribed
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='DEPTSPECIALTY', y='Percentage', data=biologicals_data, palette='Oranges_d')

# Add data labels
for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}%', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', fontsize=10)

plt.title('Percentage of Biologicals Prescribed by Department Specialty', fontsize=16)
plt.xlabel('Department Specialty', fontsize=14)
plt.ylabel('Percentage of Medications (%)', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.tight_layout()
plt.savefig('biologicals_percentage.png', dpi=300)
plt.show()

In [ ]:
total_counts

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Filter for PHARMACEUTICALCLASS values 'Biological' and 'Glucocorticoids'
filtered_df = df_DIAGMED[df_DIAGMED['PHARMACEUTICALCLASS'].isin(['BIOLOGICAL', 'GLUCOCORTICOIDS'])]

# Count the medications for each department specialty and pharmaceutical class
counts = filtered_df.groupby(['DEPTSPECIALTY', 'PHARMACEUTICALCLASS']).size().reset_index(name='Count')

# Calculate the total medications prescribed in each pharmaceutical class
total_counts = counts.groupby('PHARMACEUTICALCLASS')['Count'].transform('sum')

# Calculate the percentage of medications prescribed for each class in each department specialty
counts['Percentage'] = (counts['Count'] / total_counts) * 100

# Pivot the data to align percentages for side-by-side plotting
pivot_data = counts.pivot(index='DEPTSPECIALTY', columns='PHARMACEUTICALCLASS', values='Percentage').fillna(0)

# Ensure categories are sorted consistently
pivot_data = pivot_data.sort_index()

# Create the plot
categories = pivot_data.index
x = np.arange(len(categories))  # X-axis positions for the bars
width = 0.35  # Width of the bars

fig, ax1 = plt.subplots(figsize=(14, 8))

# Bars for Glucocorticoids on the first axis
bars1 = ax1.bar(x - width/2, pivot_data['GLUCOCORTICOIDS'], width, label='Glucocorticoids', color='green', alpha=0.7)
#ax1.set_xlabel('Department Specialty', fontsize=14)
ax1.set_xlabel('')
ax1.set_ylabel('Percentage of Glucocorticoids Prescriptions (%)', fontsize=14, color='green')
ax1.tick_params(axis='y', labelcolor='green')
ax1.set_xticks(x)
ax1.set_xticklabels(categories, rotation=45, ha='right', fontsize=12)

# Create the second axis
ax2 = ax1.twinx()

# Bars for Biologicals on the second axis
bars2 = ax2.bar(x + width/2, pivot_data['BIOLOGICAL'], width, label='Biologicals', color='blue', alpha=0.7)
ax2.set_ylabel('Percentage of Biological Prescriptions (%)', fontsize=14, color='blue')
ax2.tick_params(axis='y', labelcolor='blue')

# Add legends for both axes
#ax1.legend(loc='upper left', fontsize=12)
#ax2.legend(loc='upper right', fontsize=12)

# Add data labels to Glucocorticoids bars
for bar in bars1:
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{bar.get_height():.1f}%', ha='center', va='bottom', fontsize=10, color='green')

# Add data labels to Biologicals bars
for bar in bars2:
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{bar.get_height():.1f}%', ha='center', va='bottom', fontsize=10, color='blue')

# Add title
#plt.title('Percentage of Glucocorticoids and Biologicals Prescribed by Department Specialty', fontsize=16)

# Adjust layout and save
plt.tight_layout()
plt.savefig('separate_axes_glucocorticoids_biologicals.png', dpi=300)
plt.show()